# Set Client Margin 

In [33]:
import pandas as pd
import numpy as np
import math


#For UPS client freight & accessorial discounts, write "YES"/"NO" for rounding/not rounding to 2 decimal places.
Round_Discount = "NO" 

BarrettDisc_Fsc_FedEx = 0.35 # change Fsc discount to 0 for customer quote
BarrettDisc_Fsc_UPS = 0.2  # change Fsc discount to 0 for customer quote


In [34]:
#Set UPS margin for the client
Margin_on_Min_GRND_UPS = 0.0
Margin_on_Min_AIR_UPS = 0.0

Margin_on_UPS_Parcel_Freight_GRND = 0.0
Margin_on_UPS_Parcel_Freight_AIR = 0.0

Margin_on_Value_Add_GRND_UPS = 0.0
Margin_on_Value_Add_AIR_UPS = 0.0

In [35]:
#Set FedEx margin for the client
Margin_on_Min_GRND_FedEx = 0.0
Margin_on_Min_AIR_FedEx = 0.0

Margin_on_FedEx_Parcel_Freight_GRND = 0.0
Margin_on_FedEx_Parcel_Freight_AIR = 0.0

Margin_on_Value_Add_GRND_FedEx = 0.0
Margin_on_Value_Add_AIR_FedEx = 0.0


In [36]:
#Set DHL, USPS, and MI margin for the client
Margin_on_DHL = 0.0
Margin_on_USPS = 0.0
Margin_on_MI = 0.0

# Carrier Dim Factor 2023

In [37]:
#Dim factor 2023
dim_factor_ups = 280
dim_factor_ups_srpt = 139
dim_factor_dhl = 166
dim_factor_usps = 166
dim_factor_mi = 166
dim_factor_fedex = 300 # 280 for customer, 300 for Barrett
dim_factor_fedex_spst = 999 # 300 if cubic inch >= 3456, no quote when billed weight > 9 lbs, rate sheet has up to 70lbs

# Set the Year based on PLD 
This quote can only apply to 1 year at one time. 

In [38]:
nqd_dhl_date_1 = '2024-01-01'
nqd_dhl_date_2 = '2024-09-30'
nqd_dhl_date_3 = '2024-10-01'
nqd_dhl_date_4 = '2024-12-31'
nqd_dhl_date_1 = pd.to_datetime(nqd_dhl_date_1)
nqd_dhl_date_2 = pd.to_datetime(nqd_dhl_date_2)
nqd_dhl_date_3 = pd.to_datetime(nqd_dhl_date_3)
nqd_dhl_date_4 = pd.to_datetime(nqd_dhl_date_4)

# ======Model Structure======

Source tables:
1. carrierRateCard_2023: published freight rate card (insert newer version once a year)

   1-1. UPSAccessorial_2023: UPS accessorial rate card. (insert newer version once a year)


2. areaSurchargeZipsUs_2023: a table to assign the type of DAS Category (insert newer version once a year)

3. FSC: fuel surcharge rate (insert a new number once a week before automation)

4. zip_Outbound: after decided the origin facility, get zone from destination zip 

5. ZiptoState: get state from destination zip to decide the origin facility
    
6. PLD: for rerate

Structure:

1 - a. Create Freight Rate Card (tbl_PublishedRates)  
1 - b. Create Freight Rate Card (tbl_BarrettRates)  
1 - c. Create Freight Rate Card (tbl_clientRates)

2 - a. Create UPS accessorial rates  
2 - b. Prepare all other rates for rerate

3 - a. Prepare PLD for rerate: Load, Clean, and Format PLD  
3 - b. Rerate Begin  
3 - c. Output and analysis

# Step 1 - a & 1 - b
*rate card: Add UPS published rates, DHL Barrett rates, and USPS Barrett rates. Needs to renew once a year. Can add more carriers later.

In [39]:
# in the back up material section at the end
tbl_BarrettRates = pd.read_csv('C:\\Users\\SLiu\\BD\\model\\RateQuote\\tbl_BarrettRates.csv')
tbl_publishedRates = pd.read_csv('C:\\Users\\SLiu\\BD\\model\\RateQuote\\tbl_publishedRates.csv')
tbl_BarrettRates['ZONE'] = tbl_BarrettRates['ZONE'].astype(str)
tbl_BarrettRates['VERSION'] = tbl_BarrettRates['VERSION'].astype(str)
tbl_publishedRates['ZONE'] = tbl_publishedRates['ZONE'].astype(str)
tbl_publishedRates['VERSION'] = tbl_publishedRates['VERSION'].astype(str)

# Step 1 - c. Create Freight Rate Card (tbl_clientRates)
*Add ups, dhl, and usps clientRates in tbl_publishedRates. Read the margins located at the top of this model. 

In [40]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('Accessorial_2024.xlsx')
UPS_Rates = pd.read_excel(xls, 'UPS_Rates')

In [41]:
#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 6
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 8
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 7
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9 

UPS_Rates['WEIGHT_LOWER'] = UPS_Rates.apply(g, axis=1)
UPS_Rates['WEIGHT_UPPER'] = UPS_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'Next Day Air') | (col['SERVICE_LEVEL'] == 'Next Day Air Saver') | (col['SERVICE_LEVEL'] == '2nd A.M. Day Air') | (col['SERVICE_LEVEL'] == '2nd Day Air') | (col['SERVICE_LEVEL'] == '3 Day Select'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR_UPS))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_UPS))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        try: 
            return min(9.1, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_UPS))
        except ValueError:
            return 0
        
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb'): 
        try: 
            return min(8.86, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_UPS))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'Ground CWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_UPS))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_UPS))
        except ValueError:
            return 0

UPS_Rates['CLIENT_MARGIN_MINS'] = UPS_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
UPS_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - UPS_Rates['NET_CONTRACT_DISC']
UPS_Rates.loc[UPS_Rates['SERVICE_LEVEL'] == 'Next Day Air Early AM', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '9lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR)
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential') | (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_GRND)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR)

UPS_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = UPS_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

UPS_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = UPS_Rates.apply(j, axis=1)


# This is the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates

In [42]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('Accessorial_2024.xlsx')
FedEx_Rates = pd.read_excel(xls, 'FedEx_Rates')


In [43]:
# This is the value of the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-3oz'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '4-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-2lbs'):
        return 2
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '3 lbs'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-70lbs'):
        return 70
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '70+lbs'):
        return 999  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-3oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '4-16oz'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-2lbs'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '3 lbs'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-70lbs'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '70+lbs'):
        return 70

FedEx_Rates['WEIGHT_LOWER'] = FedEx_Rates.apply(g, axis=1)
FedEx_Rates['WEIGHT_UPPER'] = FedEx_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'FedEx Priority Overnight') | (col['SERVICE_LEVEL'] == 'FedEx Standard Overnight') | (col['SERVICE_LEVEL'] == 'FedEx 2nd Day AM') | (col['SERVICE_LEVEL'] == 'FedEx 2Day') | (col['SERVICE_LEVEL'] == 'FedEx Express Saver'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR_FedEx))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'FedEx Ground') | (col['SERVICE_LEVEL'] == 'FedEx Home Delivery'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_FedEx))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_FedEx))
        except ValueError:
            return 0
    
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_FedEx))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'FedEx MWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_FedEx))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'FedEx Ground Canada') | (col['SERVICE_LEVEL'] == 'FedEx Ground Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_FedEx))
        except ValueError:
            return 0

FedEx_Rates['CLIENT_MARGIN_MINS'] = FedEx_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
FedEx_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - FedEx_Rates['NET_CONTRACT_DISC']
FedEx_Rates.loc[FedEx_Rates['SERVICE_LEVEL'] == 'FedEx First Overnight', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '70lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_AIR)
    elif col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return
    elif (col['SERVICE_LEVEL'] == 'FedEx Ground') | (col['SERVICE_LEVEL'] == 'FedEx Home Delivery') | (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_GRND)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_AIR)

FedEx_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = FedEx_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

FedEx_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = FedEx_Rates.apply(j, axis=1)




In [44]:
#GRND ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_clientRates_1 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_clientRates_1['CLIENT_RATES_1'] = tbl_clientRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_clientRates_1['CLIENT_RATES'] = round(tbl_clientRates_1['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_clientRates_1 = tbl_clientRates_1.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_2 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_2['CLIENT_RATES_1'] = tbl_clientRates_2['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_clientRates_2['CLIENT_RATES'] = round(tbl_clientRates_2['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_clientRates_2 = tbl_clientRates_2.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_3 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_3['CLIENT_RATES_1'] = tbl_clientRates_3['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_clientRates_3['CLIENT_RATES'] = round(tbl_clientRates_3['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_clientRates_3 = tbl_clientRates_3.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_4 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_4['CLIENT_RATES_1'] = tbl_clientRates_4['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_clientRates_4['CLIENT_RATES'] = round(tbl_clientRates_4['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_clientRates_4 = tbl_clientRates_4.drop('CLIENT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_5 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_5['CLIENT_RATES_1'] = tbl_clientRates_5['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_clientRates_5['CLIENT_RATES'] = round(tbl_clientRates_5['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_clientRates_5 = tbl_clientRates_5.drop('CLIENT_RATES_1', axis=1)


#GRES 
#GRES ['WEIGHT_BREAK'] == '1-5lb'
tbl_clientRates_6 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_6['CLIENT_RATES_1'] = tbl_clientRates_6['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_clientRates_6['CLIENT_RATES'] = round(tbl_clientRates_6['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_clientRates_6 = tbl_clientRates_6.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_7 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_7['CLIENT_RATES_1'] = tbl_clientRates_7['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_clientRates_7['CLIENT_RATES'] = round(tbl_clientRates_7['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_clientRates_7 = tbl_clientRates_7.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_8 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_8['CLIENT_RATES_1'] = tbl_clientRates_8['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_clientRates_8['CLIENT_RATES'] = round(tbl_clientRates_8['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_clientRates_8 = tbl_clientRates_8.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_9 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_9['CLIENT_RATES_1'] = tbl_clientRates_9['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_clientRates_9['CLIENT_RATES'] = round(tbl_clientRates_9['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_clientRates_9 = tbl_clientRates_9.drop('CLIENT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_10 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_10['CLIENT_RATES_1'] = tbl_clientRates_10['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_clientRates_10['CLIENT_RATES'] = round(tbl_clientRates_10['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_clientRates_10 = tbl_clientRates_10.drop('CLIENT_RATES_1', axis=1)


#SRPT
#SRPT ['WEIGHT_BREAK'] == '1-3lb'
tbl_clientRates_11 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_11['CLIENT_RATES_1'] = tbl_clientRates_11['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_clientRates_11['CLIENT_RATES'] = round(tbl_clientRates_11['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_clientRates_11 = tbl_clientRates_11.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '4-6lb'
tbl_clientRates_12 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 6)]
DISCOUNT_TO_CLIENT_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_12['CLIENT_RATES_1'] = tbl_clientRates_12['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_clientRates_12['CLIENT_RATES'] = round(tbl_clientRates_12['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_clientRates_12 = tbl_clientRates_12.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '7-8lb'
tbl_clientRates_13 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 7) & (tbl_publishedRates['WEIGHT_LB'] <= 8)]
DISCOUNT_TO_CLIENT_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_13['CLIENT_RATES_1'] = tbl_clientRates_13['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_clientRates_13['CLIENT_RATES'] = round(tbl_clientRates_13['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_clientRates_13 = tbl_clientRates_13.drop('CLIENT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '9lb'
tbl_clientRates_14 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] == 9)]
DISCOUNT_TO_CLIENT_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_14['CLIENT_RATES_1'] = tbl_clientRates_14['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_clientRates_14['CLIENT_RATES'] = round(tbl_clientRates_14['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_clientRates_14 = tbl_clientRates_14.drop('CLIENT_RATES_1', axis=1)


#Others:
#UPS REDE, there is no discount, therefore REDE's client rate = published rate
tbl_clientRates_15 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDE')]
tbl_clientRates_15['CLIENT_RATES'] = tbl_clientRates_15['PUBLISHED_RATES']


#UPS RED
tbl_clientRates_16 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'RED')]
DISCOUNT_TO_CLIENT_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_16['CLIENT_RATES_1'] = tbl_clientRates_16['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_clientRates_16['CLIENT_RATES'] = round(tbl_clientRates_16['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_clientRates_16 = tbl_clientRates_16.drop('CLIENT_RATES_1', axis=1)


#UPS REDS, this does not have round up option (1/2)
tbl_clientRates_17 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDS')]
DISCOUNT_TO_CLIENT_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_17['CLIENT_RATES_1'] = tbl_clientRates_17['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_clientRates_17['CLIENT_RATES'] = round(tbl_clientRates_17['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_clientRates_17 = tbl_clientRates_17.drop('CLIENT_RATES_1', axis=1)


#UPS 2DAM, this does not have round up option (2/2)
tbl_clientRates_18 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAM')]
DISCOUNT_TO_CLIENT_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_18['CLIENT_RATES_1'] = tbl_clientRates_18['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_clientRates_18['CLIENT_RATES'] = round(tbl_clientRates_18['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_clientRates_18 = tbl_clientRates_18.drop('CLIENT_RATES_1', axis=1)


#UPS BLUE
tbl_clientRates_19 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'BLUE')]
DISCOUNT_TO_CLIENT_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_19['CLIENT_RATES_1'] = tbl_clientRates_19['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_clientRates_19['CLIENT_RATES'] = round(tbl_clientRates_19['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_clientRates_19 = tbl_clientRates_19.drop('CLIENT_RATES_1', axis=1)


#UPS ORNG
tbl_clientRates_20 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'ORNG')]
DISCOUNT_TO_CLIENT_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_20['CLIENT_RATES_1'] = tbl_clientRates_20['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_clientRates_20['CLIENT_RATES'] = round(tbl_clientRates_20['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_clientRates_20 = tbl_clientRates_20.drop('CLIENT_RATES_1', axis=1)


#UPS SRPT<1
tbl_clientRates_21 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT<1')]
DISCOUNT_TO_CLIENT_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_21['CLIENT_RATES_1'] = tbl_clientRates_21['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_clientRates_21['CLIENT_RATES'] = round(tbl_clientRates_21['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_clientRates_21 = tbl_clientRates_21.drop('CLIENT_RATES_1', axis=1)



tbl_ups_clientRates = pd.concat([tbl_clientRates_1, tbl_clientRates_2, tbl_clientRates_3, tbl_clientRates_4, 
                    tbl_clientRates_5, tbl_clientRates_6, tbl_clientRates_7, tbl_clientRates_8,
                    tbl_clientRates_9, tbl_clientRates_10, tbl_clientRates_11, tbl_clientRates_12,
                    tbl_clientRates_13, tbl_clientRates_14, tbl_clientRates_15, tbl_clientRates_16,
                    tbl_clientRates_17, tbl_clientRates_18, tbl_clientRates_19, tbl_clientRates_20,
                    tbl_clientRates_21])

C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2791379907.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_1['CLIENT_RATES_1'] = tbl_clientRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2791379907.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_1['CLIENT_RATES'] = round(tbl_clientRates_1['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2791379907.py

In [45]:
#GRNDF ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_clientRates_23 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_clientRates_23['CLIENT_RATES_1'] = tbl_clientRates_23['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_clientRates_23['CLIENT_RATES'] = round(tbl_clientRates_23['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_clientRates_23 = tbl_clientRates_23.drop('CLIENT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_24 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_24['CLIENT_RATES_1'] = tbl_clientRates_24['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_clientRates_24['CLIENT_RATES'] = round(tbl_clientRates_24['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_clientRates_24 = tbl_clientRates_24.drop('CLIENT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_25 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_25['CLIENT_RATES_1'] = tbl_clientRates_25['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_clientRates_25['CLIENT_RATES'] = round(tbl_clientRates_25['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_clientRates_25 = tbl_clientRates_25.drop('CLIENT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_26 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_26['CLIENT_RATES_1'] = tbl_clientRates_26['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_clientRates_26['CLIENT_RATES'] = round(tbl_clientRates_26['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_clientRates_26 = tbl_clientRates_26.drop('CLIENT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_27 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_27['CLIENT_RATES_1'] = tbl_clientRates_27['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_clientRates_27['CLIENT_RATES'] = round(tbl_clientRates_27['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_clientRates_27 = tbl_clientRates_27.drop('CLIENT_RATES_1', axis=1)


#HOME 
#HOME ['WEIGHT_BREAK'] == '1-5lb'
tbl_clientRates_28 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_28['CLIENT_RATES_1'] = tbl_clientRates_28['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_clientRates_28['CLIENT_RATES'] = round(tbl_clientRates_28['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_clientRates_28 = tbl_clientRates_28.drop('CLIENT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '6-10lb'
tbl_clientRates_29 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_29['CLIENT_RATES_1'] = tbl_clientRates_29['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_clientRates_29['CLIENT_RATES'] = round(tbl_clientRates_29['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_clientRates_29 = tbl_clientRates_29.drop('CLIENT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '11-20lb'
tbl_clientRates_30 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_30['CLIENT_RATES_1'] = tbl_clientRates_30['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_clientRates_30['CLIENT_RATES'] = round(tbl_clientRates_30['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_clientRates_30 = tbl_clientRates_30.drop('CLIENT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '21-30lb'
tbl_clientRates_31 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_31['CLIENT_RATES_1'] = tbl_clientRates_31['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_clientRates_31['CLIENT_RATES'] = round(tbl_clientRates_31['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_clientRates_31 = tbl_clientRates_31.drop('CLIENT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '31+lb'
tbl_clientRates_32 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_32['CLIENT_RATES_1'] = tbl_clientRates_32['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_clientRates_32['CLIENT_RATES'] = round(tbl_clientRates_32['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_clientRates_32 = tbl_clientRates_32.drop('CLIENT_RATES_1', axis=1)


#SPST
#SPST ['WEIGHT_BREAK'] == '1-2lb'
tbl_clientRates_33 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] <= 2)]
DISCOUNT_TO_CLIENT_11 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '1-2lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '1-2lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_33['CLIENT_RATES_1'] = tbl_clientRates_33['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_clientRates_33['CLIENT_RATES'] = round(tbl_clientRates_33['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_clientRates_33 = tbl_clientRates_33.drop('CLIENT_RATES_1', axis=1)



#SPST ['WEIGHT_BREAK'] == '3lb'
tbl_clientRates_34 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] >= 3) & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_12 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '3 lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '3 lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_34['CLIENT_RATES_1'] = tbl_clientRates_34['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_clientRates_34['CLIENT_RATES'] = round(tbl_clientRates_34['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_clientRates_34 = tbl_clientRates_34.drop('CLIENT_RATES_1', axis=1)


#SPST ['WEIGHT_BREAK'] == '4-70lb'
tbl_clientRates_35 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 70)]
DISCOUNT_TO_CLIENT_13 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '4-70lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '4-70lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_35['CLIENT_RATES_1'] = tbl_clientRates_35['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_clientRates_35['CLIENT_RATES'] = round(tbl_clientRates_35['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_clientRates_35 = tbl_clientRates_35.drop('CLIENT_RATES_1', axis=1)


#SPST ['WEIGHT_BREAK'] == '70lb+'
tbl_clientRates_36 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] == 70)]
DISCOUNT_TO_CLIENT_14 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '70+lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '70+lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_36['CLIENT_RATES_1'] = tbl_clientRates_36['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_clientRates_36['CLIENT_RATES'] = round(tbl_clientRates_36['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_clientRates_36 = tbl_clientRates_36.drop('CLIENT_RATES_1', axis=1)


#Others:
#FedEx 1STO, there is no discount, therefore 1STO's client rate = published rate
tbl_clientRates_37 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '1STO')]
tbl_clientRates_37['CLIENT_RATES'] = tbl_clientRates_37['PUBLISHED_RATES']*(1-BarrettDisc_1STO_FedEx)


#FedEx PROV
tbl_clientRates_38 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'PROV')]
DISCOUNT_TO_CLIENT_16 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'PROV'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'PROV'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_38['CLIENT_RATES_1'] = tbl_clientRates_38['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_clientRates_38['CLIENT_RATES'] = round(tbl_clientRates_38['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_clientRates_38 = tbl_clientRates_38.drop('CLIENT_RATES_1', axis=1)


#FedEx STOV, this does not have round up option (1/2)
tbl_clientRates_39 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'STOV')]
DISCOUNT_TO_CLIENT_17 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'STOV'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'STOV'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_39['CLIENT_RATES_1'] = tbl_clientRates_39['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_clientRates_39['CLIENT_RATES'] = round(tbl_clientRates_39['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_clientRates_39 = tbl_clientRates_39.drop('CLIENT_RATES_1', axis=1)


#FedEx 2AM, this does not have round up option (2/2)
tbl_clientRates_40 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2AM')]
DISCOUNT_TO_CLIENT_18 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2AM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2AM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_40['CLIENT_RATES_1'] = tbl_clientRates_40['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_clientRates_40['CLIENT_RATES'] = round(tbl_clientRates_40['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_clientRates_40 = tbl_clientRates_40.drop('CLIENT_RATES_1', axis=1)


#FedEx 2DAY
tbl_clientRates_41 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAY')]
DISCOUNT_TO_CLIENT_19 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2DAY'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2DAY') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_41['CLIENT_RATES_1'] = tbl_clientRates_41['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_clientRates_41['CLIENT_RATES'] = round(tbl_clientRates_41['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_clientRates_41 = tbl_clientRates_41.drop('CLIENT_RATES_1', axis=1)


#FedEx EXSA
tbl_clientRates_42 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'EXSA')]
DISCOUNT_TO_CLIENT_20 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'EXSA'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'EXSA'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_42['CLIENT_RATES_1'] = tbl_clientRates_42['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_clientRates_42['CLIENT_RATES'] = round(tbl_clientRates_42['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_clientRates_42 = tbl_clientRates_42.drop('CLIENT_RATES_1', axis=1)


#FedEx SPST<1 1-3oz
tbl_clientRates_43 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST<1') & (tbl_publishedRates['WEIGHT_OZ'] <= 3)]
DISCOUNT_TO_CLIENT_21 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '1-3oz'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '1-3oz'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_43['CLIENT_RATES_1'] = tbl_clientRates_43['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_clientRates_43['CLIENT_RATES'] = round(tbl_clientRates_43['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_clientRates_43 = tbl_clientRates_43.drop('CLIENT_RATES_1', axis=1)



#FedEx SPST<1 4-16oz
tbl_clientRates_44 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST<1') & (tbl_publishedRates['WEIGHT_OZ'] <= 16) & (tbl_publishedRates['WEIGHT_OZ'] >= 4)]
DISCOUNT_TO_CLIENT_22 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '4-16oz'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_22 = round(DISCOUNT_TO_CLIENT_22,2)
else:
    DISCOUNT_TO_CLIENT_22 = DISCOUNT_TO_CLIENT_22
CLIENT_MARGIN_MINS_22 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '4-16oz'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_clientRates_44['CLIENT_RATES_1'] = tbl_clientRates_44['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_22)
tbl_clientRates_44['CLIENT_RATES'] = round(tbl_clientRates_44['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_22)),2)
tbl_clientRates_44 = tbl_clientRates_44.drop('CLIENT_RATES_1', axis=1)


tbl_fedex_clientRates = pd.concat([tbl_clientRates_23, tbl_clientRates_24, tbl_clientRates_25, tbl_clientRates_26, 
                    tbl_clientRates_27, tbl_clientRates_28, tbl_clientRates_29, tbl_clientRates_30,
                    tbl_clientRates_31, tbl_clientRates_32, tbl_clientRates_33, tbl_clientRates_34,
                    tbl_clientRates_35, tbl_clientRates_36, tbl_clientRates_37, tbl_clientRates_38,
                    tbl_clientRates_39, tbl_clientRates_40, tbl_clientRates_41, tbl_clientRates_42,
                    tbl_clientRates_43, tbl_clientRates_44])


C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\1008874764.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_23['CLIENT_RATES_1'] = tbl_clientRates_23['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\1008874764.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_clientRates_23['CLIENT_RATES'] = round(tbl_clientRates_23['CLIENT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\100887476

In [46]:

#overwrite SPST client rates
DISCOUNT_TO_CLIENT_1 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '1-2lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
DISCOUNT_TO_CLIENT_2 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '3 lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
DISCOUNT_TO_CLIENT_3 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '4-70lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
DISCOUNT_TO_CLIENT_4 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '70+lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]

# Define the conditions
condition1 = (tbl_fedex_clientRates['SERVICE_CODE'] == 'SPST') & (tbl_fedex_clientRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_fedex_clientRates['WEIGHT_LB'] <= 2)
condition2 = (tbl_fedex_clientRates['SERVICE_CODE'] == 'SPST') & (tbl_fedex_clientRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_fedex_clientRates['WEIGHT_LB'] > 2) & (tbl_fedex_clientRates['WEIGHT_LB'] < 4)
condition3 = (tbl_fedex_clientRates['SERVICE_CODE'] == 'SPST') & (tbl_fedex_clientRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_fedex_clientRates['WEIGHT_LB'] >= 4) & (tbl_fedex_clientRates['WEIGHT_LB'] < 70)
condition4 = (tbl_fedex_clientRates['SERVICE_CODE'] == 'SPST') & (tbl_fedex_clientRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_fedex_clientRates['WEIGHT_LB'] >= 70)


# Overwrite values where the condition is met
tbl_fedex_clientRates.loc[condition1, 'CLIENT_RATES'] = tbl_fedex_clientRates.loc[condition1, 'PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - DISCOUNT_TO_CLIENT_1)))
tbl_fedex_clientRates.loc[condition2, 'CLIENT_RATES'] = tbl_fedex_clientRates.loc[condition2, 'PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - DISCOUNT_TO_CLIENT_2)))
tbl_fedex_clientRates.loc[condition3, 'CLIENT_RATES'] = tbl_fedex_clientRates.loc[condition3, 'PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - DISCOUNT_TO_CLIENT_3)))
tbl_fedex_clientRates.loc[condition4, 'CLIENT_RATES'] = tbl_fedex_clientRates.loc[condition4, 'PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - DISCOUNT_TO_CLIENT_4)))



In [47]:
# Create DHL and USPS Client Rate
tbl_dhl_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG<1_')) | 
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLEM_'))]

tbl_usps_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'USPSAG') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP') |
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAG_CI') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP_CI')]


tbl_mi_clientRates = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'MIPLE') | 
                                          (tbl_publishedRates['SERVICE_CODE'] == 'MIPH')]


tbl_dhl_clientRates['CLIENT_RATES'] = round(tbl_dhl_clientRates['BARRETT_RATES']/(1 - Margin_on_DHL),2)
tbl_usps_clientRates['CLIENT_RATES'] = round(tbl_usps_clientRates['BARRETT_RATES']/(1 - Margin_on_USPS),2)
tbl_mi_clientRates['CLIENT_RATES'] = round(tbl_mi_clientRates['BARRETT_RATES']/(1 - Margin_on_MI),2)


tbl_clientRates = pd.concat([tbl_ups_clientRates, tbl_fedex_clientRates, tbl_dhl_clientRates, tbl_usps_clientRates, tbl_mi_clientRates])
tbl_clientRates = tbl_clientRates.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'CUBIC_FT', 'PUBLISHED_RATES', 'CLIENT_RATES', 'VERSION', 'START_DATE', 'END_DATE'])

#get the finalized tbl_publishedRates
tbl_clientRates = tbl_BarrettRates.merge(tbl_clientRates[['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'CUBIC_FT', 'VERSION', 'START_DATE', 'END_DATE', 'CLIENT_RATES']], left_on=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'VERSION', 'START_DATE', 'END_DATE', 'CUBIC_FT'], right_on=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'VERSION', 'START_DATE', 'END_DATE', 'CUBIC_FT'], how='left')
#tbl_clientRates.to_csv('test.csv')



C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2179308695.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_dhl_clientRates['CLIENT_RATES'] = round(tbl_dhl_clientRates['BARRETT_RATES']/(1 - Margin_on_DHL),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2179308695.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_usps_clientRates['CLIENT_RATES'] = round(tbl_usps_clientRates['BARRETT_RATES']/(1 - Margin_on_USPS),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\2179308695.py:20: SettingWith

# Step 2 - a. Create UPS accessorial rates
*This rate is separated from freight rate

In [48]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial
xls = pd.ExcelFile('Accessorial_2024.xlsx')
UPS_Accessorial = pd.read_excel(xls, 'UPS_Accessorial')

# Add Net_charge column
UPS_Accessorial['NET_CHARGE'] = UPS_Accessorial['PUBLISHED_CHARGE']*(1-UPS_Accessorial['DISCOUNT'])

# Add ADD_MARGIN column
def k(col):   
    if UPS_Accessorial['ACCESSORIAL'].str.contains('Air').any():
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_AIR_UPS)
    else:
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_GRND_UPS)

UPS_Accessorial['ADD_MARGIN'] = UPS_Accessorial.apply(k, axis=1)  


# Assign a number to a specific cell based on column name and other row values
column_name = 'ADD_MARGIN'
condition1 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area' 
condition2 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area Alaska' 
condition3 = UPS_Accessorial['ACCESSORIAL'] == 'Remote Area Hawaii' 

UPS_Accessorial.loc[condition1, column_name] = 14.15
UPS_Accessorial.loc[condition2, column_name] = 40.50
UPS_Accessorial.loc[condition3, column_name] = 14.15


# Add PERCENTAGE column
def l(col):   
    if Round_Discount == 'YES':
        return 1 - round((col['ADD_MARGIN']/col['PUBLISHED_CHARGE']),2)
    else:
        return 1 - col['ADD_MARGIN']/col['PUBLISHED_CHARGE']

UPS_Accessorial['PERCENTAGE'] = UPS_Accessorial.apply(l, axis=1)

In [49]:
# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial
xls = pd.ExcelFile('Accessorial_2024.xlsx')
FedEx_Accessorial = pd.read_excel(xls, 'FedEx_Accessorial')

# Add Net_charge column
FedEx_Accessorial['NET_CHARGE'] = round(FedEx_Accessorial['PUBLISHED_CHARGE']*(1-FedEx_Accessorial['DISCOUNT']),2)

# Add ADD_MARGIN column
def k(col):   
    if 'Air' in col['ACCESSORIAL'] or 'Express' in col['ACCESSORIAL']:
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_AIR_FedEx)
    else:
        return col['NET_CHARGE']/(1-Margin_on_Value_Add_GRND_FedEx)

FedEx_Accessorial['ADD_MARGIN'] = FedEx_Accessorial.apply(k, axis=1)  

# Add PERCENTAGE column
def l(col):   
    if Round_Discount == 'YES':
        return 1 - round((col['ADD_MARGIN']/col['PUBLISHED_CHARGE']),2)
    else:
        return 1 - round((col['ADD_MARGIN']/col['PUBLISHED_CHARGE']),2)

FedEx_Accessorial['PERCENTAGE'] = FedEx_Accessorial.apply(l, axis=1)

# Step 2 - b. Prepare all other rates for rerate
*These rates do not need a table to house

In [50]:
# List all UPS Surcharge rates for rerate
Resi_Surcharge_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Resi Surcharge (Air)'), 'ADD_MARGIN'].values[0]
Resi_Surcharge_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Resi Surcharge (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Extended_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm Extended (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Ground = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi Extended (Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm (Air)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Extended_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Comm Extended (Air)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi (Air)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Air = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Resi Extended (Air)'), 'ADD_MARGIN'].values[0]
DAS_SurePost = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS SurePost'), 'ADD_MARGIN'].values[0]
DAS_Extended_SurePost = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'DAS Extended SurePost'), 'ADD_MARGIN'].values[0]
Remote_Area = 14.15
Remote_Area_Alaska = 40.50
Remote_Area_Hawaii = 14.15


# Get FSC ready for rerate
xls = pd.ExcelFile('FSC.xlsx')
UPS_FSC = pd.read_excel(xls, 'UPS_FSC')
DHL_FSC = pd.read_excel(xls, 'DHL_FSC')
FedEx_FSC = pd.read_excel(xls, 'FedEx_FSC')

#Get UPS_FSC doc ready
UPS_FSC['SHIP_DATE_START'] = UPS_FSC['Ship Date']
UPS_FSC['Ship Date'] = pd.to_datetime(UPS_FSC['Ship Date']) 
UPS_FSC['SHIP_DATE_END'] = UPS_FSC['Ship Date'] + pd.Timedelta(days=6)
UPS_FSC = UPS_FSC.drop('Ship Date', axis=1)

UPS_FSC['SHIP_DATE_START'] = pd.to_datetime(UPS_FSC['SHIP_DATE_START']).dt.date
UPS_FSC['SHIP_DATE_END'] = pd.to_datetime(UPS_FSC['SHIP_DATE_END']).dt.date

#Get DHL_FSC doc ready
from datetime import timedelta
from datetime import date
DHL_FSC = DHL_FSC.sort_values(by='Ship Date', ascending=True)
DHL_FSC['Ship Date'] = pd.to_datetime(DHL_FSC['Ship Date']) 
DHL_FSC['SHIP_DATE_START'] = DHL_FSC['Ship Date']
DHL_FSC['SHIP_DATE_END'] = DHL_FSC['SHIP_DATE_START'].shift(-1) - timedelta(days=1)

#Get FedEx_FSC doc ready
FedEx_FSC['SHIP_DATE_START'] = FedEx_FSC['Ship Date']
FedEx_FSC['Ship Date'] = pd.to_datetime(FedEx_FSC['Ship Date']) 
FedEx_FSC['SHIP_DATE_END'] = FedEx_FSC['Ship Date'] + pd.Timedelta(days=6)
FedEx_FSC = FedEx_FSC.drop('Ship Date', axis=1)

FedEx_FSC['SHIP_DATE_START'] = pd.to_datetime(FedEx_FSC['SHIP_DATE_START']).dt.date
FedEx_FSC['SHIP_DATE_END'] = pd.to_datetime(FedEx_FSC['SHIP_DATE_END']).dt.date

# Calculate a date that is 2 years in the future
future_date = date.today().replace(year=date.today().year + 2)
# Update the 'SHIP_DATE_END' column for rows with the latest 'SHIP_DATE_START'
DHL_FSC.loc[DHL_FSC['SHIP_DATE_START'] == max(DHL_FSC['SHIP_DATE_START']), ['SHIP_DATE_END']] = future_date

DHL_FSC['SHIP_DATE_END'] = pd.to_datetime(DHL_FSC['SHIP_DATE_END']) 
DHL_FSC = DHL_FSC.drop('Ship Date', axis=1)

# List UPS AHS rates for rerate
AH_Girth_2_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 2'), 'ADD_MARGIN'].values[0]
AH_Girth_3_4_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 3-4'), 'ADD_MARGIN'].values[0]
AH_Girth_5_up_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling Girth 5+'), 'ADD_MARGIN'].values[0]

AH_L_2_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 2'), 'ADD_MARGIN'].values[0]
AH_L_3_4_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 3-4'), 'ADD_MARGIN'].values[0]
AH_L_5_up_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling L 5+'), 'ADD_MARGIN'].values[0]

AH_W_2_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 2'), 'ADD_MARGIN'].values[0]
AH_W_3_4_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 3-4'), 'ADD_MARGIN'].values[0]
AH_W_5_up_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling W 5+'), 'ADD_MARGIN'].values[0]

AH_WGT_2_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 2'), 'ADD_MARGIN'].values[0]
AH_WGT_3_4_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 3-4'), 'ADD_MARGIN'].values[0]
AH_WGT_5_up_UPS = UPS_Accessorial.loc[(UPS_Accessorial['ACCESSORIAL'] == 'Additional Handling WGT 5+'), 'ADD_MARGIN'].values[0]


C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\3619121388.py:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2026-08-22' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  DHL_FSC.loc[DHL_FSC['SHIP_DATE_START'] == max(DHL_FSC['SHIP_DATE_START']), ['SHIP_DATE_END']] = future_date


In [51]:
# List all FedEx Surcharge rates for rerate
Resi_Surcharge_Air_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Residential Air Surcharge'), 'ADD_MARGIN'].values[0]
Resi_Surcharge_Ground_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Residential Ground Surcharge'), 'ADD_MARGIN'].values[0]
DAS_Comm_Ground_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Commercial(Ground)'), 'ADD_MARGIN'].values[0]
DAS_Comm_Extended_Ground_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Extended Commercial(Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Ground_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Residential(Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Ground_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Extended Residential(Ground)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Air_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Residential(Express)'), 'ADD_MARGIN'].values[0]
DAS_Resi_Extended_Air_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx DAS Extended Residential(Express)'), 'ADD_MARGIN'].values[0]
Remote_Area_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Remote Area Surcharge (Commercial and Residential)'), 'ADD_MARGIN'].values[0]
Remote_Area_Alaska_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Remote Area Alaska'), 'ADD_MARGIN'].values[0]
Remote_Area_Hawaii_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Remote Area Hawaii'), 'ADD_MARGIN'].values[0]

# List FedEx AHS rates for rerate
AH_Dim_2_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Dimension 2'), 'ADD_MARGIN'].values[0]
AH_Dim_3_4_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Dimension 3-4'), 'ADD_MARGIN'].values[0]
AH_Dim_5_6_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Dimension 5-6'), 'ADD_MARGIN'].values[0]
AH_Dim_7_up_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Dimension 7+'), 'ADD_MARGIN'].values[0]

AH_WGT_2_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Weight 2'), 'ADD_MARGIN'].values[0]
AH_WGT_3_4_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Weight 3-4'), 'ADD_MARGIN'].values[0]
AH_WGT_5_6_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Weight 5-6'), 'ADD_MARGIN'].values[0]
AH_WGT_7_up_FedEx = FedEx_Accessorial.loc[(FedEx_Accessorial['ACCESSORIAL'] == 'FedEx Additional Handling Weight 7+'), 'ADD_MARGIN'].values[0]



# Step 3 - a. Prepare PLD for rerate
* Regardless of various types of PLD, Client PLD, Barrett PLD, etc.

# Load PLD

In [123]:
#change the corresponding column names to below names before upload. 
#Requested columns and the corresponding type:     
# ---  ------             --------------  -----         
#1   TRACKING_NUMBER    
#2   SHIP_DATE             
#3   OLD_SERVICE --(optional)      
#4   ZIP_CODE               
#5   ZONE --(optional)               
#6   DIMENSIONS or LWH           
#7   ACTUAL_WEIGHT               
#8   RESIDENTIAL_FLAG       
#9   INTERNATIONAL_FLAG    

#Load Sample PLD
xls = pd.ExcelFile('pld_rate_model_FM_DHL&MI.xlsx')
tbl_BP_Master = pd.read_excel(xls, 'Sheet1')
#tbl_BP.head()

In [124]:
#When the original file is too large, make a copy
tbl_BP = tbl_BP_Master.copy()

In [125]:
# clean the empties if necessary
tbl_BP = tbl_BP.dropna(axis = 0, how = 'all')
tbl_BP = tbl_BP.dropna(axis = 1, how = 'all')

In [126]:
tbl_BP.shape

(104086, 30)

# Clean PLD

In [672]:
#Separate the correct data from those error data. 
tbl_BP_delete = tbl_BP[#(tbl_BP.TRACKING_NUMBER.isnull()) |
                       (tbl_BP.SHIP_DATE.isnull()) | 
                        (tbl_BP.ZIP_CODE.isnull()) | 
                        (tbl_BP.DIMENSIONS.isnull()) | 
                        (tbl_BP.ACTUAL_WEIGHT.isnull()) | 
                        (tbl_BP.ACTUAL_WEIGHT == 0) | 
                       # (tbl_BP.ZONE.isnull()) |
                        (tbl_BP.RESIDENTIAL_FLAG.isnull()) #| #will not be rated under ups except SRPT
                        #(tbl_BP.L.isnull()) | 
                        #(tbl_BP.L == 0) |  
                        #(tbl_BP.W.isnull()) |
                        #(tbl_BP.W == 0) | 
                        #(tbl_BP.H.isnull()) | 
                        #(tbl_BP.H == 0) 
                       #| 
                        #(tbl_BP['INTERNATIONAL_FLAG'] == True)
                      ]

tbl_BP = tbl_BP[~(#(tbl_BP.TRACKING_NUMBER.isnull()) |
                 (tbl_BP.SHIP_DATE.isnull()) | 
                 (tbl_BP.ZIP_CODE.isnull()) | 
                 (tbl_BP.DIMENSIONS.isnull()) | 
                 (tbl_BP.ACTUAL_WEIGHT.isnull()) | 
                 (tbl_BP.ACTUAL_WEIGHT == 0) | 
                # (tbl_BP.ZONE.isnull()) |
                 (tbl_BP.RESIDENTIAL_FLAG.isnull()) #|
                # (tbl_BP.L.isnull()) | 
                # (tbl_BP.L == 0) |  
                # (tbl_BP.W.isnull()) |
                # (tbl_BP.W == 0) | 
                # (tbl_BP.H.isnull()) | 
                # (tbl_BP.H == 0)
                  #| 
                 #(tbl_BP['INTERNATIONAL_FLAG'] == True)
)]


In [673]:
#Output those incomplete PLD that cannot be used in this model
tbl_BP_delete.to_csv('tbl_BP_delete.csv')

In [127]:
#The column type must match
#Requested columns and the corresponding type:     
# ---  ------             --------------  -----         
#1   TRACKING_NUMBER        ---------------------      int64 (or object) "can manually create one before upload"       
#2   SHIP_DATE              ---------------------      datetime64[ns] "can manually fill up before upload"
#3   OLD_SERVICE (optional) ---------------------      object        
#4   ZIP_CODE               ---------------------      object
#5   ZONE (optional)        ---------------------      object          
#6   DIMENSIONS (or LWH)    ---------------------      object (or float64 or int64)       
#7   ACTUAL_WEIGHT (lb as default)---------------      float64       
#8   RESIDENTIAL_FLAG       ---------------------      bool   
tbl_BP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104086 entries, 0 to 104085
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CustomerID          104086 non-null  int64         
 1   CustomerName        104086 non-null  object        
 2   Facility            104086 non-null  object        
 3   BarrettOrderNumber  104086 non-null  object        
 4   Reference           104086 non-null  object        
 5   PoNumber            1863 non-null    object        
 6   TrackingNumber      1863 non-null    object        
 7   ShipDate            104086 non-null  datetime64[ns]
 8   Service             104086 non-null  object        
 9   Shipper             1863 non-null    object        
 10  ShipToName          1863 non-null    object        
 11  ShipToContact       28 non-null      object        
 12  ShipToCity          104086 non-null  object        
 13  ShipToState         104086 no

In [128]:
#update column type
#tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)
#tbl_BP['ZONE'] = pd.to_numeric(tbl_BP['ZONE'], errors='coerce')
# Convert float values to integers
#tbl_BP['ZONE']= tbl_BP['ZONE'].fillna(0).astype(int)

#tbl_BP['RESIDENTIAL_FLAG'] = tbl_BP['RESIDENTIAL_FLAG'].astype('bool')
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)
#tbl_BP['SHIP_DATE'] = pd.to_datetime(tbl_BP['SHIP_DATE'])

In [129]:
#Check the Zip_Code values
tbl_BP['ZIP_CODE'].unique() 

array(['10011', '21113', '77365', ..., '68767', '73772', '63845'],
      dtype=object)

In [424]:
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)

C:\Users\sliu\AppData\Local\Temp\ipykernel_9692\2751569339.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str).str.replace('\.0$', '').astype(int)


In [130]:
#Check the value of Zone if exists
tbl_BP['ZONE'].unique()

array(['5', '4', '7', '6', '8', '2', '3'], dtype=object)

In [40]:
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('102', '2')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('108', '8')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('104', '4')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('44', '9')
#tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('46', '11')

In [60]:
#Drop duplicates based on ID column by only keeping the unique ID associated with the most recent Ship Date.  
#If there are rows where both tracking no and ship date are duplicated, only keep the first one.
tbl_BP = tbl_BP.sort_values('SHIP_DATE', ascending=False)
tbl_BP = tbl_BP.drop_duplicates(subset='TRACKING_NUMBER', keep='first')
tbl_BP = tbl_BP.drop_duplicates(subset='TRACKING_NUMBER', keep='first')
#tbl_BP.head()

# Format PLD

In [131]:
#Skip this if LWH Columns exist already. This line of code needs improvement. 
#turn dimensions into L W H columns. no need to run this if they are already separate. 
tbl_BP[['L', 'W', 'H']] = tbl_BP['DIMENSIONS'].str.split('x', expand=True)

In [132]:
#Skip this if LWH Columns are integers. 
tbl_BP[['L', 'W', 'H']] = tbl_BP[['L', 'W', 'H']].apply(pd.to_numeric)
tbl_BP['L'] = tbl_BP['L'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))
tbl_BP['W'] = tbl_BP['W'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))
tbl_BP['H'] = tbl_BP['H'].apply(lambda x: round(x,0) if x - math.floor(x) < 0.5 else np.ceil(x))


In [133]:
# Ensure that the columns are of a numeric type (float)
tbl_BP['L'] = tbl_BP['L'].astype(float)
tbl_BP['W'] = tbl_BP['W'].astype(float)
tbl_BP['H'] = tbl_BP['H'].astype(float)

# Create a temporary DataFrame with the columns 'L', 'W', and 'H'
temp_df = tbl_BP[['L', 'W', 'H']]

# Sort each row in descending order and reverse the columns to maintain the order L, W, H
sorted_temp_df = np.sort(temp_df.values, axis=1)[:, ::-1]

# Assign the sorted values back to the original DataFrame
tbl_BP[['L', 'W', 'H']] = sorted_temp_df

In [24]:
tbl_BP[['L_CI', 'W_CI', 'H_CI']] = tbl_BP['DIMENSIONS'].str.split('x', expand=True)

In [25]:
# Ensure that the columns are of a numeric type (float)
tbl_BP['L_CI'] = tbl_BP['L_CI'].astype(float)
tbl_BP['W_CI'] = tbl_BP['W_CI'].astype(float)
tbl_BP['H_CI'] = tbl_BP['H_CI'].astype(float)

# Create a temporary DataFrame with the columns 'L', 'W', and 'H'
temp_df = tbl_BP[['L_CI', 'W_CI', 'H_CI']]

# Sort each row in descending order and reverse the columns to maintain the order L, W, H
sorted_temp_df = np.sort(temp_df.values, axis=1)[:, ::-1]

# Assign the sorted values back to the original DataFrame
tbl_BP[['L_CI', 'W_CI', 'H_CI']] = sorted_temp_df

In [26]:
# Function to round down to the nearest quarter
def round_down_to_quarter_inch(value):
    return (value // 0.25) * 0.25

# Assuming df1 is your DataFrame with the columns 'Length', 'Width', and 'Height'
# Apply the function to each of the three columns
tbl_BP['L_CI'] = tbl_BP['L_CI'].apply(round_down_to_quarter_inch)
tbl_BP['W_CI'] = tbl_BP['W_CI'].apply(round_down_to_quarter_inch)
tbl_BP['H_CI'] = tbl_BP['H_CI'].apply(round_down_to_quarter_inch)


In [134]:
# Create columns of GIRTH_AND_L and CUBIC_INCH
tbl_BP['GIRTH_AND_L'] = round(tbl_BP['W'] * 2 + tbl_BP['H'] * 2 + tbl_BP['L'],0)
tbl_BP['CUBIC_INCH'] = round(tbl_BP['W'] * tbl_BP['H'] * tbl_BP['L'],0)


In [135]:
# Create ACTUAL_WEIGHT_LB(rounded up to the nearest lb) and ACTUAL_WEIGHT_OZ(rounded up to the nearest oz). 
# Still keep the same value if the cell is empty string or has any missing values.  
tbl_BP['ACTUAL_WEIGHT_LB'] = tbl_BP['ACTUAL_WEIGHT'].apply(lambda x: np.ceil(x) if pd.notna(x) and x != '' else x)
tbl_BP['ACTUAL_WEIGHT_OZ_1'] = tbl_BP['ACTUAL_WEIGHT']*16
tbl_BP['ACTUAL_WEIGHT_OZ'] = tbl_BP['ACTUAL_WEIGHT_OZ_1'].apply(lambda x: np.ceil(x) if pd.notna(x) and x != '' else x)
tbl_BP = tbl_BP.drop('ACTUAL_WEIGHT_OZ_1', axis=1)

In [66]:
# Skip this if the left 5 digits are provided already
# get the left 5 digits of the zip code or fill the zip to 5 digits
#import re
#split_col = tbl_BP['ZIP_CODE'].str.split('-')
#left_col = split_col.str[0]
#tbl_BP['ZIP_CODE'] = left_col
#tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].apply(lambda x: re.match(r'\d{5}', str(x)).group(0) if (pd.notna(x) and re.match(r'\d{5}', str(x))) else '')
# fill the 3 and 4 digits zip to 5 digits
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].astype(str)
tbl_BP['ZIP_CODE'] = tbl_BP['ZIP_CODE'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

In [30]:
# Add Cubic Inch column for regular pack and L+W column for soft pack
tbl_BP['CUBIC_FT_CI'] = np.ceil(((tbl_BP['L_CI'] * tbl_BP['W_CI'] * tbl_BP['H_CI'])/1728)*10)/10
tbl_BP['L_AND_W_CI'] = np.ceil((tbl_BP['L_CI'] + tbl_BP['W_CI'])*10)/10


# Decide the facility to use (Optional)

#Decide by most likely State

In [38]:
# When PLD does not have Zone column, the origin facility needs to be decided based on destination State. 
# Specify the columns you want to read
columns_to_read = ['Zipcode', 'State']

# Read the CSV file and select the desired columns
tbl_ZiptoState = pd.read_excel('ZiptoState.xlsx', usecols=columns_to_read)

# Drop duplicates based on specific columns
tbl_ZiptoState = tbl_ZiptoState.drop_duplicates(subset=['Zipcode'])

#Add State column to PLD
tbl_ZiptoState['Zipcode'] = tbl_ZiptoState['Zipcode'].astype(str)
tbl_BP = tbl_BP.merge(tbl_ZiptoState[["Zipcode", "State"]], left_on='ZIP_CODE', right_on='Zipcode', how='left').rename(columns={'State': 'STATE'}).drop('Zipcode', axis=1)


In [73]:
grouped_tbl_BP = tbl_BP.groupby('STATE')['TRACKING_NUMBER'].count()
ranked_tbl_BP = grouped_tbl_BP.sort_values(ascending=False)
#If this facility selection is not obvious, use the extended modal to calculate everything and calculate. 
ranked_tbl_BP

#Decide by Zone Count

In [150]:
# select facilities based on the total Zone Count. 
tbl_BP['FROM_ZIP_02038'] = '2038'
tbl_BP['FROM_ZIP_02038'] = tbl_BP['FROM_ZIP_02038'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_02038', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_02038'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].astype('str')
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].str.replace('44', '9')
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].str.replace('45', '10')
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].str.replace('46', '11')
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_08873'] = '8873'
tbl_BP['FROM_ZIP_08873'] = tbl_BP['FROM_ZIP_08873'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_08873', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_08873'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].astype('str')
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].str.replace('44', '9')
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].str.replace('45', '10')
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].str.replace('46', '11')
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_21226'] = '21226'
tbl_BP['FROM_ZIP_21226'] = tbl_BP['FROM_ZIP_21226'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_21226', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_21226'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].astype('str')
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].str.replace('44', '9')
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].str.replace('45', '10')
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].str.replace('46', '11')
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_38141'] = '38141'
tbl_BP['FROM_ZIP_38141'] = tbl_BP['FROM_ZIP_38141'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_38141', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_38141'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].astype('str')
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].str.replace('44', '9')
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].str.replace('45', '10')
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].str.replace('46', '11')
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_38654'] = '38654'
tbl_BP['FROM_ZIP_38654'] = tbl_BP['FROM_ZIP_38654'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_38654', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_38654'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].astype('str')
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].str.replace('44', '9')
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].str.replace('45', '10')
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].str.replace('46', '11')
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_38654'] = tbl_BP['ZONE_38654'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_75041'] = '75041'
tbl_BP['FROM_ZIP_75041'] = tbl_BP['FROM_ZIP_75041'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_75041', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_75041'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].astype('str')
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].str.replace('44', '9')
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].str.replace('45', '10')
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].str.replace('46', '11')
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_75041'] = tbl_BP['ZONE_75041'].astype(str).str.replace('\.0$', '').astype(int)


tbl_BP['FROM_ZIP_90640'] = '90640'
tbl_BP['FROM_ZIP_90640'] = tbl_BP['FROM_ZIP_90640'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP_90640', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE_90640'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].astype('str')
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].str.replace('44', '9')
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].str.replace('45', '10')
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].str.replace('46', '11')
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE_90640'] = tbl_BP['ZONE_90640'].astype(str).str.replace('\.0$', '').astype(int)


C:\Users\sliu\AppData\Local\Temp\ipykernel_17780\2900919867.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE_02038'] = tbl_BP['ZONE_02038'].astype(str).str.replace('\.0$', '').astype(int)
C:\Users\sliu\AppData\Local\Temp\ipykernel_17780\2900919867.py:31: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE_08873'] = tbl_BP['ZONE_08873'].astype(str).str.replace('\.0$', '').astype(int)
C:\Users\sliu\AppData\Local\Temp\ipykernel_17780\2900919867.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE_21226'] = tbl_BP['ZONE_21226'].astype(str).str.replace('\.0$', '').astype(int)
C:\Users\sliu\AppData\Local\Temp\ipykernel_17780\2900919867.py:63: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE_38141'] = tbl_BP['ZONE_38141'].astype(str).str.replace('\.0

In [151]:
# List of columns to consider
columns_to_sum = ['ZONE_02038', 'ZONE_08873', 'ZONE_21226', 'ZONE_38141', 'ZONE_38654', 'ZONE_75041', 'ZONE_90640']

# Calculate the sum for each column and get the index of the minimum value
min_index = np.argmin([tbl_BP[col].sum() for col in columns_to_sum])

# Get the column name with the minimum sum
column_with_min_value = columns_to_sum[min_index]

# Get the minimum sum value
min_value = tbl_BP[column_with_min_value].sum()

In [152]:
min_value

15633

In [153]:
column_with_min_value

'ZONE_21226'

In [154]:
# List of columns to drop
columns_to_drop = ['ZONE_02038', 'FROM_ZIP_02038', 'ZONE_08873', 'FROM_ZIP_08873', 'ZONE_38141', 'FROM_ZIP_38141', 'ZONE_38654', 'FROM_ZIP_38654', 'ZONE_75041', 'FROM_ZIP_75041', 'ZONE_90640', 'FROM_ZIP_90640']

# Drop the specified columns
tbl_BP.drop(columns=columns_to_drop, inplace=True)

tbl_BP.rename(columns={'ZONE_21226': 'ZONE'}, inplace=True)


In [155]:
tbl_BP.head()

,Payor,Ground Tracking ID Prefix,Transportation Charge Amount,Net Charge Amount,POD Delivery Date,POD Delivery Time,POD Service Area Code,Actual Weight Units,Rated Weight Amount,Rated Weight Units,...,H,ACTUAL_WEIGHT,RESIDENTIAL_FLAG,GIRTH_AND_L,CUBIC_INCH,ACTUAL_WEIGHT_LB,ACTUAL_WEIGHT_OZ,STATE,FROM_ZIP_21226,ZONE
0,Shipper,NaN,87.58,28.90,20230425.0,10:06,A2,lbs,2.0,lbs,...,5.0,2.0,True,34.0,350.0,2.0,32.0,NaN,21226,3
1,Shipper,NaN,125.60,39.42,20230425.0,10:40,A1,lbs,9.0,lbs,...,7.0,9.0,True,56.0,1456.0,9.0,144.0,IL,21226,4
2,Shipper,NaN,85.04,28.20,20230425.0,10:45,A2,lbs,2.0,lbs,...,5.0,1.0,True,34.0,350.0,1.0,16.0,MD,21226,2
3,Shipper,NaN,139.33,43.22,20230425.0,09:25,A1,lbs,10.0,lbs,...,11.0,3.0,True,62.0,2112.0,3.0,48.0,SD,21226,6
4,Shipper,NaN,193.80,58.28,20230425.0,11:38,AA,lbs,11.0,lbs,...,7.0,5.0,True,67.0,2261.0,5.0,80.0,CA,21226,8


# Specify the number of facilities to rerate

**specific one facility rerate**

In [535]:
#specify FedEx facility
OB_STATE = pd.read_csv('C:\\Users\\SLiu\\BD\\model\\RateQuote\\OB_STATE_FedEx.csv')
OB_STATE[['Origin', 'Service']] = OB_STATE['FileID'].str.split('_', expand=True)
OB_STATE.rename(columns={' Dest Zip/Postal Code': 'Dest Zip'}, inplace=True)

tbl_BP['FROM_ZIP'] = '38141' # change this to the correct "ship from" facility
tbl_BP['FROM_ZIP'] = tbl_BP['FROM_ZIP'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

OB_STATE1 = OB_STATE[(OB_STATE['Service'] == 'FXG')]
OB_STATE2 = OB_STATE[(OB_STATE['Service'] == 'FHD')]
tbl_BP1 = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == False)]
tbl_BP2 = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == True)]

tbl_BP1 = tbl_BP1.merge(OB_STATE1[["Origin", "Dest Zip", " Zone"]], left_on=['FROM_ZIP', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={' Zone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP1['ZONE'] = tbl_BP1['ZONE'].astype('str')
#tbl_BP1['ZONE'] = tbl_BP1['ZONE'].str.replace('44', '9')
#tbl_BP1['ZONE'] = tbl_BP1['ZONE'].str.replace('45', '10')
#tbl_BP1['ZONE'] = tbl_BP1['ZONE'].str.replace('46', '11')
#tbl_BP1['ZONE'] = tbl_BP1['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
#tbl_BP1['ZONE'] = tbl_BP1['ZONE'].astype(str).str.replace('\.0$', '').astype(int)

tbl_BP2 = tbl_BP2.merge(OB_STATE2[["Origin", "Dest Zip", " Zone"]], left_on=['FROM_ZIP', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={' Zone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP2['ZONE'] = tbl_BP2['ZONE'].astype('str')
#tbl_BP2['ZONE'] = tbl_BP2['ZONE'].str.replace('44', '9')
#tbl_BP2['ZONE'] = tbl_BP2['ZONE'].str.replace('45', '10')
#tbl_BP2['ZONE'] = tbl_BP2['ZONE'].str.replace('46', '11')
#tbl_BP2['ZONE'] = tbl_BP2['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
#tbl_BP2['ZONE'] = tbl_BP2['ZONE'].astype(str).str.replace('\.0$', '').astype(int)

tbl_BP = pd.concat([tbl_BP1, tbl_BP2])

tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)

tbl_BP1 = tbl_BP[(tbl_BP['ZONE'] != '0')]
tbl_BP3 = tbl_BP[(tbl_BP['ZONE'] == '0')]
# List of columns to drop
columns_to_drop = ['ZONE']

# Drop the specified columns
tbl_BP3.drop(columns=columns_to_drop, inplace=True)
tbl_BP3 = tbl_BP3.merge(OB_STATE1[["Origin", "Dest Zip", " Zone"]], left_on=['FROM_ZIP', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={' Zone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP3['ZONE'] = tbl_BP3['ZONE'].astype('str')

tbl_BP = pd.concat([tbl_BP1, tbl_BP3])
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str)

tbl_BP.rename(columns={'ZONE': 'ZONE_FedEx'}, inplace=True)



In [537]:
#specify UPS facility
OB_STATE = pd.read_csv('C:\\Users\\SLiu\\BD\\model\\RateQuote\\OB_STATE_UPS.csv')
tbl_BP['FROM_ZIP'] = '38141'
tbl_BP['FROM_ZIP'] = tbl_BP['FROM_ZIP'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['FROM_ZIP', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype('str')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('44', '9')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('45', '10')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('46', '11')
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)

tbl_BP.rename(columns={'ZONE': 'ZONE_UPS'}, inplace=True)
tbl_BP['ZONE_UPS'] = tbl_BP['ZONE_UPS'].astype('str')

C:\Users\SLiu\AppData\Local\Temp\ipykernel_2256\2498246324.py:1: DtypeWarning: Columns (6,7,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  OB_STATE = pd.read_csv('C:\\Users\\SLiu\\BD\\model\\RateQuote\\OB_STATE_UPS.csv')


**specific 2 facilities rerate**

In [281]:
tbl_BP1 = tbl_BP[(tbl_BP['FACILITY'] == 'JER')]
tbl_BP2 = tbl_BP[(tbl_BP['FACILITY'] == 'CA3')]

tbl_BP1['From Zip'] = '08873'
tbl_BP2['From Zip'] = '90640'

tbl_BP = pd.concat([tbl_BP1, tbl_BP2])

tbl_BP['From Zip'] = tbl_BP['From Zip'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['From Zip', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype('str')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('44', '9')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('45', '10')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('46', '11')
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)

C:\Users\sliu\AppData\Local\Temp\ipykernel_12636\3153116086.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)


**Specify all facility rerate**

In [186]:
tbl_BP['From Zip'] = tbl_BP['From Zip'].astype(str)
OB_STATE['Origin'] = OB_STATE['Origin'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].astype(str)
OB_STATE['Dest Zip'] = OB_STATE['Dest Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(OB_STATE[["Origin", "Dest Zip", "GNDZone"]], left_on=['From Zip', 'ZIP_CODE'], right_on=['Origin','Dest Zip'], how='left').rename(columns={'GNDZone': 'ZONE'}).drop(['Origin','Dest Zip'], axis=1)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype('str')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('44', '9')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('45', '10')
tbl_BP['ZONE'] = tbl_BP['ZONE'].str.replace('46', '11')
tbl_BP['ZONE'] = tbl_BP['ZONE'].replace('nan', np.nan).astype(float).fillna(0).astype(int)
tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)

C:\Users\sliu\AppData\Local\Temp\ipykernel_6600\2846403398.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  tbl_BP['ZONE'] = tbl_BP['ZONE'].astype(str).str.replace('\.0$', '').astype(int)


# Step 3 - b. Rerate Begin
#====================UPS====================

#REDE (UPS NDA Early): commercial & residential, lb    

#RED (UPS NDA): commercial & residential, lb

#REDS (UPS NDA Saver): commercial & residential, lb

#2DAM (UPS 2DA A.M.): commercial & residential, lb

#BLUE (UPS 2DA): commercial & residential, lb

#ORNG (UPS 3DA): commercial & residential, lb

#GRND (UPS Ground Commercial): commercial only, lb

#GRES (UPS Ground Residential): residential only, lb

#SRPT<1 (UPS Surepost 1#>): commercial & residential, oz

#SRPT (UPS Surepost): commercial & residential, lb

#====================DHL====================

#DHLG (DHL SmartMail Parcel Plus Ground 1-25): commercial & residential, lb

#DHLG<1 (DHL SmartMail Parcel Ground < 1lb): commercial & residential, oz

#DHLE (DHL SmartMail Parcel Plus Expedited 1-25): commercial & residential, lb

#DHLE<1 (DHL SmartMail Parcel Expedited  < 1lb): commercial & residential, oz

#DHLEM (DHL SmartMail Parcel Expedited Max): commercial & residential, both oz and lb

#====================USPS====================

#USPSAG (USPS Auctane GRND): commercial & residential, lb

#USPSAP (USPS Auctane PM): commercial & residential, lb

#==========================MI===========================================

#MIPLE (MI Parcel Select Lightweight Expedited): commercial & residential, oz

#MIPH (MI Parcel Select Heavyweight): commercial & residential, lb

# ========= UPS & FedEx Section ==========

In [539]:
# add a column named DAS_CATEGORY to PLD with ups accessorial category DAS, DASE, RA, HI, or AK. One package (or row) can only have one single category.
# create UPS Das
xls = pd.ExcelFile('C:\\Users\\SLiu\\BD\\model\\RateQuote\\5.ZipToDas_2024.xlsx')
tbl_area_surcharge_UPS = pd.read_excel(xls, sheet_name='UPS')
tbl_area_surcharge_FedEx = pd.read_excel(xls, sheet_name='FedEx')

tbl_area_surcharge_UPS['Zip'] = tbl_area_surcharge_UPS['Zip'].astype(str)
tbl_area_surcharge_UPS['Zip'] = tbl_area_surcharge_UPS['Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(tbl_area_surcharge_UPS[["Zip", "Type"]], left_on='ZIP_CODE', right_on='Zip', how='left')
tbl_BP['DAS_CATEGORY_UPS'] = tbl_BP['Type']
tbl_BP = tbl_BP.drop('Type', axis=1)
tbl_BP = tbl_BP.drop('Zip', axis=1)
#tbl_BP.info()
#tbl_BP['ZONE'].unique()

In [540]:
# create FedEx Das
tbl_area_surcharge_FedEx['Zip'] = tbl_area_surcharge_FedEx['Zip'].astype(str)
tbl_area_surcharge_FedEx['Zip'] = tbl_area_surcharge_FedEx['Zip'].apply(lambda x: '0' + x if len(x) == 4 else ('00' + x if (len(x) == 3) else x))

tbl_BP = tbl_BP.merge(tbl_area_surcharge_FedEx[["Zip", "Type"]], left_on='ZIP_CODE', right_on='Zip', how='left')
tbl_BP['DAS_CATEGORY_FedEx'] = tbl_BP['Type']
tbl_BP = tbl_BP.drop('Type', axis=1)
tbl_BP = tbl_BP.drop('Zip', axis=1)
#tbl_BP.info()
#tbl_BP['ZONE'].unique()

# REDE (UPS NDA Early) 

In [541]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_1 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'REDE')]
tbl_PLD_original_REDE = tbl_BP.copy()

In [542]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_REDE['BILLED_WEIGHT_LB'] = tbl_PLD_original_REDE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)


#add Freight column to PLD
tbl_PLD_original_REDE['ZONE_UPS'] = tbl_PLD_original_REDE['ZONE_UPS'].astype(str)
tbl_PLD_original_REDE = tbl_PLD_original_REDE.merge(tbl_clientRates_1[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_REDE = tbl_PLD_original_REDE.rename(columns={'CLIENT_RATES': 'FRT_REDE'})
   

#add Residential Surcharge column to PLD
tbl_PLD_original_REDE['RES_REDE'] = tbl_PLD_original_REDE.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_REDE['DAS_REDE'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_REDE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_REDE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_REDE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')



#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_REDE = tbl_PLD_original_REDE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_REDE.rename(columns={'Domestic Air': 'FSC%_REDE'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_REDE['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_REDE['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_REDE['AHS_1'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                          AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_REDE['AHS_2'] = tbl_PLD_original_REDE.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_REDE['AHS_3'] = tbl_PLD_original_REDE.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_REDE['AHS_4'] = tbl_PLD_original_REDE.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_REDE['AHS_REDE'] = tbl_PLD_original_REDE['AHS_1'] + tbl_PLD_original_REDE['AHS_2'] + tbl_PLD_original_REDE['AHS_3'] + tbl_PLD_original_REDE['AHS_4']
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_REDE['TOTAL_REDE'] = round(tbl_PLD_original_REDE.apply(lambda row: 
                                                                  (1 + row['FSC%_REDE']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_REDE'] + row['RES_REDE'] + row['DAS_REDE'] + row['AHS_REDE']) if not pd.isna(row['FRT_REDE']) and row['FRT_REDE'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_REDE.to_csv('test.csv')



# 1STO (FedEx First Overnight)

In [120]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_1 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == '1STO')]
tbl_PLD_original_1STO = tbl_BP.copy()

In [121]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_1STO['BILLED_WEIGHT_LB'] = tbl_PLD_original_1STO.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_1STO['ZONE_FedEx'] = tbl_PLD_original_1STO['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_1STO['ZONE_FedEx'] = tbl_PLD_original_1STO['ZONE_FedEx'].astype(str)
tbl_PLD_original_1STO = tbl_PLD_original_1STO.merge(tbl_clientRates_1[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_1STO = tbl_PLD_original_1STO.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_1STO = tbl_PLD_original_1STO.rename(columns={'CLIENT_RATES': 'FRT_1STO'})


#add Residential Surcharge column to PLD
tbl_PLD_original_1STO['RES_1STO'] = tbl_PLD_original_1STO.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_1STO['DAS_1STO'] = tbl_PLD_original_1STO.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_1STO['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_1STO['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_1STO['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_1STO = tbl_PLD_original_1STO.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_1STO.rename(columns={'Domestic Air FedEx': 'FSC%_1STO'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_1STO['AHS_1STO_1'] = tbl_PLD_original_1STO.apply(calculate_ahs1, axis=1)
tbl_PLD_original_1STO['AHS_1STO_2'] = tbl_PLD_original_1STO.apply(calculate_ahs2, axis=1)
tbl_PLD_original_1STO['AHS_1STO'] = tbl_PLD_original_1STO['AHS_1STO_1'] + tbl_PLD_original_1STO['AHS_1STO_2']
tbl_PLD_original_1STO = tbl_PLD_original_1STO.drop(['AHS_1STO_1','AHS_1STO_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_1STO['TOTAL_1STO'] = round(tbl_PLD_original_1STO.apply(lambda row: 
                                                                  (1 + row['FSC%_1STO']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_1STO'] + row['RES_1STO'] + row['DAS_1STO'] + row['AHS_1STO']) if not pd.isna(row['FRT_1STO']) and row['FRT_1STO'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_1STO.to_csv('test.csv')



# RED (UPS NDA)

In [549]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_2 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'RED')]
tbl_PLD_original_RED = tbl_BP.copy()

In [550]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_RED['BILLED_WEIGHT_LB'] = tbl_PLD_original_RED.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_RED['ZONE_UPS'] = tbl_PLD_original_RED['ZONE_UPS'].astype(str)
tbl_PLD_original_RED = tbl_PLD_original_RED.merge(tbl_clientRates_2[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_RED = tbl_PLD_original_RED.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_RED = tbl_PLD_original_RED.rename(columns={'CLIENT_RATES': 'FRT_RED'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_RED['RES_RED'] = tbl_PLD_original_RED.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_RED['DAS_RED'] = tbl_PLD_original_RED.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_RED['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_RED['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_RED['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_RED = tbl_PLD_original_RED.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_RED.rename(columns={'Domestic Air': 'FSC%_RED'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_RED['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_RED['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_RED['AHS_1'] = tbl_PLD_original_RED.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_RED['AHS_2'] = tbl_PLD_original_RED.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_RED['AHS_3'] = tbl_PLD_original_RED.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_RED['AHS_4'] = tbl_PLD_original_RED.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_RED['AHS_RED'] = tbl_PLD_original_RED['AHS_1'] + tbl_PLD_original_RED['AHS_2'] + tbl_PLD_original_RED['AHS_3'] + tbl_PLD_original_RED['AHS_4']
tbl_PLD_original_RED = tbl_PLD_original_RED.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_RED['TOTAL_RED'] = round(tbl_PLD_original_RED.apply(lambda row: 
                                                                  (1 + row['FSC%_RED']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_RED'] + row['RES_RED'] + row['DAS_RED'] + row['AHS_RED']) if not pd.isna(row['FRT_RED']) and row['FRT_RED'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_RED.to_csv('test.csv')

# PROV (FedEx Next Day Air by 10a)

In [122]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_2 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'PROV')]
tbl_PLD_original_PROV = tbl_BP.copy()

In [123]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_PROV['BILLED_WEIGHT_LB'] = tbl_PLD_original_PROV.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_PROV['ZONE_FedEx'] = tbl_PLD_original_PROV['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_PROV['ZONE_FedEx'] = tbl_PLD_original_PROV['ZONE_FedEx'].astype(str)
tbl_PLD_original_PROV = tbl_PLD_original_PROV.merge(tbl_clientRates_2[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_PROV = tbl_PLD_original_PROV.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_PROV = tbl_PLD_original_PROV.rename(columns={'CLIENT_RATES': 'FRT_PROV'})


#add Residential Surcharge column to PLD
tbl_PLD_original_PROV['RES_PROV'] = tbl_PLD_original_PROV.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_PROV['DAS_PROV'] = tbl_PLD_original_PROV.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_PROV['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_PROV['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_PROV['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_PROV = tbl_PLD_original_PROV.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_PROV.rename(columns={'Domestic Air FedEx': 'FSC%_PROV'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_PROV['AHS_PROV_1'] = tbl_PLD_original_PROV.apply(calculate_ahs1, axis=1)
tbl_PLD_original_PROV['AHS_PROV_2'] = tbl_PLD_original_PROV.apply(calculate_ahs2, axis=1)
tbl_PLD_original_PROV['AHS_PROV'] = tbl_PLD_original_PROV['AHS_PROV_1'] + tbl_PLD_original_PROV['AHS_PROV_2']
tbl_PLD_original_PROV = tbl_PLD_original_PROV.drop(['AHS_PROV_1','AHS_PROV_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_PROV['TOTAL_PROV'] = round(tbl_PLD_original_PROV.apply(lambda row: 
                                                                  (1 + row['FSC%_PROV']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_PROV'] + row['RES_PROV'] + row['DAS_PROV'] + row['AHS_PROV']) if not pd.isna(row['FRT_PROV']) and row['FRT_PROV'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_PROV.to_csv('test.csv')



# REDS (UPS NDA Saver)

In [552]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_3 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'REDS')]
tbl_PLD_original_REDS = tbl_BP.copy()

In [553]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_REDS['BILLED_WEIGHT_LB'] = tbl_PLD_original_REDS.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_REDS['ZONE_UPS'] = tbl_PLD_original_REDS['ZONE_UPS'].astype(str)
tbl_PLD_original_REDS = tbl_PLD_original_REDS.merge(tbl_clientRates_3[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_REDS = tbl_PLD_original_REDS.rename(columns={'CLIENT_RATES': 'FRT_REDS'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_REDS['RES_REDS'] = tbl_PLD_original_REDS.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_REDS['DAS_REDS'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_REDS['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_REDS['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_REDS['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_REDS = tbl_PLD_original_REDS.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_REDS.rename(columns={'Domestic Air': 'FSC%_REDS'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_REDS['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_REDS['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_REDS['AHS_1'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_REDS['AHS_2'] = tbl_PLD_original_REDS.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_REDS['AHS_3'] = tbl_PLD_original_REDS.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_REDS['AHS_4'] = tbl_PLD_original_REDS.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_REDS['AHS_REDS'] = tbl_PLD_original_REDS['AHS_1'] + tbl_PLD_original_REDS['AHS_2'] + tbl_PLD_original_REDS['AHS_3'] + tbl_PLD_original_REDS['AHS_4']
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_REDS['TOTAL_REDS'] = round(tbl_PLD_original_REDS.apply(lambda row: 
                                                                  (1 + row['FSC%_REDS']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_REDS'] + row['RES_REDS'] + row['DAS_REDS'] + row['AHS_REDS']) if not pd.isna(row['FRT_REDS']) and row['FRT_REDS'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_REDS.to_csv('test.csv')

# STOV (FedEx Next Day Air by 3p)

In [124]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_3 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'STOV')]
tbl_PLD_original_STOV = tbl_BP.copy()

In [125]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_STOV['BILLED_WEIGHT_LB'] = tbl_PLD_original_STOV.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_STOV['ZONE_FedEx'] = tbl_PLD_original_STOV['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_STOV['ZONE_FedEx'] = tbl_PLD_original_STOV['ZONE_FedEx'].astype(str)
tbl_PLD_original_STOV = tbl_PLD_original_STOV.merge(tbl_clientRates_3[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_STOV = tbl_PLD_original_STOV.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_STOV = tbl_PLD_original_STOV.rename(columns={'CLIENT_RATES': 'FRT_STOV'})


#add Residential Surcharge column to PLD
tbl_PLD_original_STOV['RES_STOV'] = tbl_PLD_original_STOV.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_STOV['DAS_STOV'] = tbl_PLD_original_STOV.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_STOV['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_STOV['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_STOV['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_STOV = tbl_PLD_original_STOV.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_STOV.rename(columns={'Domestic Air FedEx': 'FSC%_STOV'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_STOV['AHS_STOV_1'] = tbl_PLD_original_STOV.apply(calculate_ahs1, axis=1)
tbl_PLD_original_STOV['AHS_STOV_2'] = tbl_PLD_original_STOV.apply(calculate_ahs2, axis=1)
tbl_PLD_original_STOV['AHS_STOV'] = tbl_PLD_original_STOV['AHS_STOV_1'] + tbl_PLD_original_STOV['AHS_STOV_2']
tbl_PLD_original_STOV = tbl_PLD_original_STOV.drop(['AHS_STOV_1','AHS_STOV_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_STOV['TOTAL_STOV'] = round(tbl_PLD_original_STOV.apply(lambda row: 
                                                                  (1 + row['FSC%_STOV']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_STOV'] + row['RES_STOV'] + row['DAS_STOV'] + row['AHS_STOV']) if not pd.isna(row['FRT_STOV']) and row['FRT_STOV'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_STOV.to_csv('test.csv')



# 2DAM (UPS 2DA A.M.)

In [555]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_4 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == '2DAM')]
tbl_PLD_original_2DAM = tbl_BP.copy()

In [556]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_2DAM['BILLED_WEIGHT_LB'] = tbl_PLD_original_2DAM.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_2DAM['ZONE_UPS'] = tbl_PLD_original_2DAM['ZONE_UPS'].astype(str)
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.merge(tbl_clientRates_4[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.rename(columns={'CLIENT_RATES': 'FRT_2DAM'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_2DAM['RES_2DAM'] = tbl_PLD_original_2DAM.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_2DAM['DAS_2DAM'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_2DAM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_2DAM['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_2DAM['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_2DAM.rename(columns={'Domestic Air': 'FSC%_2DAM'}, inplace=True)


#add AHS column to PLD
tbl_PLD_original_2DAM['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_2DAM['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_2DAM['AHS_1'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_2DAM['AHS_2'] = tbl_PLD_original_2DAM.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_2DAM['AHS_3'] = tbl_PLD_original_2DAM.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_2DAM['AHS_4'] = tbl_PLD_original_2DAM.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_2DAM['AHS_2DAM'] = tbl_PLD_original_2DAM['AHS_1'] + tbl_PLD_original_2DAM['AHS_2'] + tbl_PLD_original_2DAM['AHS_3'] + tbl_PLD_original_2DAM['AHS_4']
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_2DAM['TOTAL_2DAM'] = round(tbl_PLD_original_2DAM.apply(lambda row: 
                                                                  (1 + row['FSC%_2DAM']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_2DAM'] + row['RES_2DAM'] + row['DAS_2DAM'] + row['AHS_2DAM']) if not pd.isna(row['FRT_2DAM']) and row['FRT_2DAM'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_2DAM.to_csv('test.csv')

# 2AM (FedEx 2nd Day AM)

In [126]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_4 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == '2AM')]
tbl_PLD_original_2AM = tbl_BP.copy()

In [127]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_2AM['BILLED_WEIGHT_LB'] = tbl_PLD_original_2AM.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_2AM['ZONE_FedEx'] = tbl_PLD_original_2AM['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_2AM['ZONE_FedEx'] = tbl_PLD_original_2AM['ZONE_FedEx'].astype(str)
tbl_PLD_original_2AM = tbl_PLD_original_2AM.merge(tbl_clientRates_4[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_2AM = tbl_PLD_original_2AM.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_2AM = tbl_PLD_original_2AM.rename(columns={'CLIENT_RATES': 'FRT_2AM'})


#add Residential Surcharge column to PLD
tbl_PLD_original_2AM['RES_2AM'] = tbl_PLD_original_2AM.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_2AM['DAS_2AM'] = tbl_PLD_original_2AM.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_2AM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_2AM['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_2AM['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_2AM = tbl_PLD_original_2AM.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_2AM.rename(columns={'Domestic Air FedEx': 'FSC%_2AM'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_2AM['AHS_2AM_1'] = tbl_PLD_original_2AM.apply(calculate_ahs1, axis=1)
tbl_PLD_original_2AM['AHS_2AM_2'] = tbl_PLD_original_2AM.apply(calculate_ahs2, axis=1)
tbl_PLD_original_2AM['AHS_2AM'] = tbl_PLD_original_2AM['AHS_2AM_1'] + tbl_PLD_original_2AM['AHS_2AM_2']
tbl_PLD_original_2AM = tbl_PLD_original_2AM.drop(['AHS_2AM_1','AHS_2AM_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_2AM['TOTAL_2AM'] = round(tbl_PLD_original_2AM.apply(lambda row: 
                                                                  (1 + row['FSC%_2AM']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_2AM'] + row['RES_2AM'] + row['DAS_2AM'] + row['AHS_2AM']) if not pd.isna(row['FRT_2AM']) and row['FRT_2AM'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_2AM.to_csv('test.csv')



# BLUE (UPS 2DA)

In [570]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_5 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'BLUE')]
tbl_PLD_original_BLUE = tbl_BP.copy()

In [571]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_BLUE['BILLED_WEIGHT_LB'] = tbl_PLD_original_BLUE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_BLUE['ZONE_UPS'] = tbl_PLD_original_BLUE['ZONE_UPS'].astype(str)
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.merge(tbl_clientRates_5[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.rename(columns={'CLIENT_RATES': 'FRT_BLUE'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_BLUE['RES_BLUE'] = tbl_PLD_original_BLUE.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_BLUE['DAS_BLUE'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_BLUE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_BLUE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_BLUE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')

df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date

#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_BLUE.rename(columns={'Domestic Air': 'FSC%_BLUE'}, inplace=True)




#add AHS column to PLD
tbl_PLD_original_BLUE['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_BLUE['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_BLUE['AHS_1'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_BLUE['AHS_2'] = tbl_PLD_original_BLUE.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_BLUE['AHS_3'] = tbl_PLD_original_BLUE.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_BLUE['AHS_4'] = tbl_PLD_original_BLUE.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_BLUE['AHS_BLUE'] = tbl_PLD_original_BLUE['AHS_1'] + tbl_PLD_original_BLUE['AHS_2'] + tbl_PLD_original_BLUE['AHS_3'] + tbl_PLD_original_BLUE['AHS_4']
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_BLUE['TOTAL_BLUE'] = round(tbl_PLD_original_BLUE.apply(lambda row: 
                                                                  (1 + row['FSC%_BLUE']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_BLUE'] + row['RES_BLUE'] + row['DAS_BLUE'] + row['AHS_BLUE']) if not pd.isna(row['FRT_BLUE']) and row['FRT_BLUE'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_BLUE.to_csv('test.csv')

# 2DAY (FedEx 2 Day Air)

In [128]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_5 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == '2DAY')]
tbl_PLD_original_2DAY = tbl_BP.copy()

In [129]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_2DAY['BILLED_WEIGHT_LB'] = tbl_PLD_original_2DAY.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_2DAY['ZONE_FedEx'] = tbl_PLD_original_2DAY['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_2DAY['ZONE_FedEx'] = tbl_PLD_original_2DAY['ZONE_FedEx'].astype(str)
tbl_PLD_original_2DAY = tbl_PLD_original_2DAY.merge(tbl_clientRates_5[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_2DAY = tbl_PLD_original_2DAY.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_2DAY = tbl_PLD_original_2DAY.rename(columns={'CLIENT_RATES': 'FRT_2DAY'})


#add Residential Surcharge column to PLD
tbl_PLD_original_2DAY['RES_2DAY'] = tbl_PLD_original_2DAY.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_2DAY['DAS_2DAY'] = tbl_PLD_original_2DAY.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_2DAY['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_2DAY['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_2DAY['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_2DAY = tbl_PLD_original_2DAY.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_2DAY.rename(columns={'Domestic Air FedEx': 'FSC%_2DAY'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_2DAY['AHS_2DAY_1'] = tbl_PLD_original_2DAY.apply(calculate_ahs1, axis=1)
tbl_PLD_original_2DAY['AHS_2DAY_2'] = tbl_PLD_original_2DAY.apply(calculate_ahs2, axis=1)
tbl_PLD_original_2DAY['AHS_2DAY'] = tbl_PLD_original_2DAY['AHS_2DAY_1'] + tbl_PLD_original_2DAY['AHS_2DAY_2']
tbl_PLD_original_2DAY = tbl_PLD_original_2DAY.drop(['AHS_2DAY_1','AHS_2DAY_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_2DAY['TOTAL_2DAY'] = round(tbl_PLD_original_2DAY.apply(lambda row: 
                                                                  (1 + row['FSC%_2DAY']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_2DAY'] + row['RES_2DAY'] + row['DAS_2DAY'] + row['AHS_2DAY']) if not pd.isna(row['FRT_2DAY']) and row['FRT_2DAY'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_2DAY.to_csv('test.csv')



# ORNG (UPS 3DA)

In [573]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_6 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'ORNG')]
tbl_PLD_original_ORNG = tbl_BP.copy()

In [574]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_ORNG['BILLED_WEIGHT_LB'] = tbl_PLD_original_ORNG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_ORNG['ZONE_UPS'] = tbl_PLD_original_ORNG['ZONE_UPS'].astype(str)
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.merge(tbl_clientRates_6[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.rename(columns={'CLIENT_RATES': 'FRT_ORNG'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_ORNG['RES_ORNG'] = tbl_PLD_original_ORNG.apply(lambda row: Resi_Surcharge_Air if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_ORNG['DAS_ORNG'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                            DAS_Comm_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_ORNG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_ORNG['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_ORNG['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground'], axis=1)
tbl_PLD_original_ORNG.rename(columns={'Domestic Air': 'FSC%_ORNG'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_ORNG['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_ORNG['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_ORNG['AHS_1'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_ORNG['AHS_2'] = tbl_PLD_original_ORNG.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_ORNG['AHS_3'] = tbl_PLD_original_ORNG.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_ORNG['AHS_4'] = tbl_PLD_original_ORNG.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_ORNG['AHS_ORNG'] = tbl_PLD_original_ORNG['AHS_1'] + tbl_PLD_original_ORNG['AHS_2'] + tbl_PLD_original_ORNG['AHS_3'] + tbl_PLD_original_ORNG['AHS_4']
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_ORNG['TOTAL_ORNG'] = round(tbl_PLD_original_ORNG.apply(lambda row: 
                                                                  (1 + row['FSC%_ORNG']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_ORNG'] + row['RES_ORNG'] + row['DAS_ORNG'] + row['AHS_ORNG']) if not pd.isna(row['FRT_ORNG']) and row['FRT_ORNG'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_ORNG.to_csv('test.csv')

# EXSA (FedEx 3 Day Air)

In [130]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_6 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'EXSA')]
tbl_PLD_original_EXSA = tbl_BP.copy()

In [131]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_EXSA['BILLED_WEIGHT_LB'] = tbl_PLD_original_EXSA.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_EXSA['ZONE_FedEx'] = tbl_PLD_original_EXSA['ZONE_FedEx'].replace('17', '9')
tbl_PLD_original_EXSA['ZONE_FedEx'] = tbl_PLD_original_EXSA['ZONE_FedEx'].astype(str)
tbl_PLD_original_EXSA = tbl_PLD_original_EXSA.merge(tbl_clientRates_6[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_EXSA = tbl_PLD_original_EXSA.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_EXSA = tbl_PLD_original_EXSA.rename(columns={'CLIENT_RATES': 'FRT_EXSA'})


#add Residential Surcharge column to PLD
tbl_PLD_original_EXSA['RES_EXSA'] = tbl_PLD_original_EXSA.apply(lambda row: Resi_Surcharge_Air_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_EXSA['DAS_EXSA'] = tbl_PLD_original_EXSA.apply(lambda row: 
                                                            0 if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (0 if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Air_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_EXSA['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_EXSA['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_EXSA['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_EXSA = tbl_PLD_original_EXSA.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Ground FedEx'], axis=1)
tbl_PLD_original_EXSA.rename(columns={'Domestic Air FedEx': 'FSC%_EXSA'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_EXSA['AHS_EXSA_1'] = tbl_PLD_original_EXSA.apply(calculate_ahs1, axis=1)
tbl_PLD_original_EXSA['AHS_EXSA_2'] = tbl_PLD_original_EXSA.apply(calculate_ahs2, axis=1)
tbl_PLD_original_EXSA['AHS_EXSA'] = tbl_PLD_original_EXSA['AHS_EXSA_1'] + tbl_PLD_original_EXSA['AHS_EXSA_2']
tbl_PLD_original_EXSA = tbl_PLD_original_EXSA.drop(['AHS_EXSA_1','AHS_EXSA_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_EXSA['TOTAL_EXSA'] = round(tbl_PLD_original_EXSA.apply(lambda row: 
                                                                  (1 + row['FSC%_EXSA']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_EXSA'] + row['RES_EXSA'] + row['DAS_EXSA'] + row['AHS_EXSA']) if not pd.isna(row['FRT_EXSA']) and row['FRT_EXSA'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_EXSA.to_csv('test.csv')



# GRND (UPS Ground Commercial)

In [578]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_7 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'GRND')]
tbl_PLD_original_GRND = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == False)].copy()

In [579]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_GRND['BILLED_WEIGHT_LB'] = tbl_PLD_original_GRND.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_GRND['ZONE_UPS'] = tbl_PLD_original_GRND['ZONE_UPS'].astype(str)
tbl_PLD_original_GRND = tbl_PLD_original_GRND.merge(tbl_clientRates_7[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_GRND = tbl_PLD_original_GRND.rename(columns={'CLIENT_RATES': 'FRT_GRND'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_GRND['RES_GRND'] = tbl_PLD_original_GRND.apply(lambda row: Resi_Surcharge_Ground if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_GRND['DAS_GRND'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                            DAS_Comm_Ground if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_GRND['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_GRND['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_GRND['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_GRND = tbl_PLD_original_GRND.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_GRND.rename(columns={'Ground': 'FSC%_GRND'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_GRND['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_GRND['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_GRND['AHS_1'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_GRND['AHS_2'] = tbl_PLD_original_GRND.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_GRND['AHS_3'] = tbl_PLD_original_GRND.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_GRND['AHS_4'] = tbl_PLD_original_GRND.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_GRND['AHS_GRND'] = tbl_PLD_original_GRND['AHS_1'] + tbl_PLD_original_GRND['AHS_2'] + tbl_PLD_original_GRND['AHS_3'] + tbl_PLD_original_GRND['AHS_4']
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_GRND['TOTAL_GRND'] = round(tbl_PLD_original_GRND.apply(lambda row: 
                                                                  (1 + row['FSC%_GRND']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_GRND'] + row['RES_GRND'] + row['DAS_GRND'] + row['AHS_GRND']) if not pd.isna(row['FRT_GRND']) and row['FRT_GRND'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_GRND.to_csv('test.csv')

# GRNDF (FedEx Ground)

In [132]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_7 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'GRNDF')]
tbl_PLD_original_GRNDF = tbl_BP.copy()

In [133]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_GRNDF['BILLED_WEIGHT_LB'] = tbl_PLD_original_GRNDF.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_GRNDF['ZONE_FedEx'] = tbl_PLD_original_GRNDF['ZONE_FedEx'].astype(str)
tbl_PLD_original_GRNDF = tbl_PLD_original_GRNDF.merge(tbl_clientRates_7[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_GRNDF = tbl_PLD_original_GRNDF.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_GRNDF = tbl_PLD_original_GRNDF.rename(columns={'CLIENT_RATES': 'FRT_GRNDF'})


#add Residential Surcharge column to PLD
tbl_PLD_original_GRNDF['RES_GRNDF'] = tbl_PLD_original_GRNDF.apply(lambda row: Resi_Surcharge_Ground_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_GRNDF['DAS_GRNDF'] = tbl_PLD_original_GRNDF.apply(lambda row: 
                                                            DAS_Comm_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_GRNDF['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_GRNDF['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_GRNDF['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_GRNDF = tbl_PLD_original_GRNDF.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air FedEx'], axis=1)
tbl_PLD_original_GRNDF.rename(columns={'Ground FedEx': 'FSC%_GRNDF'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_GRNDF['AHS_GRNDF_1'] = tbl_PLD_original_GRNDF.apply(calculate_ahs1, axis=1)
tbl_PLD_original_GRNDF['AHS_GRNDF_2'] = tbl_PLD_original_GRNDF.apply(calculate_ahs2, axis=1)
tbl_PLD_original_GRNDF['AHS_GRNDF'] = tbl_PLD_original_GRNDF['AHS_GRNDF_1'] + tbl_PLD_original_GRNDF['AHS_GRNDF_2']
tbl_PLD_original_GRNDF = tbl_PLD_original_GRNDF.drop(['AHS_GRNDF_1','AHS_GRNDF_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_GRNDF['TOTAL_GRNDF'] = round(tbl_PLD_original_GRNDF.apply(lambda row: 
                                                                  (1 + row['FSC%_GRNDF']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_GRNDF'] + row['RES_GRNDF'] + row['DAS_GRNDF'] + row['AHS_GRNDF']) if not pd.isna(row['FRT_GRNDF']) and row['FRT_GRNDF'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_GRNDF.to_csv('test.csv')



# GRES (UPS Ground Residential)

In [581]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_8 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'GRES')]
tbl_PLD_original_GRES = tbl_BP[(tbl_BP['RESIDENTIAL_FLAG'] == True)].copy()

In [582]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_GRES['BILLED_WEIGHT_LB'] = tbl_PLD_original_GRES.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups)), axis=1)

#add Freight column to PLD
tbl_PLD_original_GRES['ZONE_UPS'] = tbl_PLD_original_GRES['ZONE_UPS'].astype(str)
tbl_PLD_original_GRES = tbl_PLD_original_GRES.merge(tbl_clientRates_8[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_GRES = tbl_PLD_original_GRES.rename(columns={'CLIENT_RATES': 'FRT_GRES'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_GRES['RES_GRES'] = tbl_PLD_original_GRES.apply(lambda row: Resi_Surcharge_Ground if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_GRES['DAS_GRES'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                            DAS_Comm_Ground if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground if (row['DAS_CATEGORY_UPS'] == 'DASUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground if (row['DAS_CATEGORY_UPS'] == 'DASEUPS' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_GRES['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_GRES['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_GRES['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_GRES = tbl_PLD_original_GRES.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_GRES.rename(columns={'Ground': 'FSC%_GRES'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_GRES['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_GRES['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_GRES['AHS_1'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_GRES['AHS_2'] = tbl_PLD_original_GRES.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_GRES['AHS_3'] = tbl_PLD_original_GRES.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_GRES['AHS_4'] = tbl_PLD_original_GRES.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_GRES['AHS_GRES'] = tbl_PLD_original_GRES['AHS_1'] + tbl_PLD_original_GRES['AHS_2'] + tbl_PLD_original_GRES['AHS_3'] + tbl_PLD_original_GRES['AHS_4']
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_GRES['TOTAL_GRES'] = round(tbl_PLD_original_GRES.apply(lambda row: 
                                                                  (1 + row['FSC%_GRES']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_GRES'] + row['RES_GRES'] + row['DAS_GRES'] + row['AHS_GRES']) if not pd.isna(row['FRT_GRES']) and row['FRT_GRES'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_GRES.to_csv('test.csv')

# HOME (FedEx Residential Ground)

In [134]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_8 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'HOME')]
tbl_PLD_original_HOME = tbl_BP.copy()

In [135]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_HOME['BILLED_WEIGHT_LB'] = tbl_PLD_original_HOME.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)


#add Freight column to PLD
tbl_PLD_original_HOME['ZONE_FedEx'] = tbl_PLD_original_HOME['ZONE_FedEx'].astype(str)
tbl_PLD_original_HOME = tbl_PLD_original_HOME.merge(tbl_clientRates_8[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_HOME = tbl_PLD_original_HOME.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_HOME = tbl_PLD_original_HOME.rename(columns={'CLIENT_RATES': 'FRT_HOME'})


#add Residential Surcharge column to PLD
tbl_PLD_original_HOME['RES_HOME'] = tbl_PLD_original_HOME.apply(lambda row: Resi_Surcharge_Ground_FedEx if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_HOME['DAS_HOME'] = tbl_PLD_original_HOME.apply(lambda row: 
                                                            DAS_Comm_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_HOME['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_HOME['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_HOME['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_HOME = tbl_PLD_original_HOME.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air FedEx'], axis=1)
tbl_PLD_original_HOME.rename(columns={'Ground FedEx': 'FSC%_HOME'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_HOME['AHS_HOME_1'] = tbl_PLD_original_HOME.apply(calculate_ahs1, axis=1)
tbl_PLD_original_HOME['AHS_HOME_2'] = tbl_PLD_original_HOME.apply(calculate_ahs2, axis=1)
tbl_PLD_original_HOME['AHS_HOME'] = tbl_PLD_original_HOME['AHS_HOME_1'] + tbl_PLD_original_HOME['AHS_HOME_2']
tbl_PLD_original_HOME = tbl_PLD_original_HOME.drop(['AHS_HOME_1','AHS_HOME_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_HOME['TOTAL_HOME'] = round(tbl_PLD_original_HOME.apply(lambda row: 
                                                                  (1 + row['FSC%_HOME']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_HOME'] + row['RES_HOME'] + row['DAS_HOME'] + row['AHS_HOME']) if not pd.isna(row['FRT_HOME']) and row['FRT_HOME'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_HOME.to_csv('test.csv')



# SRPT<1 (UPS Surepost 1#>)

In [586]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_9 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SRPT<1')]
tbl_PLD_original_SRPT1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [587]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SRPT1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_SRPT1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups_srpt))*16 if (row['CUBIC_INCH'] >= 3456) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_SRPT1['ZONE_UPS'] = tbl_PLD_original_SRPT1['ZONE_UPS'].astype(str)
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.merge(tbl_clientRates_9[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE_UPS'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.rename(columns={'CLIENT_RATES': 'FRT_SRPT1'})
   
    
#add Residential Surcharge column to PLD
tbl_PLD_original_SRPT1['RES_SRPT1'] = tbl_PLD_original_SRPT1.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] == True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SRPT1['DAS_SRPT1'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                            DAS_SurePost if (row['DAS_CATEGORY_UPS'] == 'DASUPS') else 
                                                           (DAS_Extended_SurePost if (row['DAS_CATEGORY_UPS'] == 'DASEUPS') else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))), axis=1)



#add fuel surcharge % column to PLD
tbl_PLD_original_SRPT1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SRPT1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SRPT1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_SRPT1.rename(columns={'Ground': 'FSC%_SRPT1'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_SRPT1['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_SRPT1['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_SRPT1['AHS_1'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_SRPT1['AHS_2'] = tbl_PLD_original_SRPT1.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT1['AHS_3'] = tbl_PLD_original_SRPT1.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_SRPT1['AHS_4'] = tbl_PLD_original_SRPT1.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT1['AHS_SRPT1'] = tbl_PLD_original_SRPT1['AHS_1'] + tbl_PLD_original_SRPT1['AHS_2'] + tbl_PLD_original_SRPT1['AHS_3'] + tbl_PLD_original_SRPT1['AHS_4']
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SRPT1['TOTAL_SRPT1'] = round(tbl_PLD_original_SRPT1.apply(lambda row: 
                                                                  (1 + row['FSC%_SRPT1']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_SRPT1'] + row['RES_SRPT1'] + row['DAS_SRPT1'] + row['AHS_SRPT1']) if not pd.isna(row['FRT_SRPT1']) and row['FRT_SRPT1'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SRPT1.to_csv('test.csv')

# SPST<1 (FedEx Deferred Ground <1)

In [452]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_9 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SPST<1')]
tbl_PLD_original_SPST1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [453]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SPST1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_SPST1.apply(lambda row: 
                                                                          max(np.ceil(row['ACTUAL_WEIGHT']*16), np.ceil((row['CUBIC_INCH']/dim_factor_fedex_spst)*16)) if (row['CUBIC_INCH'] < 3456) else 
                                                                          max(np.ceil(row['ACTUAL_WEIGHT']*16), np.ceil((row['CUBIC_INCH']/dim_factor_fedex)*16)), axis=1)


#add Freight column to PLD
tbl_PLD_original_SPST1['ZONE_FedEx'] = tbl_PLD_original_SPST1['ZONE_FedEx'].astype(str)
tbl_PLD_original_SPST1 = tbl_PLD_original_SPST1.merge(tbl_clientRates_9[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE_FedEx'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_SPST1 = tbl_PLD_original_SPST1.drop(['WEIGHT_OZ', 'ZONE'], axis=1)
tbl_PLD_original_SPST1 = tbl_PLD_original_SPST1.rename(columns={'CLIENT_RATES': 'FRT_SPST1'})



In [455]:

#add Residential Surcharge column to PLD
tbl_PLD_original_SPST1['RES_SPST1'] = tbl_PLD_original_SPST1.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] == True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SPST1['DAS_SPST1'] = tbl_PLD_original_SPST1.apply(lambda row: 
                                                            DAS_Comm_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_SPST1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SPST1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SPST1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_SPST1 = tbl_PLD_original_SPST1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air FedEx'], axis=1)
tbl_PLD_original_SPST1.rename(columns={'Ground FedEx': 'FSC%_SPST1'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_SPST1['AHS_SPST1_1'] = tbl_PLD_original_SPST1.apply(calculate_ahs1, axis=1)
tbl_PLD_original_SPST1['AHS_SPST1_2'] = tbl_PLD_original_SPST1.apply(calculate_ahs2, axis=1)
tbl_PLD_original_SPST1['AHS_SPST1'] = tbl_PLD_original_SPST1['AHS_SPST1_1'] + tbl_PLD_original_SPST1['AHS_SPST1_2']
tbl_PLD_original_SPST1 = tbl_PLD_original_SPST1.drop(['AHS_SPST1_1','AHS_SPST1_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SPST1['TOTAL_SPST1'] = round(tbl_PLD_original_SPST1.apply(lambda row: 
                                                                  (1 + row['FSC%_SPST1']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_SPST1'] + row['RES_SPST1'] + row['DAS_SPST1'] + row['AHS_SPST1']) if not pd.isna(row['FRT_SPST1']) and row['FRT_SPST1'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SPST1.to_csv('test.csv')



# SRPT (UPS Surepost)

In [589]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_10 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SRPT')]
tbl_PLD_original_SRPT = tbl_BP.copy()

In [590]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SRPT['BILLED_WEIGHT_LB'] = tbl_PLD_original_SRPT.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_ups_srpt)) if (row['CUBIC_INCH'] >= 3456) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)




#add Freight column to PLD
tbl_PLD_original_SRPT['ZONE_UPS'] = tbl_PLD_original_SRPT['ZONE_UPS'].astype(str)
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.merge(tbl_clientRates_10[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_UPS'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.rename(columns={'CLIENT_RATES': 'FRT_SRPT'})
   

#add Residential Surcharge column to PLD
tbl_PLD_original_SRPT['RES_SRPT'] = tbl_PLD_original_SRPT.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] == True else 0, axis=1)

#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SRPT['DAS_SRPT'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                            DAS_SurePost if (row['DAS_CATEGORY_UPS'] == 'DASUPS') else 
                                                           (DAS_Extended_SurePost if (row['DAS_CATEGORY_UPS'] == 'DASEUPS') else 
                                                           (Remote_Area if (row['DAS_CATEGORY_UPS'] == 'RAUPS') else 
                                                           (Remote_Area_Alaska if (row['DAS_CATEGORY_UPS'] == 'AKUPS') else
                                                           (Remote_Area_Hawaii if (row['DAS_CATEGORY_UPS'] == 'HIUPS') else 0)))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_SRPT['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SRPT['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SRPT['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(UPS_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Ground column as FSC. 
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air'], axis=1)
tbl_PLD_original_SRPT.rename(columns={'Ground': 'FSC%_SRPT'}, inplace=True)



#add AHS column to PLD
tbl_PLD_original_SRPT['ZONE_UPS'] = pd.to_numeric(tbl_PLD_original_SRPT['ZONE_UPS'], errors='coerce').fillna(0).astype(int)

tbl_PLD_original_SRPT['AHS_1'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                           AH_Girth_2_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] == 2) else 
                                                           (AH_Girth_3_4_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else
                                                           (AH_Girth_5_up_UPS if (row['GIRTH_AND_L'] > 105 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                                    
tbl_PLD_original_SRPT['AHS_2'] = tbl_PLD_original_SRPT.apply(lambda row: 
                                                           AH_L_2_UPS if (row['L'] > 48 and row['ZONE_UPS'] == 2) else 
                                                           (AH_L_3_4_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_L_5_up_UPS if (row['L'] > 48 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT['AHS_3'] = tbl_PLD_original_SRPT.apply(lambda row:
                                                           AH_W_2_UPS if (row['W'] > 30 and row['ZONE_UPS'] == 2) else 
                                                           (AH_W_3_4_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_W_5_up_UPS if (row['W'] > 30 and row['ZONE_UPS'] > 4) else 0)), axis=1)
                                        
tbl_PLD_original_SRPT['AHS_4'] = tbl_PLD_original_SRPT.apply(lambda row:
                                                           AH_WGT_2_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] == 2) else 
                                                           (AH_WGT_3_4_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 2 and row['ZONE_UPS'] < 5) else 
                                                           (AH_WGT_5_up_UPS if (row['ACTUAL_WEIGHT'] > 50 and row['ZONE_UPS'] > 4) else 0)), axis=1)

tbl_PLD_original_SRPT['AHS_SRPT'] = tbl_PLD_original_SRPT['AHS_1'] + tbl_PLD_original_SRPT['AHS_2'] + tbl_PLD_original_SRPT['AHS_3'] + tbl_PLD_original_SRPT['AHS_4']
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop(['AHS_1','AHS_2','AHS_3','AHS_4'], axis=1)

#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SRPT['TOTAL_SRPT'] = round(tbl_PLD_original_SRPT.apply(lambda row: 
                                                                  (1 + row['FSC%_SRPT']*(1-BarrettDisc_Fsc_UPS)) * (row['FRT_SRPT'] + row['RES_SRPT'] + row['DAS_SRPT'] + row['AHS_SRPT']) if not pd.isna(row['FRT_SRPT']) and row['FRT_SRPT'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SRPT.to_csv('test.csv')

# SPST (FedEx Deferred Ground)

In [518]:
#Get the right client rate and the corresponding PLD
tbl_clientRates_10 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'SPST')]
tbl_PLD_original_SPST = tbl_BP.copy()

In [519]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_SPST['BILLED_WEIGHT_LB'] = tbl_PLD_original_SPST.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex_spst)) if (row['CUBIC_INCH'] < 3456) else 
                                                                                    max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_fedex)), axis=1)

#add Freight column to PLD
tbl_PLD_original_SPST['ZONE_FedEx'] = tbl_PLD_original_SPST['ZONE_FedEx'].astype(str)
tbl_PLD_original_SPST = tbl_PLD_original_SPST.merge(tbl_clientRates_10[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE_FedEx'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_SPST = tbl_PLD_original_SPST.drop(['WEIGHT_LB', 'ZONE'], axis=1)
tbl_PLD_original_SPST = tbl_PLD_original_SPST.rename(columns={'CLIENT_RATES': 'FRT_SPST'})


#add Residential Surcharge column to PLD
tbl_PLD_original_SPST['RES_SPST'] = tbl_PLD_original_SPST.apply(lambda row: 0 if row['RESIDENTIAL_FLAG'] is True else 0, axis=1)


#add DAS column to PLD, Pay attention to the flags under Residential Flag
tbl_PLD_original_SPST['DAS_SPST'] = tbl_PLD_original_SPST.apply(lambda row: 
                                                            DAS_Comm_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is False) else 
                                                           (DAS_Comm_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is False) else
                                                           (DAS_Resi_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (DAS_Resi_Extended_Ground_FedEx if (row['DAS_CATEGORY_FedEx'] == 'DASEFDX' and row['RESIDENTIAL_FLAG'] is True) else 
                                                           (Remote_Area_FedEx if (row['DAS_CATEGORY_FedEx'] == 'RAFDX') else 
                                                           (Remote_Area_Alaska_FedEx if (row['DAS_CATEGORY_FedEx'] == 'AKFDX') else
                                                           (Remote_Area_Hawaii_FedEx if (row['DAS_CATEGORY_FedEx'] == 'HIFDX') else 0)))))), axis=1)


#add fuel surcharge % column to PLD
tbl_PLD_original_SPST['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_SPST['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_SPST['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(FedEx_FSC, how='cross')


#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_SPST = tbl_PLD_original_SPST.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END', 'Domestic Air FedEx'], axis=1)
tbl_PLD_original_SPST.rename(columns={'Ground FedEx': 'FSC%_SPST'}, inplace=True)


#add AHS column to PLD
# Define the conditions and their corresponding values
def calculate_ahs1(row):
    if any([row['GIRTH_AND_L'] > 105, row['L'] > 48, row['W'] > 30]):
        if row['ZONE_FedEx'] == '2':
            return AH_Dim_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_Dim_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_Dim_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_Dim_7_up_FedEx
    else:
        return 0

def calculate_ahs2(row):
    if row['ACTUAL_WEIGHT'] > 50:
        if row['ZONE_FedEx'] == '2':
            return AH_WGT_2_FedEx
        elif row['ZONE_FedEx'] in ('3','4'):
            return AH_WGT_3_4_FedEx
        elif row['ZONE_FedEx'] in ('5','6'):
            return AH_WGT_5_6_FedEx
        elif row['ZONE_FedEx'] in ('7','8','9','10','11','12','17','26','9','10','17','99'):
            return AH_WGT_7_up_FedEx

    else:
        return 0
        
# Apply the function to the DataFrame
tbl_PLD_original_SPST['AHS_SPST_1'] = tbl_PLD_original_SPST.apply(calculate_ahs1, axis=1)
tbl_PLD_original_SPST['AHS_SPST_2'] = tbl_PLD_original_SPST.apply(calculate_ahs2, axis=1)
tbl_PLD_original_SPST['AHS_SPST'] = tbl_PLD_original_SPST['AHS_SPST_1'] + tbl_PLD_original_SPST['AHS_SPST_2']
tbl_PLD_original_SPST = tbl_PLD_original_SPST.drop(['AHS_SPST_1','AHS_SPST_2'], axis=1)


#Get Total rate. Set total rate to null if the freight rate is null and/or some package parameters are out of carrier handling capability.  
tbl_PLD_original_SPST['TOTAL_SPST'] = round(tbl_PLD_original_SPST.apply(lambda row: 
                                                                  (1 + row['FSC%_SPST']*(1-BarrettDisc_Fsc_FedEx)) * (row['FRT_SPST'] + row['RES_SPST'] + row['DAS_SPST'] + row['AHS_SPST']) if not pd.isna(row['FRT_SPST']) and row['FRT_SPST'] != '' else np.nan, axis=1),2)
#tbl_PLD_original_SPST.to_csv('test.csv')



# DHLG (DHL SmartMail Parcel Plus Ground)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound  
5123556: Montebello -> 90640


# DHLG

In [384]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_11 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLG_5122893')]
tbl_PLD_original_DHLG = tbl_BP.copy()

In [385]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLG['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLG['ZONE'] = tbl_PLD_original_DHLG['ZONE'].astype(str)
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.merge(tbl_clientRates_11[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.rename(columns={'CLIENT_RATES': 'FRT_DHLG'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLG['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()



# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLG.rename(columns={'FSC': 'FSC%_DHLG'}, inplace=True)
tbl_PLD_original_DHLG['FSC_DHLG'] = tbl_PLD_original_DHLG.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLG'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLG']), axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLG['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLG['NQD_DHLG_1'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)

#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)
tbl_PLD_original_DHLG['NQD_DHLG_2'] = tbl_PLD_original_DHLG.apply(lambda row:
                                                           np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)


#longest side_1
tbl_PLD_original_DHLG['NQD_DHLG_3'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLG['NQD_DHLG_4'] = tbl_PLD_original_DHLG.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLG['NQD_DHLG_5'] = tbl_PLD_original_DHLG.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLG['NQD_DHLG_OTHER'] = tbl_PLD_original_DHLG['NQD_DHLG_1'] + tbl_PLD_original_DHLG['NQD_DHLG_3'] + tbl_PLD_original_DHLG['NQD_DHLG_4'] + tbl_PLD_original_DHLG['NQD_DHLG_5']
tbl_PLD_original_DHLG['NQD_DHLG_GIRTH_AND_L'] = tbl_PLD_original_DHLG['NQD_DHLG_2']
tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop(['NQD_DHLG_1','NQD_DHLG_2','NQD_DHLG_3','NQD_DHLG_4','NQD_DHLG_5'], axis=1)


#Get Total
tbl_PLD_original_DHLG['TOTAL_DHLG'] = round(tbl_PLD_original_DHLG.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLG'] == '') | (pd.isna(row['FRT_DHLG'])) | (row['NQD_DHLG_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLG_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLG'] + (row['FSC_DHLG'] + row['NQD_DHLG_OTHER'] + row['NQD_DHLG_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)


#tbl_PLD_original_DHLG.to_csv('test.csv')


# DHLG<1 (DHL SmartMail Parcel Ground)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLG<1

In [460]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_12 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLG<1_5122739')]
tbl_PLD_original_DHLG1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [461]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLG1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLG1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_DHLG1['ZONE'] = tbl_PLD_original_DHLG1['ZONE'].astype(str)
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.merge(tbl_clientRates_12[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.rename(columns={'CLIENT_RATES': 'FRT_DHLG1'})

#add fuel surcharge % column to PLD
tbl_PLD_original_DHLG1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLG1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLG1.rename(columns={'FSC': 'FSC%_DHLG1'}, inplace=True)
tbl_PLD_original_DHLG1['FSC_DHLG1'] = tbl_PLD_original_DHLG1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLG1'], axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLG1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLG1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLG1['NQD_DHLG1_1'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

tbl_PLD_original_DHLG1['NQD_DHLG1_2'] = tbl_PLD_original_DHLG1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)


#longest side_1
tbl_PLD_original_DHLG1['NQD_DHLG1_3'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLG1['NQD_DHLG1_4'] = tbl_PLD_original_DHLG1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLG1['NQD_DHLG1_5'] = tbl_PLD_original_DHLG1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLG1['NQD_DHLG1_OTHER'] = tbl_PLD_original_DHLG1['NQD_DHLG1_1'] + tbl_PLD_original_DHLG1['NQD_DHLG1_3'] + tbl_PLD_original_DHLG1['NQD_DHLG1_4'] + tbl_PLD_original_DHLG1['NQD_DHLG1_5']
tbl_PLD_original_DHLG1['NQD_DHLG1_GIRTH_AND_L'] = tbl_PLD_original_DHLG1['NQD_DHLG1_2']
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop(['NQD_DHLG1_1','NQD_DHLG1_2','NQD_DHLG1_3','NQD_DHLG1_4','NQD_DHLG1_5'], axis=1)


#Get Total
tbl_PLD_original_DHLG1['TOTAL_DHLG1'] = round(tbl_PLD_original_DHLG1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLG1'] == '') | (pd.isna(row['FRT_DHLG1'])) | (row['NQD_DHLG1_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLG1_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLG1'] + (row['FSC_DHLG1'] + row['NQD_DHLG1_OTHER'] + row['NQD_DHLG1_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLG1.to_csv('test.csv')

# DHLE (DHL SmartMail Parcel Plus Expedited)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

In [136]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_13 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLE_5122893')]
tbl_PLD_original_DHLE = tbl_BP.copy()

In [137]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLE['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLE['ZONE'] = tbl_PLD_original_DHLE['ZONE'].astype(str)
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.merge(tbl_clientRates_13[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.rename(columns={'CLIENT_RATES': 'FRT_DHLE'})



#add fuel surcharge % column to PLD
tbl_PLD_original_DHLE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLE['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLE.rename(columns={'FSC': 'FSC%_DHLE'}, inplace=True)
tbl_PLD_original_DHLE['FSC_DHLE'] = tbl_PLD_original_DHLE.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLE'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLE']), axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLE['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLE['NQD_DHLE_1'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)
tbl_PLD_original_DHLE['NQD_DHLE_2'] = tbl_PLD_original_DHLE.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)


#longest side_1
tbl_PLD_original_DHLE['NQD_DHLE_3'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLE['NQD_DHLE_4'] = tbl_PLD_original_DHLE.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLE['NQD_DHLE_5'] = tbl_PLD_original_DHLE.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLE['NQD_DHLE_OTHER'] = tbl_PLD_original_DHLE['NQD_DHLE_1'] + tbl_PLD_original_DHLE['NQD_DHLE_3'] + tbl_PLD_original_DHLE['NQD_DHLE_4'] + tbl_PLD_original_DHLE['NQD_DHLE_5']
tbl_PLD_original_DHLE['NQD_DHLE_GIRTH_AND_L'] = tbl_PLD_original_DHLE['NQD_DHLE_2']
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop(['NQD_DHLE_1','NQD_DHLE_2','NQD_DHLE_3','NQD_DHLE_4','NQD_DHLE_5'], axis=1)


#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

tbl_PLD_original_DHLE['TOTAL_DHLE'] = round(tbl_PLD_original_DHLE.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLE'] == '') | (pd.isna(row['FRT_DHLE'])) | (row['NQD_DHLE_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLE_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLE'] + (row['FSC_DHLE'] + row['NQD_DHLE_OTHER'] + row['NQD_DHLE_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLE.to_csv('test.csv')

In [102]:
tbl_PLD_original_DHLE.to_excel('test.xlsx')

# DHLE<1 (DHL SmartMail Parcel Expedited)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLE<1

In [138]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_14 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLE<1_5122893')]
tbl_PLD_original_DHLE1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1)].copy()

In [139]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLE1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLE1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_DHLE1['ZONE'] = tbl_PLD_original_DHLE1['ZONE'].astype(str)
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.merge(tbl_clientRates_14[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.rename(columns={'CLIENT_RATES': 'FRT_DHLE1'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLE1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLE1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLE1.rename(columns={'FSC': 'FSC%_DHLE1'}, inplace=True)
tbl_PLD_original_DHLE1['FSC_DHLE1'] = tbl_PLD_original_DHLE1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLE1'], axis=1)


#add NQD Surcharge column to PLD
tbl_PLD_original_DHLE1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLE1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLE1['NQD_DHLE1_1'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

tbl_PLD_original_DHLE1['NQD_DHLE1_2'] = tbl_PLD_original_DHLE1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLE1['NQD_DHLE1_3'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLE1['NQD_DHLE1_4'] = tbl_PLD_original_DHLE1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLE1['NQD_DHLE1_5'] = tbl_PLD_original_DHLE1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLE1['NQD_DHLE1_OTHER'] = tbl_PLD_original_DHLE1['NQD_DHLE1_1'] + tbl_PLD_original_DHLE1['NQD_DHLE1_3'] + tbl_PLD_original_DHLE1['NQD_DHLE1_4'] + tbl_PLD_original_DHLE1['NQD_DHLE1_5']
tbl_PLD_original_DHLE1['NQD_DHLE1_GIRTH_AND_L'] = tbl_PLD_original_DHLE1['NQD_DHLE1_2']
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop(['NQD_DHLE1_1','NQD_DHLE1_2','NQD_DHLE1_3','NQD_DHLE1_4','NQD_DHLE1_5'], axis=1)


#Get Total
tbl_PLD_original_DHLE1['TOTAL_DHLE1'] = round(tbl_PLD_original_DHLE1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLE1'] == '') | (pd.isna(row['FRT_DHLE1'])) | (row['NQD_DHLE1_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLE1_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLE1'] + (row['FSC_DHLE1'] + row['NQD_DHLE1_OTHER'] + row['NQD_DHLE1_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)

#tbl_PLD_original_DHLE1.to_csv('test.csv')

In [114]:
tbl_PLD_original_DHLE1.to_excel('test.xlsx')

# DHLEM (DHL SmartMail Parcel Expedited Max)

5114358: Franklin (FRA) -> 02038_Outbound  
5122444: Somerset, no DHLEM (JER) -> 08873_Outbound  
5122723: 8150 Nail Olive Branch (MS2) -> 38654_Outbound  
5122739: Nail rd Olive branch (MS3) -> 38654_Outbound  
5122855: Garland TX (TX1) -> 75041_Outbound  
5122890: Oofos -> is/will be in Byhalia, MS)  
5122893: Memphis (TN2, TN3) -> 38141_Outbound  
5123280: (MAN)  
5123282: Bridgewater (MA5)  
5123283: Curtis Bay (GBM) -> 21226_Outbound

# DHLEM

In [153]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_15 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLEM_5122893')]
tbl_PLD_original_DHLEM1 = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 1)].copy()


In [154]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLEM1['BILLED_WEIGHT_OZ'] = tbl_PLD_original_DHLEM1.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl))*16 if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLEM1['ZONE'] = tbl_PLD_original_DHLEM1['ZONE'].astype(str)
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.merge(tbl_clientRates_15[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.rename(columns={'CLIENT_RATES': 'FRT_DHLEM'})


#add fuel surcharge % column to PLD
tbl_PLD_original_DHLEM1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM1['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
# when actual weight < 1, use actual weight non round up to get FSC, otherwise, use billed weight to get FSC
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLEM1['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLEM1.rename(columns={'FSC': 'FSC%_DHLEM'}, inplace=True)
tbl_PLD_original_DHLEM1['FSC_DHLEM'] = tbl_PLD_original_DHLEM1.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLEM'], axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLEM1['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM1['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim, billed weight is used to determine NQD, there is no <1 lb here
tbl_PLD_original_DHLEM1['NQD_DHLEM_1'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            2.5 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this, so we make it null)
#1 over 1lbs, under lbs
#and those < 1 lb will use lb to determine NQD
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16)
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use actual weight
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

tbl_PLD_original_DHLEM1['NQD_DHLEM_2'] = tbl_PLD_original_DHLEM1.apply(lambda row:
                                                            np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)


#longest side_1
tbl_PLD_original_DHLEM1['NQD_DHLEM_3'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            2.5 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['ACTUAL_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLEM1['NQD_DHLEM_4'] = tbl_PLD_original_DHLEM1.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLEM1['NQD_DHLEM_5'] = tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLEM1['NQD_DHLEM_OTHER'] = tbl_PLD_original_DHLEM1['NQD_DHLEM_1'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_3'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_4'] + tbl_PLD_original_DHLEM1['NQD_DHLEM_5']
tbl_PLD_original_DHLEM1['NQD_DHLEM_GIRTH_AND_L'] = tbl_PLD_original_DHLEM1['NQD_DHLEM_2']
tbl_PLD_original_DHLEM1 = tbl_PLD_original_DHLEM1.drop(['NQD_DHLEM_1','NQD_DHLEM_2','NQD_DHLEM_3','NQD_DHLEM_4','NQD_DHLEM_5'], axis=1)


#Get Total
tbl_PLD_original_DHLEM1['TOTAL_DHLEM'] = round(tbl_PLD_original_DHLEM1.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLEM'] == '') | (pd.isna(row['FRT_DHLEM'])) | (row['NQD_DHLEM_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLEM_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLEM'] + (row['FSC_DHLEM'] + row['NQD_DHLEM_OTHER'] + row['NQD_DHLEM_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)


In [155]:
#Select the rate to use, #Decide the part of PLD to rerate
#get client rate and a copy of PLD
tbl_clientRates_16 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'DHLEM_5122893')]
tbl_PLD_original_DHLEM = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] > 1)].copy()


In [156]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_DHLEM['BILLED_WEIGHT_LB'] = tbl_PLD_original_DHLEM.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_dhl)) if (row['CUBIC_INCH'] >= 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

#add Freight column to PLD
tbl_PLD_original_DHLEM['ZONE'] = tbl_PLD_original_DHLEM['ZONE'].astype(str)
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.merge(tbl_clientRates_16[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.rename(columns={'CLIENT_RATES': 'FRT_DHLEM'})



#add fuel surcharge % column to PLD
tbl_PLD_original_DHLEM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM['SHIP_DATE']).dt.date
# get unique values from column 'SHIP_DATE' and create a new dataframe
new_df = pd.DataFrame({'SHIP_DATE': tbl_PLD_original_DHLEM['SHIP_DATE'].unique()})
# use this new dataframe to merge to FSC
df_merge = new_df.merge(DHL_FSC, how='cross')
df_merge['SHIP_DATE'] = pd.to_datetime(df_merge['SHIP_DATE']).dt.date
#slice the merged dataframe to make sure ship_date is between the ship_data_start and end. Now we have a FSC file with Ship_date col to merge back to pld so that the pld can get FSC.
df_merge = df_merge.query('SHIP_DATE >= SHIP_DATE_START and SHIP_DATE <= SHIP_DATE_END')
df_merge = df_merge.drop_duplicates()
# Get Domestic Air column as FSC. 
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.merge(df_merge, on=['SHIP_DATE'], how='left').drop(['SHIP_DATE_START', 'SHIP_DATE_END'], axis=1)
tbl_PLD_original_DHLEM.rename(columns={'FSC': 'FSC%_DHLEM'}, inplace=True)
tbl_PLD_original_DHLEM['FSC_DHLEM'] = tbl_PLD_original_DHLEM.apply(lambda row: row['ACTUAL_WEIGHT']*row['FSC%_DHLEM'] if (row['ACTUAL_WEIGHT'] <= 1) else
                                                                          (row['BILLED_WEIGHT_LB']*row['FSC%_DHLEM']), axis=1)




#add NQD Surcharge column to PLD
tbl_PLD_original_DHLEM['SHIP_DATE'] = pd.to_datetime(tbl_PLD_original_DHLEM['SHIP_DATE'])

# billed weight calculation situations: 
# situation 1: actual weight <= 1 lb, use np.ceil(row['BILLED_WEIGHT_OZ']/16) or below, use 1 lb
# situation 2: actual weight > 1 lb, and cubic inch < 1728, use round up (actual weight)
# situation 3: actual weight > 1 lb, and cubic inch >= 1728, use billed weight

#dim
tbl_PLD_original_DHLEM['NQD_DHLEM_1'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (row['ACTUAL_WEIGHT'] >= 1 and row['CUBIC_INCH'] >= 1728 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else 
                                                           0), axis=1)


#girth + L(As > 84 is unaccepted, needs a special character to mark it, however, we cannot add up this)

tbl_PLD_original_DHLEM['NQD_DHLEM_2'] = tbl_PLD_original_DHLEM.apply(lambda row:
                                                           np.nan if (row['GIRTH_AND_L'] > 84) else (0), axis=1)

#longest side_1
tbl_PLD_original_DHLEM['NQD_DHLEM_3'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            2.5 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_3 and row['SHIP_DATE'] <= nqd_dhl_date_4) else 
                                                           (2 * row['BILLED_WEIGHT_LB'] if (max(row['L'], row['W'], row['H']) > 27 and row['SHIP_DATE'] >= nqd_dhl_date_1 and row['SHIP_DATE'] <= nqd_dhl_date_2) else
                                                           0), axis=1)

#longest side_2
tbl_PLD_original_DHLEM['NQD_DHLEM_4'] = tbl_PLD_original_DHLEM.apply(lambda row:
                                                           4.5 if (max(row['L'], row['W'], row['H']) > 22 and max(row['L'], row['W'], row['H']) <= 30) else
                                                           (15.5 if (max(row['L'], row['W'], row['H']) > 30) else 
                                                           0), axis=1)
#volume
tbl_PLD_original_DHLEM['NQD_DHLEM_5'] = tbl_PLD_original_DHLEM.apply(lambda row: 
                                                            15.5 if (row['CUBIC_INCH'] > 3456) else 
                                                           (0), axis=1)


tbl_PLD_original_DHLEM['NQD_DHLEM_OTHER'] = tbl_PLD_original_DHLEM['NQD_DHLEM_1'] + tbl_PLD_original_DHLEM['NQD_DHLEM_3'] + tbl_PLD_original_DHLEM['NQD_DHLEM_4'] + tbl_PLD_original_DHLEM['NQD_DHLEM_5']
tbl_PLD_original_DHLEM['NQD_DHLEM_GIRTH_AND_L'] = tbl_PLD_original_DHLEM['NQD_DHLEM_2']
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop(['NQD_DHLEM_1','NQD_DHLEM_2','NQD_DHLEM_3','NQD_DHLEM_4','NQD_DHLEM_5'], axis=1)


#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

tbl_PLD_original_DHLEM['TOTAL_DHLEM'] = round(tbl_PLD_original_DHLEM.apply(lambda row: 
                                                                  np.nan if ((row['FRT_DHLEM'] == '') | (pd.isna(row['FRT_DHLEM'])) | (row['NQD_DHLEM_GIRTH_AND_L'] == '') | (pd.isna(row['NQD_DHLEM_GIRTH_AND_L'] == '')))
                                                                  else
                                                                  (row['FRT_DHLEM'] + (row['FSC_DHLEM'] + row['NQD_DHLEM_OTHER'] + row['NQD_DHLEM_GIRTH_AND_L'])/(1-Margin_on_DHL)), axis=1),2)



In [157]:
tbl_PLD_original_DHLEM = pd.concat([tbl_PLD_original_DHLEM1, tbl_PLD_original_DHLEM], axis=0)
#tbl_PLD_original_DHLEM.to_csv('test.csv')

In [158]:
tbl_PLD_original_DHLEM.to_excel('test.xlsx')

# USPSAG

In [241]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_17 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAG')]
tbl_PLD_original_USPSAG = tbl_BP.copy()

In [242]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps)) if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

tbl_PLD_original_USPSAG['BILLED_WEIGHT_OZ'] = tbl_PLD_original_USPSAG.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps))*16 if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            row['BILLED_WEIGHT_OZ']/16 if (row['BILLED_WEIGHT_OZ'] <= 16) else 
                                                           (row['BILLED_WEIGHT_LB']), axis=1)

#usps will only rate for those not exceeding 70 lbs
tbl_PLD_original_USPSAG = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['BILLED_WEIGHT_LB'] <= 70)]


#add Freight column to PLD
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['GIRTH_AND_L'] > 108) & (tbl_PLD_original_USPSAG['GIRTH_AND_L'] <= 130)]
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG[(tbl_PLD_original_USPSAG['GIRTH_AND_L'] <= 108)]

#main case
tbl_PLD_original_USPSAG_none3['ZONE'] = tbl_PLD_original_USPSAG_none3['ZONE'].astype(str)
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.merge(tbl_clientRates_17[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_USPSAG_none3 = tbl_PLD_original_USPSAG_none3.rename(columns={'CLIENT_RATES': 'FRT_USPSAG'})

#3. for parcels that measure in combined length and girth more than 108 inches but not more than 130 inches, use oversized prices, 
#regardless of weight, based on the applicable zone.

tbl_oversize = tbl_clientRates_17[(tbl_clientRates_17['WEIGHT_LB'] == 9999)]
tbl_PLD_original_USPSAG_3['ZONE'] = tbl_PLD_original_USPSAG_3['ZONE'].astype(str)
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG_3.merge(tbl_oversize[["ZONE", "CLIENT_RATES"]], left_on=['ZONE'], right_on=['ZONE'], how='left')
tbl_PLD_original_USPSAG_3 = tbl_PLD_original_USPSAG_3.rename(columns={'CLIENT_RATES': 'FRT_USPSAG'})

#combine again
tbl_PLD_original_USPSAG = pd.concat([tbl_PLD_original_USPSAG_3, tbl_PLD_original_USPSAG_none3], axis=0)



In [162]:
# USPSAG_CI
# Regular Package
tbl_clientRates_21 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAG_CI')]
tbl_PLD_original_USPSAG_RP = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 20) & (tbl_BP['L_CI'] <= 18) & (tbl_BP['CUBIC_INCH'] < 1728)].copy()


KeyError: 'L_CI'

In [ ]:
tbl_PLD_original_USPSAG_RP['ZONE'] = tbl_PLD_original_USPSAG_RP['ZONE'].astype(str)
#This is slow code, but clearer than cross join. Can try cross join if later too slow. 
def get_rate(zone, cubic_inch_ci):
    # Filter df1 for the given zone
    filtered_tbl_clientRates_21 = tbl_clientRates_21[tbl_clientRates_21['ZONE'] == zone]

    # Sort and iterate to find the correct 'cubic inch' range
    for index, row in filtered_tbl_clientRates_21.sort_values('CUBIC_FT').iterrows():
        if cubic_inch_ci <= row['CUBIC_FT']:
            return row['CLIENT_RATES']
    return None  # or some default value if no range is found

# Apply the function to df2 to create the new 'rate' column
tbl_PLD_original_USPSAG_RP['FRT_USPSAG_RP'] = tbl_PLD_original_USPSAG_RP.apply(lambda row: get_rate(row['ZONE'], row['CUBIC_FT_CI']), axis=1)



In [35]:
# USPSAG_CI
# soft Package
tbl_clientRates_22 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAG_CI')]
tbl_PLD_original_USPSAG_SP = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 20) & (tbl_BP['L_CI'] <= 18) & (tbl_BP['CUBIC_INCH'] < 1728) & (tbl_BP['L_CI'] + tbl_BP['W_CI'] <= 36)].copy()
tbl_PLD_original_USPSAG_SP['ZONE'] = tbl_PLD_original_USPSAG_SP['ZONE'].astype(str)

In [36]:
ground_softpack_tier = pd.DataFrame({
    'Tier': ['Tier1', 'Tier2', 'Tier3', 'Tier4', 'Tier5', 'Tier6', 'Tier7', 'Tier8', 'Tier9', 'Tier10'],
    'Min': [0, 16, 21, 24, 26, 28, 30, 32, 34, 35],
    'Max': [16, 21, 24, 26, 28, 30, 32, 34, 35, 36] 
})

tbl_PLD_original_USPSAG_SP['Tier'] = np.nan

# Vectorized assignment
for _, row in ground_softpack_tier.iterrows():
    mask = (tbl_PLD_original_USPSAG_SP['L_AND_W_CI'] > row['Min']) & (tbl_PLD_original_USPSAG_SP['L_AND_W_CI'] <= row['Max'])
    tbl_PLD_original_USPSAG_SP.loc[mask, 'Tier'] = row['Tier']

tbl_PLD_original_USPSAG_SP = tbl_PLD_original_USPSAG_SP.merge(tbl_clientRates_22[["ZONE", "TIER", "CLIENT_RATES"]], left_on=['ZONE','Tier'], right_on=['ZONE','TIER'], how='left').rename(columns={'CLIENT_RATES': 'FRT_USPSAG_SP'})

tbl_PLD_original_USPSAG_SP = tbl_PLD_original_USPSAG_SP.drop(['Tier', 'TIER'], axis=1)



In [37]:
tbl_PLD_original_USPSAG = tbl_PLD_original_USPSAG.merge(tbl_PLD_original_USPSAG_RP[["TRACKING_NUMBER", "FRT_USPSAG_RP"]], left_on=['TRACKING_NUMBER'], right_on=['TRACKING_NUMBER'], how='left').merge(tbl_PLD_original_USPSAG_SP[["TRACKING_NUMBER", "FRT_USPSAG_SP"]], left_on=['TRACKING_NUMBER'], right_on=['TRACKING_NUMBER'], how='left')


In [38]:


# Calculate the minimum rate for each row
tbl_PLD_original_USPSAG['FRT_USPSAG_final'] = tbl_PLD_original_USPSAG[['FRT_USPSAG', 'FRT_USPSAG_RP', 'FRT_USPSAG_SP']].min(axis=1)

# Drop the original rate columns
#tbl_PLD_original_USPSAG.drop(['FRT_USPSAG_RP', 'FRT_USPSAG_SP'], axis=1, inplace=True)




In [243]:
#6. Parcels that exceed 22 inches but not greater than 30 inches in length, add $4 per package. 
#7. parcels that exceed 30 inches in length, add $7 per package. 

tbl_PLD_original_USPSAG['EXTRA_USPSAG_1'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            4 if (row['L'] > 22 and row['L'] <= 30) else 
                                                           (18 if (row['L'] > 30) else
                                                           (0)), axis=1)


#8. parcels that exceed 2 cubic feet (3456 cubic inches), add $15 per package. 
tbl_PLD_original_USPSAG['EXTRA_USPSAG_2'] = tbl_PLD_original_USPSAG.apply(lambda row:
                                                            30 if (row['CUBIC_INCH'] > 3456) else (0), axis=1)

#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

#Get Total, revise the logic that if the freight rate is 0 or some rate is 0, the total will be 0. ups can handle everything. 
tbl_PLD_original_USPSAG['TOTAL_USPSAG'] = round(tbl_PLD_original_USPSAG.apply(lambda row: 
                                                                  (row['FRT_USPSAG'] + (row['EXTRA_USPSAG_1'] + row['EXTRA_USPSAG_2'])/(1-Margin_on_USPS)) if row['FRT_USPSAG'] != '' else '', axis=1),2)                         
#tbl_PLD_original_USPSAG.to_csv('test.csv')


# USPSAP

In [244]:
#Get the right client rate and the corresponding PLD
#get client rate and a copy of PLD
tbl_clientRates_18 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAP')]
tbl_PLD_original_USPSAP = tbl_BP.copy()

In [245]:
#Rerate start
#add billed weight column to PLD
tbl_PLD_original_USPSAP['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAP.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps)) if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)

tbl_PLD_original_USPSAP['BILLED_WEIGHT_OZ'] = tbl_PLD_original_USPSAP.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_usps))*16 if (row['CUBIC_INCH'] > 1728) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)

tbl_PLD_original_USPSAP['BILLED_WEIGHT_LB'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            0.5 if (row['BILLED_WEIGHT_OZ'] <= 8) else
                                                            (1 if (row['BILLED_WEIGHT_OZ'] > 8 and row['BILLED_WEIGHT_OZ'] <= 16) else             
                                                            (row['BILLED_WEIGHT_LB'])), axis=1)

#add Freight column to PLD
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP[(tbl_PLD_original_USPSAP['GIRTH_AND_L'] > 108) & (tbl_PLD_original_USPSAP['GIRTH_AND_L'] <= 130)]
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP[(tbl_PLD_original_USPSAP['GIRTH_AND_L'] <= 108)]

#main case
tbl_PLD_original_USPSAP_none3['ZONE'] = tbl_PLD_original_USPSAP_none3['ZONE'].astype(str)
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.merge(tbl_clientRates_18[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_USPSAP_none3 = tbl_PLD_original_USPSAP_none3.rename(columns={'CLIENT_RATES': 'FRT_USPSAP'})

#3. for parcels that measure in combined length and girth more than 108 inches but not more than 130 inches, use oversized prices, 
#regardless of weight, based on the applicable zone.

tbl_oversize = tbl_clientRates_18[(tbl_clientRates_18['WEIGHT_LB'] == 9999)]
tbl_PLD_original_USPSAP_3['ZONE'] = tbl_PLD_original_USPSAP_3['ZONE'].astype(str)
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP_3.merge(tbl_oversize[["ZONE", "CLIENT_RATES"]], left_on=['ZONE'], right_on=['ZONE'], how='left')
tbl_PLD_original_USPSAP_3 = tbl_PLD_original_USPSAP_3.rename(columns={'CLIENT_RATES': 'FRT_USPSAP'})

#combine again
tbl_PLD_original_USPSAP = pd.concat([tbl_PLD_original_USPSAP_3, tbl_PLD_original_USPSAP_none3], axis=0)

#6. Parcels that exceed 22 inches but not greater than 30 inches in length, add $4 per package. 
#7. parcels that exceed 30 inches in length, add $7 per package. 




In [43]:
# USPSAP_CI
# Regular Package
tbl_clientRates_23 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAP_CI')]
tbl_PLD_original_USPSAP_RP = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 20) & (tbl_BP['L_CI'] <= 18) & (tbl_BP['CUBIC_INCH'] < 1728)].copy()
tbl_PLD_original_USPSAP_RP['ZONE'] = tbl_PLD_original_USPSAP_RP['ZONE'].astype(str)

In [44]:
#This is slow code, but clearer than cross join. Can try cross join if later too slow. 
def get_rate(zone, cubic_inch_ci):
    # Filter df1 for the given zone
    filtered_tbl_clientRates_23 = tbl_clientRates_23[tbl_clientRates_23['ZONE'] == zone]

    # Sort and iterate to find the correct 'cubic inch' range
    for index, row in filtered_tbl_clientRates_23.sort_values('CUBIC_FT').iterrows():
        if cubic_inch_ci <= row['CUBIC_FT']:
            return row['CLIENT_RATES']
    return None  # or some default value if no range is found

# Apply the function to df2 to create the new 'rate' column
tbl_PLD_original_USPSAP_RP['FRT_USPSAP_RP'] = tbl_PLD_original_USPSAP_RP.apply(lambda row: get_rate(row['ZONE'], row['CUBIC_FT_CI']), axis=1)



In [45]:
# USPSAG_CI
# soft Package
tbl_clientRates_24 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'USPSAP_CI')]
tbl_PLD_original_USPSAP_SP = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] <= 20) & (tbl_BP['L_CI'] <= 18) & (tbl_BP['CUBIC_INCH'] < 1728) & (tbl_BP['L_CI'] + tbl_BP['W_CI'] <= 36)].copy()
tbl_PLD_original_USPSAP_SP['ZONE'] = tbl_PLD_original_USPSAP_SP['ZONE'].astype(str)

In [46]:

priority_softpack_tier = pd.DataFrame({
    'Tier': ['Tier1', 'Tier2', 'Tier3', 'Tier4', 'Tier5'],
    'Min': [0, 21, 27, 31, 34],
    'Max': [21, 27, 31, 34, 36] 
})


tbl_PLD_original_USPSAP_SP['Tier'] = np.nan

# Vectorized assignment
for _, row in priority_softpack_tier.iterrows():
    mask = (tbl_PLD_original_USPSAP_SP['L_AND_W_CI'] > row['Min']) & (tbl_PLD_original_USPSAP_SP['L_AND_W_CI'] <= row['Max'])
    tbl_PLD_original_USPSAP_SP.loc[mask, 'Tier'] = row['Tier']

tbl_PLD_original_USPSAP_SP = tbl_PLD_original_USPSAP_SP.merge(tbl_clientRates_24[["ZONE", "TIER", "CLIENT_RATES"]], left_on=['ZONE','Tier'], right_on=['ZONE','TIER'], how='left').rename(columns={'CLIENT_RATES': 'FRT_USPSAP_SP'})

tbl_PLD_original_USPSAP_SP = tbl_PLD_original_USPSAP_SP.drop(['Tier', 'TIER'], axis=1)



In [47]:
tbl_PLD_original_USPSAP = tbl_PLD_original_USPSAP.merge(tbl_PLD_original_USPSAP_RP[["TRACKING_NUMBER", "FRT_USPSAP_RP"]], left_on=['TRACKING_NUMBER'], right_on=['TRACKING_NUMBER'], how='left').merge(tbl_PLD_original_USPSAP_SP[["TRACKING_NUMBER", "FRT_USPSAP_SP"]], left_on=['TRACKING_NUMBER'], right_on=['TRACKING_NUMBER'], how='left')


In [48]:


# Calculate the minimum rate for each row
tbl_PLD_original_USPSAP['FRT_USPSAP_final'] = tbl_PLD_original_USPSAP[['FRT_USPSAP', 'FRT_USPSAP_RP', 'FRT_USPSAP_SP']].min(axis=1)

# Drop the original rate columns
#tbl_PLD_original_USPSAP.drop(['FRT_USPSAP_RP', 'FRT_USPSAP_SP'], axis=1, inplace=True)



In [246]:

tbl_PLD_original_USPSAP['EXTRA_USPSAP_1'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            4 if (row['L'] > 22 and row['L'] <= 30) else 
                                                           (18 if (row['L'] > 30) else
                                                            (0)), axis=1)


#8. parcels that exceed 2 cubic feet (3456 cubic inches), add $15 per package. 
tbl_PLD_original_USPSAP['EXTRA_USPSAP_2'] = tbl_PLD_original_USPSAP.apply(lambda row:
                                                            30 if (row['CUBIC_INCH'] > 3456) else (0), axis=1)

#Get Total
# note that for those with non-round up actual weight, if it is < 1, we still assign it the freight rate. 

#Get Total, revise the logic that if the freight rate is 0 or some rate is 0, the total will be 0. ups can handle everything. 
tbl_PLD_original_USPSAP['TOTAL_USPSAP'] = round(tbl_PLD_original_USPSAP.apply(lambda row: 
                                                                  (row['FRT_USPSAP'] + (row['EXTRA_USPSAP_1'] + row['EXTRA_USPSAP_2'])/(1-Margin_on_USPS)) if row['FRT_USPSAP'] != '' else '', axis=1),2)                         
#tbl_PLD_original_USPSAP.to_csv('test.csv')

# MI

#MIPLE (MI Parcel Select Lightweight Expedited): commercial & residential, oz

#MIPH (MI Parcel Select Heavyweight): commercial & residential, lb

# MIPLE (MI Parcel Select Lightweight Expedited)

In [144]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_19 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'MIPLE')]
tbl_PLD_original_MIPLE = tbl_BP[(tbl_BP['ACTUAL_WEIGHT'] < 1) & (tbl_BP['L'] <= 26) & (tbl_BP['W'] <= 26) & (tbl_BP['H'] <= 26)].copy()

In [145]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_MIPLE['BILLED_WEIGHT_OZ'] = tbl_PLD_original_MIPLE.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_mi))*16 if (row['CUBIC_INCH'] > 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_OZ']), axis=1)


#add Freight column to PLD
tbl_PLD_original_MIPLE['ZONE'] = tbl_PLD_original_MIPLE['ZONE'].astype(str)
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.merge(tbl_clientRates_19[["WEIGHT_OZ", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_OZ', 'ZONE'], right_on=['WEIGHT_OZ', 'ZONE'], how='left')
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.drop('WEIGHT_OZ', axis=1)
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.rename(columns={'CLIENT_RATES': 'FRT_MIPLE'})

#add fuel surcharge column to PLD
tbl_PLD_original_MIPLE['FSC_MIPLE'] = tbl_PLD_original_MIPLE.apply(lambda row: row['FRT_MIPLE']*0.065, axis=1)

#add NQD Surcharge column to PLD

# A Non-standard piece fee of $4.00 will apply to any piece with a single side over 22" or with total dimensions over 2 ft.
tbl_PLD_original_MIPLE['NQD_MIPLE'] = tbl_PLD_original_MIPLE.apply(lambda row:
                                                            4 if any([row['L'] > 22, row['W'] > 22, row['H'] > 22, row['CUBIC_INCH'] > 3456]) else
                                                           (0), axis=1)

#Get Total
tbl_PLD_original_MIPLE['TOTAL_MIPLE'] = round(tbl_PLD_original_MIPLE.apply(lambda row: 
                                                                  np.nan if ((row['FRT_MIPLE'] == '') | (pd.isna(row['FRT_MIPLE'])))
                                                                  else
                                                                  ((row['FRT_MIPLE'] + row['FSC_MIPLE']) + (row['NQD_MIPLE'])/(1-Margin_on_MI)), axis=1),2)

#tbl_PLD_original_MIPLE.to_csv('test.csv')

In [121]:
tbl_PLD_original_MIPLE.to_excel('test.xlsx')

# MIPH (MI Parcel Select Heavyweight)

In [146]:
#select the rate to use, #Decide the part of PLD to rerate
tbl_clientRates_20 = tbl_clientRates[(tbl_clientRates['SERVICE_CODE'] == 'MIPH')]
tbl_PLD_original_MIPH = tbl_BP[(tbl_BP['L'] <= 26) & (tbl_BP['W'] <= 26) & (tbl_BP['H'] <= 26)].copy()

In [147]:
#Start to rerate
#add billed weight column to PLD
tbl_PLD_original_MIPH['BILLED_WEIGHT_LB'] = tbl_PLD_original_MIPH.apply(lambda row: max(row['ACTUAL_WEIGHT_LB'], np.ceil(row['CUBIC_INCH']/dim_factor_mi)) if (row['CUBIC_INCH'] > 1728 and row['ACTUAL_WEIGHT'] >= 1) else
                                                                          (row['ACTUAL_WEIGHT_LB']), axis=1)



#add Freight column to PLD
tbl_PLD_original_MIPH['ZONE'] = tbl_PLD_original_MIPH['ZONE'].astype(str)
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.merge(tbl_clientRates_20[["WEIGHT_LB", "ZONE", "CLIENT_RATES"]], left_on=['BILLED_WEIGHT_LB', 'ZONE'], right_on=['WEIGHT_LB', 'ZONE'], how='left')
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.drop('WEIGHT_LB', axis=1)
tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.rename(columns={'CLIENT_RATES': 'FRT_MIPH'})

#add fuel surcharge column to PLD
tbl_PLD_original_MIPH['FSC_MIPH'] = tbl_PLD_original_MIPH.apply(lambda row: row['FRT_MIPH']*0.065, axis=1)


#add NQD Surcharge column to PLD

# A Non-standard piece fee of $4.00 will apply to any piece with a single side over 22" or with total dimensions over 2 ft.
tbl_PLD_original_MIPH['NQD_MIPH'] = tbl_PLD_original_MIPH.apply(lambda row:
                                                            4 if any([row['L'] > 22, row['W'] > 22, row['H'] > 22, row['CUBIC_INCH'] > 3456]) else
                                                           (0), axis=1)

#Get Total
tbl_PLD_original_MIPH['TOTAL_MIPH'] = round(tbl_PLD_original_MIPH.apply(lambda row: 
                                                                  np.nan if ((row['FRT_MIPH'] == '') | (pd.isna(row['FRT_MIPH'])))
                                                                  else
                                                                  ((row['FRT_MIPH'] + row['FSC_MIPH']) + (row['NQD_MIPH'])/(1-Margin_on_MI)), axis=1),2)

#tbl_PLD_original_MIPH.to_csv('test.csv')

In [122]:
tbl_PLD_original_MIPH.to_excel('test.xlsx')

# 1.  Model Output

# Specify the columns to keep in the model

In [148]:
tbl_BP.columns

Index(['CustomerID', 'CustomerName', 'Facility', 'BarrettOrderNumber',
       'Reference', 'PoNumber', 'TrackingNumber', 'ShipDate', 'Service',
       'Shipper', 'ShipToName', 'ShipToContact', 'ShipToCity', 'ShipToState',
       'ZipCode', 'ShipToCountry', 'Zone', 'Quantity', 'Dimensions',
       'ActualWeight', 'ResidentialFlag', 'JoChannel', 'TRACKING_NUMBER',
       'SHIP_DATE', 'OLD_SERVICE', 'ZIP_CODE', 'ZONE', 'DIMENSIONS',
       'ACTUAL_WEIGHT', 'RESIDENTIAL_FLAG', 'L', 'W', 'H', 'GIRTH_AND_L',
       'CUBIC_INCH', 'ACTUAL_WEIGHT_LB', 'ACTUAL_WEIGHT_OZ'],
      dtype='object')

In [149]:
# Make a copy of the DataFrame with only the specified columns
tbl_PLD_original_rerated = tbl_BP.copy()

# Specify the prioritized Services

In [484]:
# Do not run below code during the 1st try (usually due to fsc data duplications) 
tbl_PLD_original_REDE = tbl_PLD_original_REDE.drop_duplicates()
tbl_PLD_original_RED = tbl_PLD_original_RED.drop_duplicates()
tbl_PLD_original_REDS = tbl_PLD_original_REDS.drop_duplicates()
tbl_PLD_original_2DAM = tbl_PLD_original_2DAM.drop_duplicates()
tbl_PLD_original_BLUE = tbl_PLD_original_BLUE.drop_duplicates()
tbl_PLD_original_ORNG = tbl_PLD_original_ORNG.drop_duplicates()
tbl_PLD_original_GRND = tbl_PLD_original_GRND.drop_duplicates()
tbl_PLD_original_GRES = tbl_PLD_original_GRES.drop_duplicates()
tbl_PLD_original_SRPT1 = tbl_PLD_original_SRPT1.drop_duplicates()
tbl_PLD_original_SRPT = tbl_PLD_original_SRPT.drop_duplicates()

tbl_PLD_original_DHLG = tbl_PLD_original_DHLG.drop_duplicates()
tbl_PLD_original_DHLG1 = tbl_PLD_original_DHLG1.drop_duplicates()
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop_duplicates()
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop_duplicates()
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop_duplicates()

tbl_PLD_original_USPSAG = tbl_PLD_original_USPSAG.drop_duplicates()
tbl_PLD_original_USPSAP = tbl_PLD_original_USPSAP.drop_duplicates()

tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.drop_duplicates()
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.drop_duplicates()

In [150]:
tbl_PLD_original_DHLE = tbl_PLD_original_DHLE.drop_duplicates()
tbl_PLD_original_DHLE1 = tbl_PLD_original_DHLE1.drop_duplicates()
tbl_PLD_original_DHLEM = tbl_PLD_original_DHLEM.drop_duplicates()

tbl_PLD_original_MIPH = tbl_PLD_original_MIPH.drop_duplicates()
tbl_PLD_original_MIPLE = tbl_PLD_original_MIPLE.drop_duplicates()

In [82]:
# List only the total rates of all carrier services. 
tbl_PLD_original_rerated['TOTAL_REDE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['TOTAL_REDE'])
tbl_PLD_original_rerated['TOTAL_RED'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['TOTAL_RED'])
tbl_PLD_original_rerated['TOTAL_REDS'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['TOTAL_REDS'])
tbl_PLD_original_rerated['TOTAL_2DAM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['TOTAL_2DAM'])
tbl_PLD_original_rerated['TOTAL_BLUE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['TOTAL_BLUE'])
tbl_PLD_original_rerated['TOTAL_ORNG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['TOTAL_ORNG'])
tbl_PLD_original_rerated['TOTAL_GRND'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['TOTAL_GRND'])
tbl_PLD_original_rerated['TOTAL_GRES'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['TOTAL_GRES'])
tbl_PLD_original_rerated['TOTAL_SRPT1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['TOTAL_SRPT1'])
tbl_PLD_original_rerated['TOTAL_SRPT'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['TOTAL_SRPT'])

tbl_PLD_original_rerated['TOTAL_1STO'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['TOTAL_1STO'])
tbl_PLD_original_rerated['TOTAL_PROV'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['TOTAL_PROV'])
tbl_PLD_original_rerated['TOTAL_STOV'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['TOTAL_STOV'])
tbl_PLD_original_rerated['TOTAL_2AM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['TOTAL_2AM'])
tbl_PLD_original_rerated['TOTAL_2DAY'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['TOTAL_2DAY'])
tbl_PLD_original_rerated['TOTAL_EXSA'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['TOTAL_EXSA'])
tbl_PLD_original_rerated['TOTAL_GRNDF'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['TOTAL_GRNDF'])
tbl_PLD_original_rerated['TOTAL_HOME'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['TOTAL_HOME'])
tbl_PLD_original_rerated['TOTAL_SPST1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['TOTAL_SPST1'])
tbl_PLD_original_rerated['TOTAL_SPST'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['TOTAL_SPST'])


tbl_PLD_original_rerated['TOTAL_DHLG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['TOTAL_DHLG'])
tbl_PLD_original_rerated['TOTAL_DHLG1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['TOTAL_DHLG1'])
tbl_PLD_original_rerated['TOTAL_DHLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['TOTAL_DHLE'])
tbl_PLD_original_rerated['TOTAL_DHLE1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['TOTAL_DHLE1'])
tbl_PLD_original_rerated['TOTAL_DHLEM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['TOTAL_DHLEM'])

tbl_PLD_original_rerated['TOTAL_USPSAG'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAG.set_index('TRACKING_NUMBER')['TOTAL_USPSAG'])
tbl_PLD_original_rerated['TOTAL_USPSAP'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAP.set_index('TRACKING_NUMBER')['TOTAL_USPSAP'])

tbl_PLD_original_rerated['TOTAL_MIPLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['TOTAL_MIPLE'])
tbl_PLD_original_rerated['TOTAL_MIPH'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['TOTAL_MIPH'])




NameError: name 'tbl_PLD_original_REDE' is not defined

In [151]:
tbl_PLD_original_rerated['TOTAL_DHLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['TOTAL_DHLE'])
tbl_PLD_original_rerated['TOTAL_DHLE1'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['TOTAL_DHLE1'])
tbl_PLD_original_rerated['TOTAL_DHLEM'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['TOTAL_DHLEM'])

tbl_PLD_original_rerated['TOTAL_MIPLE'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['TOTAL_MIPLE'])
tbl_PLD_original_rerated['TOTAL_MIPH'] = tbl_PLD_original_rerated['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['TOTAL_MIPH'])


# Base Model

In [152]:
# Base model output. Make sure to check the obnormal rates due to obnormal dim, etc. 
tbl_PLD_original_rerated.to_excel('Barrett_DHL&MI.xlsx')

# Best Way Model

In [815]:
columns_to_keep_1 = [
'TRACKING_NUMBER', 'SHIP_DATE', 'OLD_SERVICE', 'ZIP_CODE', 'ZONE_old',
       'L', 'W', 'H', 'ACTUAL_WEIGHT', 'RESIDENTIAL_FLAG', 'State',
       'GIRTH_AND_L', 'CUBIC_INCH', 'ACTUAL_WEIGHT_LB', 'ACTUAL_WEIGHT_OZ',
       'FROM_ZIP', 'ZONE', 'DAS_CATEGORY']

columns_to_keep_2 = ['TOTAL_REDE',
     'TOTAL_RED', 'TOTAL_REDS', 'TOTAL_2DAM', 
    'TOTAL_BLUE', 'TOTAL_ORNG',
       'TOTAL_GRND', 'TOTAL_GRES', 
               'TOTAL_SRPT1', 'TOTAL_SRPT', 
                    'TOTAL_DHLG','TOTAL_DHLG1', 
                     'TOTAL_DHLE', 'TOTAL_DHLE1', 
       'TOTAL_DHLEM',
        'TOTAL_USPSAG',  'TOTAL_USPSAP', 
    'TOTAL_MIPLE', 'TOTAL_MIPH',
   'TOTAL_1STO', 'TOTAL_PROV', 'TOTAL_STOV', 'TOTAL_2AM', 'TOTAL_2DAY', 
   'TOTAL_EXSA', 'TOTAL_GRNDF', 'TOTAL_HOME', 'TOTAL_SPST1', 'TOTAL_SPST'
                     
                    ]

# Make a copy of the DataFrame with only the specified columns
tbl_PLD_original_rerated_1 = tbl_PLD_original_rerated[columns_to_keep_1].copy()
tbl_PLD_original_rerated_2 = tbl_PLD_original_rerated[columns_to_keep_2].copy()

# Select min value from each row
tbl_PLD_original_rerated_2 = tbl_PLD_original_rerated_2.fillna(float('inf'))
selected_values = tbl_PLD_original_rerated_2.apply(lambda row: row.loc[row.idxmin()], axis=1)

# Create a new column based on the corresponding column name
tbl_PLD_original_rerated_2['SELECTED_SERVICE'] = tbl_PLD_original_rerated_2.idxmin(axis=1)

# Add the selected values as a new column
tbl_PLD_original_rerated_2['CARRIER_COST_MIN'] = selected_values

tbl_PLD_original_rerated_bestWay = pd.concat([tbl_PLD_original_rerated_1, tbl_PLD_original_rerated_2], axis=1)

In [816]:
tbl_REDE = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_REDE')].copy()
tbl_RED = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_RED')].copy()
tbl_REDS = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_REDS')].copy()
tbl_2DAM = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2DAM')].copy()
tbl_BLUE = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_BLUE')].copy()

tbl_ORNG = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_ORNG')].copy()
tbl_GRND = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRND')].copy()
tbl_GRES = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRES')].copy()
tbl_SRPT1 = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SRPT1')].copy()
tbl_SRPT = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SRPT')].copy()

tbl_1STO = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_1STO')].copy()
tbl_PROV = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_PROV')].copy()
tbl_STOV = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_STOV')].copy()
tbl_2AM = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2AM')].copy()
tbl_2DAY = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2DAY')].copy()

tbl_EXSA = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_EXSA')].copy()
tbl_GRNDF = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRNDF')].copy()
tbl_HOME = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_HOME')].copy()
tbl_SPST1 = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SPST1')].copy()
tbl_SPST = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SPST')].copy()


tbl_DHLG = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLG')].copy()
tbl_DHLG1 = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLG1')].copy()
tbl_DHLE = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLE')].copy()
tbl_DHLE1 = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLE1')].copy()
tbl_DHLEM = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLEM')].copy()

tbl_USPSAG = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_USPSAG')].copy()
tbl_USPSAP = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_USPSAP')].copy()

tbl_MIPLE = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_MIPLE')].copy()
tbl_MIPH = tbl_PLD_original_rerated_bestWay[(tbl_PLD_original_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_MIPH')].copy()


In [817]:
tbl_REDE['FRT_UPS'] = tbl_REDE['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['FRT_REDE'])
tbl_REDE['FSC%_UPS'] = tbl_REDE['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['FSC%_REDE'])
tbl_REDE['RES_UPS'] = tbl_REDE['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['RES_REDE'])
tbl_REDE['DAS_UPS'] = tbl_REDE['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['DAS_REDE'])
tbl_REDE['AHS_UPS'] = tbl_REDE['TRACKING_NUMBER'].map(tbl_PLD_original_REDE.set_index('TRACKING_NUMBER')['AHS_REDE'])

tbl_RED['FRT_UPS'] = tbl_RED['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['FRT_RED'])
tbl_RED['FSC%_UPS'] = tbl_RED['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['FSC%_RED'])
tbl_RED['RES_UPS'] = tbl_RED['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['RES_RED'])
tbl_RED['DAS_UPS'] = tbl_RED['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['DAS_RED'])
tbl_RED['AHS_UPS'] = tbl_RED['TRACKING_NUMBER'].map(tbl_PLD_original_RED.set_index('TRACKING_NUMBER')['AHS_RED'])

tbl_REDS['FRT_UPS'] = tbl_REDS['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['FRT_REDS'])
tbl_REDS['FSC%_UPS'] = tbl_REDS['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['FSC%_REDS'])
tbl_REDS['RES_UPS'] = tbl_REDS['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['RES_REDS'])
tbl_REDS['DAS_UPS'] = tbl_REDS['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['DAS_REDS'])
tbl_REDS['AHS_UPS'] = tbl_REDS['TRACKING_NUMBER'].map(tbl_PLD_original_REDS.set_index('TRACKING_NUMBER')['AHS_REDS'])

tbl_2DAM['FRT_UPS'] = tbl_2DAM['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['FRT_2DAM'])
tbl_2DAM['FSC%_UPS'] = tbl_2DAM['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['FSC%_2DAM'])
tbl_2DAM['RES_UPS'] = tbl_2DAM['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['RES_2DAM'])
tbl_2DAM['DAS_UPS'] = tbl_2DAM['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['DAS_2DAM'])
tbl_2DAM['AHS_UPS'] = tbl_2DAM['TRACKING_NUMBER'].map(tbl_PLD_original_2DAM.set_index('TRACKING_NUMBER')['AHS_2DAM'])

tbl_BLUE['FRT_UPS'] = tbl_BLUE['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['FRT_BLUE'])
tbl_BLUE['FSC%_UPS'] = tbl_BLUE['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['FSC%_BLUE'])
tbl_BLUE['RES_UPS'] = tbl_BLUE['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['RES_BLUE'])
tbl_BLUE['DAS_UPS'] = tbl_BLUE['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['DAS_BLUE'])
tbl_BLUE['AHS_UPS'] = tbl_BLUE['TRACKING_NUMBER'].map(tbl_PLD_original_BLUE.set_index('TRACKING_NUMBER')['AHS_BLUE'])


tbl_ORNG['FRT_UPS'] = tbl_ORNG['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['FRT_ORNG'])
tbl_ORNG['FSC%_UPS'] = tbl_ORNG['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['FSC%_ORNG'])
tbl_ORNG['RES_UPS'] = tbl_ORNG['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['RES_ORNG'])
tbl_ORNG['DAS_UPS'] = tbl_ORNG['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['DAS_ORNG'])
tbl_ORNG['AHS_UPS'] = tbl_ORNG['TRACKING_NUMBER'].map(tbl_PLD_original_ORNG.set_index('TRACKING_NUMBER')['AHS_ORNG'])

tbl_GRND['FRT_UPS'] = tbl_GRND['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['FRT_GRND'])
tbl_GRND['FSC%_UPS'] = tbl_GRND['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['FSC%_GRND'])
tbl_GRND['RES_UPS'] = tbl_GRND['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['RES_GRND'])
tbl_GRND['DAS_UPS'] = tbl_GRND['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['DAS_GRND'])
tbl_GRND['AHS_UPS'] = tbl_GRND['TRACKING_NUMBER'].map(tbl_PLD_original_GRND.set_index('TRACKING_NUMBER')['AHS_GRND'])

tbl_GRES['FRT_UPS'] = tbl_GRES['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['FRT_GRES'])
tbl_GRES['FSC%_UPS'] = tbl_GRES['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['FSC%_GRES'])
tbl_GRES['RES_UPS'] = tbl_GRES['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['RES_GRES'])
tbl_GRES['DAS_UPS'] = tbl_GRES['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['DAS_GRES'])
tbl_GRES['AHS_UPS'] = tbl_GRES['TRACKING_NUMBER'].map(tbl_PLD_original_GRES.set_index('TRACKING_NUMBER')['AHS_GRES'])

tbl_SRPT1['FRT_UPS'] = tbl_SRPT1['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['FRT_SRPT1'])
tbl_SRPT1['FSC%_UPS'] = tbl_SRPT1['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['FSC%_SRPT1'])
tbl_SRPT1['RES_UPS'] = tbl_SRPT1['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['RES_SRPT1'])
tbl_SRPT1['DAS_UPS'] = tbl_SRPT1['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['DAS_SRPT1'])
tbl_SRPT1['AHS_UPS'] = tbl_SRPT1['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT1.set_index('TRACKING_NUMBER')['AHS_SRPT1'])

tbl_SRPT['FRT_UPS'] = tbl_SRPT['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['FRT_SRPT'])
tbl_SRPT['FSC%_UPS'] = tbl_SRPT['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['FSC%_SRPT'])
tbl_SRPT['RES_UPS'] = tbl_SRPT['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['RES_SRPT'])
tbl_SRPT['DAS_UPS'] = tbl_SRPT['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['DAS_SRPT'])
tbl_SRPT['AHS_UPS'] = tbl_SRPT['TRACKING_NUMBER'].map(tbl_PLD_original_SRPT.set_index('TRACKING_NUMBER')['AHS_SRPT'])






tbl_1STO['FRT_FedEx'] = tbl_1STO['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['FRT_1STO'])
tbl_1STO['FSC%_FDX'] = tbl_1STO['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['FSC%_1STO'])
tbl_1STO['RES_FDX'] = tbl_1STO['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['RES_1STO'])
tbl_1STO['DAS_FDX'] = tbl_1STO['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['DAS_1STO'])
tbl_1STO['AHS_FDX'] = tbl_1STO['TRACKING_NUMBER'].map(tbl_PLD_original_1STO.set_index('TRACKING_NUMBER')['AHS_1STO'])

tbl_PROV['FRT_FDX'] = tbl_PROV['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['FRT_PROV'])
tbl_PROV['FSC%_FDX'] = tbl_PROV['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['FSC%_PROV'])
tbl_PROV['RES_FDX'] = tbl_PROV['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['RES_PROV'])
tbl_PROV['DAS_FDX'] = tbl_PROV['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['DAS_PROV'])
tbl_PROV['AHS_FDX'] = tbl_PROV['TRACKING_NUMBER'].map(tbl_PLD_original_PROV.set_index('TRACKING_NUMBER')['AHS_PROV'])

tbl_STOV['FRT_FDX'] = tbl_STOV['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['FRT_STOV'])
tbl_STOV['FSC%_FDX'] = tbl_STOV['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['FSC%_STOV'])
tbl_STOV['RES_FDX'] = tbl_STOV['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['RES_STOV'])
tbl_STOV['DAS_FDX'] = tbl_STOV['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['DAS_STOV'])
tbl_STOV['AHS_FDX'] = tbl_STOV['TRACKING_NUMBER'].map(tbl_PLD_original_STOV.set_index('TRACKING_NUMBER')['AHS_STOV'])



tbl_2AM['FRT_FDX'] = tbl_2AM['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['FRT_2AM'])
tbl_2AM['FSC%_FDX'] = tbl_2AM['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['FSC%_2AM'])
tbl_2AM['RES_FDX'] = tbl_2AM['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['RES_2AM'])
tbl_2AM['DAS_FDX'] = tbl_2AM['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['DAS_2AM'])
tbl_2AM['AHS_FDX'] = tbl_2AM['TRACKING_NUMBER'].map(tbl_PLD_original_2AM.set_index('TRACKING_NUMBER')['AHS_2AM'])

tbl_2DAY['FRT_FDX'] = tbl_2DAY['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['FRT_2DAY'])
tbl_2DAY['FSC%_FDX'] = tbl_2DAY['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['FSC%_2DAY'])
tbl_2DAY['RES_FDX'] = tbl_2DAY['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['RES_2DAY'])
tbl_2DAY['DAS_FDX'] = tbl_2DAY['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['DAS_2DAY'])
tbl_2DAY['AHS_FDX'] = tbl_2DAY['TRACKING_NUMBER'].map(tbl_PLD_original_2DAY.set_index('TRACKING_NUMBER')['AHS_2DAY'])


tbl_EXSA['FRT_FDX'] = tbl_EXSA['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['FRT_EXSA'])
tbl_EXSA['FSC%_FDX'] = tbl_EXSA['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['FSC%_EXSA'])
tbl_EXSA['RES_FDX'] = tbl_EXSA['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['RES_EXSA'])
tbl_EXSA['DAS_FDX'] = tbl_EXSA['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['DAS_EXSA'])
tbl_EXSA['AHS_FDX'] = tbl_EXSA['TRACKING_NUMBER'].map(tbl_PLD_original_EXSA.set_index('TRACKING_NUMBER')['AHS_EXSA'])


tbl_GRNDF['FRT_FDX'] = tbl_GRNDF['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['FRT_GRNDF'])
tbl_GRNDF['FSC%_FDX'] = tbl_GRNDF['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['FSC%_GRNDF'])
tbl_GRNDF['RES_FDX'] = tbl_GRNDF['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['RES_GRNDF'])
tbl_GRNDF['DAS_FDX'] = tbl_GRNDF['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['DAS_GRNDF'])
tbl_GRNDF['AHS_FDX'] = tbl_GRNDF['TRACKING_NUMBER'].map(tbl_PLD_original_GRNDF.set_index('TRACKING_NUMBER')['AHS_GRNDF'])



tbl_HOME['FRT_FDX'] = tbl_HOME['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['FRT_HOME'])
tbl_HOME['FSC%_FDX'] = tbl_HOME['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['FSC%_HOME'])
tbl_HOME['RES_FDX'] = tbl_HOME['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['RES_HOME'])
tbl_HOME['DAS_FDX'] = tbl_HOME['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['DAS_HOME'])
tbl_HOME['AHS_FDX'] = tbl_HOME['TRACKING_NUMBER'].map(tbl_PLD_original_HOME.set_index('TRACKING_NUMBER')['AHS_HOME'])


tbl_SPST1['FRT_FDX'] = tbl_SPST1['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['FRT_SPST1'])
tbl_SPST1['FSC%_FDX'] = tbl_SPST1['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['FSC%_SPST1'])
tbl_SPST1['RES_FDX'] = tbl_SPST1['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['RES_SPST1'])
tbl_SPST1['DAS_FDX'] = tbl_SPST1['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['DAS_SPST1'])
tbl_SPST1['AHS_FDX'] = tbl_SPST1['TRACKING_NUMBER'].map(tbl_PLD_original_SPST1.set_index('TRACKING_NUMBER')['AHS_SPST1'])


tbl_SPST['FRT_FDX'] = tbl_SPST['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['FRT_SPST'])
tbl_SPST['FSC%_FDX'] = tbl_SPST['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['FSC%_SPST'])
tbl_SPST['RES_FDX'] = tbl_SPST['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['RES_SPST'])
tbl_SPST['DAS_FDX'] = tbl_SPST['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['DAS_SPST'])
tbl_SPST['AHS_FDX'] = tbl_SPST['TRACKING_NUMBER'].map(tbl_PLD_original_SPST.set_index('TRACKING_NUMBER')['AHS_SPST'])




tbl_DHLG['FRT_DHL'] = tbl_DHLG['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['FRT_DHLG'])
tbl_DHLG['FSC%_DHL'] = tbl_DHLG['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['FSC%_DHLG'])
tbl_DHLG['FSC_DHL'] = tbl_DHLG['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['FSC_DHLG'])
tbl_DHLG['NQD_GIRTH_AND_L_DHL'] = tbl_DHLG['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['NQD_DHLG_GIRTH_AND_L'])
tbl_DHLG['NQD_OTHER_DHL'] = tbl_DHLG['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG.set_index('TRACKING_NUMBER')['NQD_DHLG_OTHER'])

tbl_DHLG1['FRT_DHL'] = tbl_DHLG1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['FRT_DHLG1'])
tbl_DHLG1['FSC%_DHL'] = tbl_DHLG1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['FSC%_DHLG1'])
tbl_DHLG1['FSC_DHL'] = tbl_DHLG1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['FSC_DHLG1'])
tbl_DHLG1['NQD_GIRTH_AND_L_DHL'] = tbl_DHLG1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['NQD_DHLG1_GIRTH_AND_L'])
tbl_DHLG1['NQD_OTHER_DHL'] = tbl_DHLG1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLG1.set_index('TRACKING_NUMBER')['NQD_DHLG1_OTHER'])

tbl_DHLE['FRT_DHL'] = tbl_DHLE['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['FRT_DHLE'])
tbl_DHLE['FSC%_DHL'] = tbl_DHLE['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['FSC%_DHLE'])
tbl_DHLE['FSC_DHL'] = tbl_DHLE['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['FSC_DHLE'])
tbl_DHLE['NQD_GIRTH_AND_L_DHL'] = tbl_DHLE['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['NQD_DHLE_GIRTH_AND_L'])
tbl_DHLE['NQD_OTHER_DHL'] = tbl_DHLE['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE.set_index('TRACKING_NUMBER')['NQD_DHLE_OTHER'])

tbl_DHLE1['FRT_DHL'] = tbl_DHLE1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['FRT_DHLE1'])
tbl_DHLE1['FSC%_DHL'] = tbl_DHLE1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['FSC%_DHLE1'])
tbl_DHLE1['FSC_DHL'] = tbl_DHLE1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['FSC_DHLE1'])
tbl_DHLE1['NQD_GIRTH_AND_L_DHL'] = tbl_DHLE1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['NQD_DHLE1_GIRTH_AND_L'])
tbl_DHLE1['NQD_OTHER_DHL'] = tbl_DHLE1['TRACKING_NUMBER'].map(tbl_PLD_original_DHLE1.set_index('TRACKING_NUMBER')['NQD_DHLE1_OTHER'])

tbl_DHLEM['FRT_DHL'] = tbl_DHLEM['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['FRT_DHLEM'])
tbl_DHLEM['FSC%_DHL'] = tbl_DHLEM['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['FSC%_DHLEM'])
tbl_DHLEM['FSC_DHL'] = tbl_DHLEM['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['FSC_DHLEM'])
tbl_DHLEM['NQD_GIRTH_AND_L_DHL'] = tbl_DHLEM['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['NQD_DHLEM_GIRTH_AND_L'])
tbl_DHLEM['NQD_OTHER_DHL'] = tbl_DHLEM['TRACKING_NUMBER'].map(tbl_PLD_original_DHLEM.set_index('TRACKING_NUMBER')['NQD_DHLEM_OTHER'])


tbl_USPSAG['FRT_USPS'] = tbl_USPSAG['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAG.set_index('TRACKING_NUMBER')['FRT_USPSAG'])
tbl_USPSAG['EXTRA_USPS_1'] = tbl_USPSAG['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAG.set_index('TRACKING_NUMBER')['EXTRA_USPSAG_1'])
tbl_USPSAG['EXTRA_USPS_2'] = tbl_USPSAG['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAG.set_index('TRACKING_NUMBER')['EXTRA_USPSAG_2'])

tbl_USPSAP['FRT_USPS'] = tbl_USPSAP['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAP.set_index('TRACKING_NUMBER')['FRT_USPSAP'])
tbl_USPSAP['EXTRA_USPS_1'] = tbl_USPSAP['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAP.set_index('TRACKING_NUMBER')['EXTRA_USPSAP_1'])
tbl_USPSAP['EXTRA_USPS_2'] = tbl_USPSAP['TRACKING_NUMBER'].map(tbl_PLD_original_USPSAP.set_index('TRACKING_NUMBER')['EXTRA_USPSAP_2'])


tbl_MIPLE['FRT_MIPLE'] = tbl_MIPLE['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['FRT_MIPLE'])
tbl_MIPLE['FSC_MIPLE'] = tbl_MIPLE['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['FSC_MIPLE'])
tbl_MIPLE['NQD_MIPLE'] = tbl_MIPLE['TRACKING_NUMBER'].map(tbl_PLD_original_MIPLE.set_index('TRACKING_NUMBER')['NQD_MIPLE'])

tbl_MIPH['FRT_MIPH'] = tbl_MIPH['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['FRT_MIPH'])
tbl_MIPH['FSC_MIPH'] = tbl_MIPH['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['FSC_MIPH'])
tbl_MIPH['NQD_MIPH'] = tbl_MIPH['TRACKING_NUMBER'].map(tbl_PLD_original_MIPH.set_index('TRACKING_NUMBER')['NQD_MIPH'])



#tbl_PLD_original_REDE, tbl_PLD_original_RED, tbl_PLD_original_REDS, tbl_PLD_original_2DAM, tbl_PLD_original_BLUE
#tbl_PLD_original_ORNG, tbl_PLD_original_GRND, tbl_PLD_original_GRES, tbl_PLD_original_SRPT1, tbl_PLD_original_SRPT
#tbl_PLD_original_DHLG, tbl_PLD_original_DHLG1, tbl_PLD_original_DHLE, tbl_PLD_original_DHLE1, tbl_PLD_original_DHLEM
#tbl_PLD_original_USPSAG, tbl_PLD_original_USPSAP, tbl_PLD_original_MIPLE, tbl_PLD_original_MIPH


In [818]:
tbl_PLD_rerated_bestWay = pd.concat([
   tbl_REDE, tbl_RED, tbl_REDS, tbl_2DAM, 
    tbl_BLUE, 
    tbl_ORNG, tbl_GRND, tbl_GRES, 
   tbl_SRPT1, tbl_SRPT, tbl_1STO, tbl_PROV, tbl_STOV, tbl_2AM, 
    tbl_2DAY, 
    tbl_EXSA, tbl_GRNDF, tbl_HOME, 
   tbl_SPST1, tbl_SPST,
    tbl_DHLG, tbl_DHLG1, 
    tbl_DHLE, tbl_DHLE1, tbl_DHLEM, 
    tbl_USPSAG, 
   tbl_USPSAP, 
   tbl_MIPLE, tbl_MIPH
])


In [819]:
# Replace 'inf' with an empty string in selected columns
columns_to_replace = [
   'TOTAL_REDE', 'TOTAL_RED', 'TOTAL_REDS', 'TOTAL_2DAM', 
    'TOTAL_BLUE', 
'TOTAL_ORNG', 'TOTAL_GRND', 'TOTAL_GRES', 
 'TOTAL_SRPT1','TOTAL_SRPT', 
    'TOTAL_1STO', 'TOTAL_PROV', 'TOTAL_STOV', 'TOTAL_2AM', 
    'TOTAL_2DAY', 
'TOTAL_EXSA', 'TOTAL_GRNDF', 'TOTAL_HOME', 
 'TOTAL_SPST1','TOTAL_SPST', 
    'TOTAL_DHLG', 'TOTAL_DHLG1', 
    'TOTAL_DHLE', 'TOTAL_DHLE1',
    'TOTAL_DHLEM', 
'TOTAL_USPSAG', 
    'TOTAL_USPSAP', 
    'TOTAL_MIPLE', 'TOTAL_MIPH',
    'CARRIER_COST_MIN']
tbl_PLD_rerated_bestWay[columns_to_replace] = tbl_PLD_rerated_bestWay[columns_to_replace].replace(np.inf, '')


In [820]:
# Update the values
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_REDE', 'SELECTED_SERVICE'] = 'UPS NDA Early'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_RED', 'SELECTED_SERVICE'] = 'UPS NDA'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_REDS', 'SELECTED_SERVICE'] = 'UPS NDA Saver'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2DAM', 'SELECTED_SERVICE'] = 'UPS 2DA A.M.'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_BLUE', 'SELECTED_SERVICE'] = 'UPS 2DA'

tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_ORNG', 'SELECTED_SERVICE'] = 'UPS 3DA'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRND', 'SELECTED_SERVICE'] = 'UPS Ground Commercial'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRES', 'SELECTED_SERVICE'] = 'UPS Ground Residential'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SRPT1', 'SELECTED_SERVICE'] = 'UPS Surepost 1#>'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SRPT', 'SELECTED_SERVICE'] = 'UPS Surepost'

tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_1STO', 'SELECTED_SERVICE'] = 'FedEx First Overnight'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_PROV', 'SELECTED_SERVICE'] = 'FedEx Priority Overnight'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_STOV', 'SELECTED_SERVICE'] = 'FedEx Standard Overnight'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2AM', 'SELECTED_SERVICE'] = 'FedEx 2nd Day AM'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_2DAY', 'SELECTED_SERVICE'] = 'FedEx 2Day'

tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_EXSA', 'SELECTED_SERVICE'] = 'FedEx Express Saver'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_GRNDF', 'SELECTED_SERVICE'] = 'FedEx Ground'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_HOME', 'SELECTED_SERVICE'] = 'FedEx Home Delivery'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SPST1', 'SELECTED_SERVICE'] = 'FedEx SmartPost 1#>'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_SPST', 'SELECTED_SERVICE'] = 'FedEx SmartPost'


tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLG', 'SELECTED_SERVICE'] = 'DHL SmartMail Parcel Plus Ground 1-25'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLG1', 'SELECTED_SERVICE'] = 'DHL SmartMail Parcel Ground < 1lb'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLE', 'SELECTED_SERVICE'] = 'DHL SmartMail Parcel Plus Expedited 1-25'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLE1', 'SELECTED_SERVICE'] = 'DHL SmartMail Parcel Expedited  < 1lb'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_DHLEM', 'SELECTED_SERVICE'] = 'DHL SmartMail Parcel Expedited Max'

tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_USPSAG', 'SELECTED_SERVICE'] = 'USPS Auctane GRND'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_USPSAP', 'SELECTED_SERVICE'] = 'USPS Auctane PM'

tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_MIPLE', 'SELECTED_SERVICE'] = 'MI Parcel Select Lightweight Expedited'
tbl_PLD_rerated_bestWay.loc[tbl_PLD_rerated_bestWay['SELECTED_SERVICE'] == 'TOTAL_MIPH', 'SELECTED_SERVICE'] = 'MI Parcel Select Heavyweight'



In [821]:
tbl_PLD_rerated_bestWay.to_csv('YEC_20%Other_9%CPP_BW.csv')

In [822]:
tbl_PLD_rerated_bestWay.shape

(95509, 58)

# ******End******

# Backup Materials


| Carrier | Service | Residential | WeightType | RateType   |
|:--------|:---------|:-------------|:------------|:------------|
| UPS     | REDE (UPS NDA Early) / RED (UPS NDA) / REDS (UPS NDA Saver) / 2DAM (UPS 2DA A.M.) / BLUE (UPS 2DA) / ORNG (UPS 3DA) / SRPT (UPS Surepost) | both         | lb         | Published   |
| UPS     | GRND (UPS Ground Commercial)                                                                                                              | commercial   | lb         | Published   |
| UPS     | GRES (UPS Ground Residential)                                                                                                             | residential  | lb         | Published   |
| UPS     | SRPT<1 (UPS Surepost 1#>)                                                                                                                 | both         | oz         | Published   |
| DHL     | DHLG (DHL SmartMail Parcel Plus Ground 1-25)/DHLE (DHL SmartMail Parcel Plus Expedited 1-25) | both | lb | Barrett |
| DHL     | DHLG<1 (DHL SmartMail Parcel Ground < 1lb)/DHLE<1 (DHL SmartMail Parcel Expedited  < 1lb)/DHLEM (DHL SmartMail Parcel Expedited Max) | both | lb | Barrett |
| USPS    | USPSAG (USPS Auctane GRND) | both | lb | Barrett |
| USPS    | USPSAP (USPS Auctane PM)| both | lb | Barrett |
| MI      | MIPLE (MI Parcel Select Lightweight Expedited)|  both | oz | Barrett |
| MI      | MIPH (MI Parcel Select Heavyweight)|  both | lb | Barrett |



In [16]:
import pandas as pd
import numpy as np
import math

# Read carrierRateCard_2023 and remake the tables showing below columns.
# SERVICE_CODE, SERVICE_PLAINTEXT, ZONE, WEIGHT_OZ, WEIGHT_LB, PUBLISHED_RATES, VERSION, START_DATE, END_DATE
xls = pd.ExcelFile('carrierRateCard_2024.xlsx')
# Load UPS rate card
df1 = pd.read_excel(xls, 'REDE')
df2 = pd.read_excel(xls, 'RED')
df3 = pd.read_excel(xls, 'REDS')
df4 = pd.read_excel(xls, '2DAM')
df5 = pd.read_excel(xls, 'BLUE')
df6 = pd.read_excel(xls, 'ORNG')
df7 = pd.read_excel(xls, 'GRND')
df8 = pd.read_excel(xls, 'GRES')
df9 = pd.read_excel(xls, 'SRPT<1')
df10 = pd.read_excel(xls, 'SRPT')

# Load USPS rate card
df16 = pd.read_excel(xls, 'USPSAG_CPP')
df17 = pd.read_excel(xls, 'USPSAP_CPP')
df20 = pd.read_excel(xls, 'USPSAG_CPPCI')
df21 = pd.read_excel(xls, 'USPSAP_CPPCI')
df22 = pd.read_excel(xls, 'USPSAG_AUCT')
df23 = pd.read_excel(xls, 'USPSAP_AUCT')
df24 = pd.read_excel(xls, 'USPSAG_AUCTCI')
df25 = pd.read_excel(xls, 'USPSAP_AUCTCI')

# Load MI rate card
df18 = pd.read_excel(xls, 'MIPLE')
df19 = pd.read_excel(xls, 'MIPH')

# Load DHL rate card
# 5114358_dhl_sm_parcel_Franklin
df11_1 = pd.read_excel(xls, 'DHLG_5114358')
df12_2 = pd.read_excel(xls, 'DHLG<1_5114358')
df13_3 = pd.read_excel(xls, 'DHLE_5114358')
df14_4 = pd.read_excel(xls, 'DHLE<1_5114358')
df15_5 = pd.read_excel(xls, 'DHLEM_5114358')

# 5122444_dhl_sm_parcel_Somerset
df11_6 = pd.read_excel(xls, 'DHLG_5122444')
df12_7 = pd.read_excel(xls, 'DHLG<1_5122444')
df13_8 = pd.read_excel(xls, 'DHLE_5122444')
df14_9 = pd.read_excel(xls, 'DHLE<1_5122444')
df15_10 = pd.read_excel(xls, 'DHLEM_5122444')

# 5122723_dhl_sm_parcel_8150 Nail Olive Branch
df11_11 = pd.read_excel(xls, 'DHLG_5122723')
df12_12 = pd.read_excel(xls, 'DHLG<1_5122723')
df13_13 = pd.read_excel(xls, 'DHLE_5122723')
df14_14 = pd.read_excel(xls, 'DHLE<1_5122723')
df15_15 = pd.read_excel(xls, 'DHLEM_5122723')

# 5122739_dhl_sm_parcel_Nail rd Olive branch
df11_16 = pd.read_excel(xls, 'DHLG_5122739')
df12_17 = pd.read_excel(xls, 'DHLG<1_5122739')
df13_18 = pd.read_excel(xls, 'DHLE_5122739')
df14_19 = pd.read_excel(xls, 'DHLE<1_5122739')
df15_20 = pd.read_excel(xls, 'DHLEM_5122739')

# 5122855_dhl_sm_parcel_Garland TX
df11_21 = pd.read_excel(xls, 'DHLG_5122855')
df12_22 = pd.read_excel(xls, 'DHLG<1_5122855')
df13_23 = pd.read_excel(xls, 'DHLE_5122855')
df14_24 = pd.read_excel(xls, 'DHLE<1_5122855')
df15_25 = pd.read_excel(xls, 'DHLEM_5122855')

# 5122890_dhl_sm_parcel_Oofos
df11_26 = pd.read_excel(xls, 'DHLG_5122890')
df12_27 = pd.read_excel(xls, 'DHLG<1_5122890')
df13_28 = pd.read_excel(xls, 'DHLE_5122890')
df14_29 = pd.read_excel(xls, 'DHLE<1_5122890')
df15_30 = pd.read_excel(xls, 'DHLEM_5122890')

# 5122893_dhl_sm_parcel Memphis
df11_31 = pd.read_excel(xls, 'DHLG_5122893')
df12_32 = pd.read_excel(xls, 'DHLG<1_5122893')
df13_33 = pd.read_excel(xls, 'DHLE_5122893')
df14_34 = pd.read_excel(xls, 'DHLE<1_5122893')
df15_35 = pd.read_excel(xls, 'DHLEM_5122893')

# 5123280_dhl_sm_parcel_MAN
df11_36 = pd.read_excel(xls, 'DHLG_5123280')
df12_37 = pd.read_excel(xls, 'DHLG<1_5123280')
df13_38 = pd.read_excel(xls, 'DHLE_5123280')
df14_39 = pd.read_excel(xls, 'DHLE<1_5123280')
df15_40 = pd.read_excel(xls, 'DHLEM_5123280')

# 5123282_dhl_sm_parcel Bridgewater
df11_41 = pd.read_excel(xls, 'DHLG_5123282')
df12_42 = pd.read_excel(xls, 'DHLG<1_5123282')
df13_43 = pd.read_excel(xls, 'DHLE_5123282')
df14_44 = pd.read_excel(xls, 'DHLE<1_5123282')
df15_45 = pd.read_excel(xls, 'DHLEM_5123282')

# 5123283_dhl_sm_parcel_Curtis Bay
df11_46 = pd.read_excel(xls, 'DHLG_5123283')
df12_47 = pd.read_excel(xls, 'DHLG<1_5123283')
df13_48 = pd.read_excel(xls, 'DHLE_5123283')
df14_49 = pd.read_excel(xls, 'DHLE<1_5123283')
df15_50 = pd.read_excel(xls, 'DHLEM_5123283')

# 5123283_dhl_sm_parcel_Curtis Bay
df11_51 = pd.read_excel(xls, 'DHLG_5123556')
df12_52 = pd.read_excel(xls, 'DHLG<1_5123556')
df13_53 = pd.read_excel(xls, 'DHLE_5123556')
df14_54 = pd.read_excel(xls, 'DHLE<1_5123556')
df15_55 = pd.read_excel(xls, 'DHLEM_5123556')


df26 = pd.read_excel(xls, '1STO')
df27 = pd.read_excel(xls, 'PROV')
df28 = pd.read_excel(xls, 'STOV')
df29 = pd.read_excel(xls, '2AM')
df30 = pd.read_excel(xls, '2DAY')
df31 = pd.read_excel(xls, 'EXSA')
df32 = pd.read_excel(xls, 'GRNDF')
df33 = pd.read_excel(xls, 'HOME')
df34 = pd.read_excel(xls, 'SPST<1')
df35 = pd.read_excel(xls, 'SPST')



In [17]:
# Re-arrange the tables
# REDE, Barrett Rates = Published Rates. 
# made sure the tables are completely loaded into the dataframe by checking df1.shape

df1 = df1.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df2 = df2.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df3 = df3.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df4 = df4.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df5 = df5.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df6 = df6.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df7 = df7.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df8 = df8.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df9 = df9.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='PUBLISHED_RATES')
df10 = df10.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')


# put cpp price as Barrett Rate to ease the calculation
df16 = df16.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df17 = df17.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df20 = df20.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df21 = df21.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df22 = df22.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df23 = df23.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df24 = df24.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')
df25 = df25.set_index('Zones').unstack().rename_axis(('ZONE','CUBIC_FT')).reset_index(name='BARRETT_RATES')

df18 = df18.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df19 = df19.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')

df11_1 = df11_1.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_2 = df12_2.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_3 = df13_3.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_4 = df14_4.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_5 = df15_5.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_6 = df11_6.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_7 = df12_7.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_8 = df13_8.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_9 = df14_9.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_10 = df15_10.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_11 = df11_11.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_12 = df12_12.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_13 = df13_13.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_14 = df14_14.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_15 = df15_15.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_16 = df11_16.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_17 = df12_17.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_18 = df13_18.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_19 = df14_19.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_20 = df15_20.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_21 = df11_21.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_22 = df12_22.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_23 = df13_23.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_24 = df14_24.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_25 = df15_25.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_26 = df11_26.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_27 = df12_27.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_28 = df13_28.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_29 = df14_29.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_30 = df15_30.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_31 = df11_31.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_32 = df12_32.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_33 = df13_33.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_34 = df14_34.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_35 = df15_35.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_36 = df11_36.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_37 = df12_37.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_38 = df13_38.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_39 = df14_39.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_40 = df15_40.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_41 = df11_41.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_42 = df12_42.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_43 = df13_43.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_44 = df14_44.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_45 = df15_45.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_46 = df11_46.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_47 = df12_47.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_48 = df13_48.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_49 = df14_49.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_50 = df15_50.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df11_51 = df11_51.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df12_52 = df12_52.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df13_53 = df13_53.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='BARRETT_RATES')
df14_54 = df14_54.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')
df15_55 = df15_55.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='BARRETT_RATES')

df26 = df26.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df27 = df27.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df28 = df28.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df29 = df29.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df30 = df30.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df31 = df31.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df32 = df32.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df33 = df33.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')
df34 = df34.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_OZ')).reset_index(name='PUBLISHED_RATES')
df35 = df35.set_index('Zones').unstack().rename_axis(('ZONE','WEIGHT_LB')).reset_index(name='PUBLISHED_RATES')


# Make sure the dataframes have both weight in lb and oz columns
df1['WEIGHT_OZ'] = df1['WEIGHT_LB']*16
df2['WEIGHT_OZ'] = df2['WEIGHT_LB']*16
df3['WEIGHT_OZ'] = df3['WEIGHT_LB']*16
df4['WEIGHT_OZ'] = df4['WEIGHT_LB']*16
df5['WEIGHT_OZ'] = df5['WEIGHT_LB']*16
df6['WEIGHT_OZ'] = df6['WEIGHT_LB']*16
df7['WEIGHT_OZ'] = df7['WEIGHT_LB']*16
df8['WEIGHT_OZ'] = df8['WEIGHT_LB']*16
df9['WEIGHT_LB'] = df9['WEIGHT_OZ']/16
df10['WEIGHT_OZ'] = df10['WEIGHT_LB']*16

df16['WEIGHT_OZ'] = df16['WEIGHT_LB']*16
df17['WEIGHT_OZ'] = df17['WEIGHT_LB']*16

df22['WEIGHT_OZ'] = df22['WEIGHT_LB']*16
df23['WEIGHT_OZ'] = df23['WEIGHT_LB']*16

df18['WEIGHT_LB'] = df18['WEIGHT_OZ']/16
df19['WEIGHT_OZ'] = df19['WEIGHT_LB']*16

df11_1['WEIGHT_OZ'] = df11_1['WEIGHT_LB']*16
df12_2['WEIGHT_LB'] = df12_2['WEIGHT_OZ']/16
df13_3['WEIGHT_OZ'] = df13_3['WEIGHT_LB']*16
df14_4['WEIGHT_LB'] = df14_4['WEIGHT_OZ']/16
df15_5['WEIGHT_LB'] = df15_5['WEIGHT_OZ']/16

df11_6['WEIGHT_OZ'] = df11_6['WEIGHT_LB']*16
df12_7['WEIGHT_LB'] = df12_7['WEIGHT_OZ']/16
df13_8['WEIGHT_OZ'] = df13_8['WEIGHT_LB']*16
df14_9['WEIGHT_LB'] = df14_9['WEIGHT_OZ']/16
df15_10['WEIGHT_LB'] = df15_10['WEIGHT_OZ']/16

df11_11['WEIGHT_OZ'] = df11_11['WEIGHT_LB']*16
df12_12['WEIGHT_LB'] = df12_12['WEIGHT_OZ']/16
df13_13['WEIGHT_OZ'] = df13_13['WEIGHT_LB']*16
df14_14['WEIGHT_LB'] = df14_14['WEIGHT_OZ']/16
df15_15['WEIGHT_LB'] = df15_15['WEIGHT_OZ']/16

df11_16['WEIGHT_OZ'] = df11_16['WEIGHT_LB']*16
df12_17['WEIGHT_LB'] = df12_17['WEIGHT_OZ']/16
df13_18['WEIGHT_OZ'] = df13_18['WEIGHT_LB']*16
df14_19['WEIGHT_LB'] = df14_19['WEIGHT_OZ']/16
df15_20['WEIGHT_LB'] = df15_20['WEIGHT_OZ']/16

df11_21['WEIGHT_OZ'] = df11_21['WEIGHT_LB']*16
df12_22['WEIGHT_LB'] = df12_22['WEIGHT_OZ']/16
df13_23['WEIGHT_OZ'] = df13_23['WEIGHT_LB']*16
df14_24['WEIGHT_LB'] = df14_24['WEIGHT_OZ']/16
df15_25['WEIGHT_LB'] = df15_25['WEIGHT_OZ']/16

df11_26['WEIGHT_OZ'] = df11_26['WEIGHT_LB']*16
df12_27['WEIGHT_LB'] = df12_27['WEIGHT_OZ']/16
df13_28['WEIGHT_OZ'] = df13_28['WEIGHT_LB']*16
df14_29['WEIGHT_LB'] = df14_29['WEIGHT_OZ']/16
df15_30['WEIGHT_LB'] = df15_30['WEIGHT_OZ']/16

df11_31['WEIGHT_OZ'] = df11_31['WEIGHT_LB']*16
df12_32['WEIGHT_LB'] = df12_32['WEIGHT_OZ']/16
df13_33['WEIGHT_OZ'] = df13_33['WEIGHT_LB']*16
df14_34['WEIGHT_LB'] = df14_34['WEIGHT_OZ']/16
df15_35['WEIGHT_LB'] = df15_35['WEIGHT_OZ']/16

df11_36['WEIGHT_OZ'] = df11_36['WEIGHT_LB']*16
df12_37['WEIGHT_LB'] = df12_37['WEIGHT_OZ']/16
df13_38['WEIGHT_OZ'] = df13_38['WEIGHT_LB']*16
df14_39['WEIGHT_LB'] = df14_39['WEIGHT_OZ']/16
df15_40['WEIGHT_LB'] = df15_40['WEIGHT_OZ']/16

df11_41['WEIGHT_OZ'] = df11_41['WEIGHT_LB']*16
df12_42['WEIGHT_LB'] = df12_42['WEIGHT_OZ']/16
df13_43['WEIGHT_OZ'] = df13_43['WEIGHT_LB']*16
df14_44['WEIGHT_LB'] = df14_44['WEIGHT_OZ']/16
df15_45['WEIGHT_LB'] = df15_45['WEIGHT_OZ']/16

df11_46['WEIGHT_OZ'] = df11_46['WEIGHT_LB']*16
df12_47['WEIGHT_LB'] = df12_47['WEIGHT_OZ']/16
df13_48['WEIGHT_OZ'] = df13_48['WEIGHT_LB']*16
df14_49['WEIGHT_LB'] = df14_49['WEIGHT_OZ']/16
df15_50['WEIGHT_LB'] = df15_50['WEIGHT_OZ']/16


df11_51['WEIGHT_OZ'] = df11_51['WEIGHT_LB']*16
df12_52['WEIGHT_LB'] = df12_52['WEIGHT_OZ']/16
df13_53['WEIGHT_OZ'] = df13_53['WEIGHT_LB']*16
df14_54['WEIGHT_LB'] = df14_54['WEIGHT_OZ']/16
df15_55['WEIGHT_LB'] = df15_55['WEIGHT_OZ']/16

df26['WEIGHT_OZ'] = df26['WEIGHT_LB']*16
df27['WEIGHT_OZ'] = df27['WEIGHT_LB']*16
df28['WEIGHT_OZ'] = df28['WEIGHT_LB']*16
df29['WEIGHT_OZ'] = df29['WEIGHT_LB']*16
df30['WEIGHT_OZ'] = df30['WEIGHT_LB']*16
df31['WEIGHT_OZ'] = df31['WEIGHT_LB']*16
df32['WEIGHT_OZ'] = df32['WEIGHT_LB']*16
df33['WEIGHT_OZ'] = df33['WEIGHT_LB']*16
df34['WEIGHT_LB'] = df34['WEIGHT_OZ']/16
df35['WEIGHT_OZ'] = df35['WEIGHT_LB']*16


# use below if the charts have empty spaces. No need to use here as the source tables are already clean.
# df1 = df1.dropna(axis = 0, how = 'all')
# df1 = df1.dropna(axis = 1, how = 'all')
# df1 = df1.dropna()


# Add Service code and Service columns
df1['SERVICE_PLAINTEXT']='UPS NDA Early'
df2['SERVICE_PLAINTEXT']='UPS NDA'
df3['SERVICE_PLAINTEXT']='UPS NDA Saver'
df4['SERVICE_PLAINTEXT']='UPS 2DA A.M.'
df5['SERVICE_PLAINTEXT']='UPS 2DA'
df6['SERVICE_PLAINTEXT']='UPS 3DA'
df7['SERVICE_PLAINTEXT']='UPS Ground Commercial'
df8['SERVICE_PLAINTEXT']='UPS Ground Residential'
df9['SERVICE_PLAINTEXT']='UPS Surepost 1#>'
df10['SERVICE_PLAINTEXT']='UPS Surepost'

df16['SERVICE_PLAINTEXT']='USPS Ground Cpp'
df17['SERVICE_PLAINTEXT']='USPS Priority Cpp'

df20['SERVICE_PLAINTEXT']='USPS Ground Cpp Cubic Inch'
df21['SERVICE_PLAINTEXT']='USPS Priority Cpp Cubic Inch'

df22['SERVICE_PLAINTEXT']='USPS Ground Auctane'
df23['SERVICE_PLAINTEXT']='USPS Priority Auctane'

df24['SERVICE_PLAINTEXT']='USPS Ground Auctane Cubic Inch'
df25['SERVICE_PLAINTEXT']='USPS Priority Auctane Cubic Inch'

df18['SERVICE_PLAINTEXT']='MI Parcel Select Lightweight Expedited'
df19['SERVICE_PLAINTEXT']='MI Parcel Select Heavyweight'

# Load DHL rate card
# 5114358_dhl_sm_parcel_Franklin
# 5122444_dhl_sm_parcel_Somerset, no DHLEM
# 5122723_dhl_sm_parcel_8150 Nail Olive Branch
# 5122739_dhl_sm_parcel_Nail rd Olive branch
# 5122855_dhl_sm_parcel_Garland TX
# 5122890_dhl_sm_parcel_Oofos
# 5122893_dhl_sm_parcell Memphis
# 5123280_dhl_sm_parcel_MAN
# 5123282_dhl_sm_parcel Bridgewater
# 5123283_dhl_sm_parcel_Curtis Bay

df11_1['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5114358'
df12_2['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5114358'
df13_3['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5114358'
df14_4['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5114358'
df15_5['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5114358'

df11_6['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122444'
df12_7['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122444'
df13_8['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122444'
df14_9['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122444'
df15_10['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122444'

df11_11['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122723'
df12_12['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122723'
df13_13['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122723'
df14_14['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122723'
df15_15['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122723'

df11_16['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122739'
df12_17['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122739'
df13_18['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122739'
df14_19['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122739'
df15_20['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122739'

df11_21['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122855'
df12_22['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122855'
df13_23['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122855'
df14_24['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122855'
df15_25['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122855'

df11_26['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122890'
df12_27['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122890'
df13_28['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122890'
df14_29['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122890'
df15_30['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122890'

df11_31['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5122893'
df12_32['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5122893'
df13_33['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5122893'
df14_34['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5122893'
df15_35['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5122893'

df11_36['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123280'
df12_37['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123280'
df13_38['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123280'
df14_39['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123280'
df15_40['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123280'

df11_41['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123282'
df12_42['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123282'
df13_43['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123282'
df14_44['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123282'
df15_45['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123282'

df11_46['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123283'
df12_47['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123283'
df13_48['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123283'
df14_49['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123283'
df15_50['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123283'

df11_51['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Ground 1-25 5123556'
df12_52['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Ground < 1lb 5123556'
df13_53['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Plus Expedited 1-25 5123556'
df14_54['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited < 1lb 5123556'
df15_55['SERVICE_PLAINTEXT']='DHL SmartMail Parcel Expedited Max 5123556'

df26['SERVICE_PLAINTEXT']='FedEx First Overnight'
df27['SERVICE_PLAINTEXT']='FedEx Priority Overnight'
df28['SERVICE_PLAINTEXT']='FedEx Standard Overnight'
df29['SERVICE_PLAINTEXT']='FedEx 2nd Day AM'
df30['SERVICE_PLAINTEXT']='FedEx 2Day'
df31['SERVICE_PLAINTEXT']='FedEx Express Saver'
df32['SERVICE_PLAINTEXT']='FedEx Ground'
df33['SERVICE_PLAINTEXT']='FedEx Home Delivery'
df34['SERVICE_PLAINTEXT']='FedEx SmartPost 1#>'
df35['SERVICE_PLAINTEXT']='FedEx SmartPost'



df1['SERVICE_CODE']='REDE'
df2['SERVICE_CODE']='RED'
df3['SERVICE_CODE']='REDS'
df4['SERVICE_CODE']='2DAM'
df5['SERVICE_CODE']='BLUE'
df6['SERVICE_CODE']='ORNG'
df7['SERVICE_CODE']='GRND'
df8['SERVICE_CODE']='GRES'
df9['SERVICE_CODE']='SRPT<1'
df10['SERVICE_CODE']='SRPT'                               
                             
df16['SERVICE_CODE']='USPSAG_CPP'
df17['SERVICE_CODE']='USPSAP_CPP'

df20['SERVICE_CODE']='USPSAG_CPPCI'
df21['SERVICE_CODE']='USPSAP_CPPCI'

df22['SERVICE_CODE']='USPSAG_AUCT'
df23['SERVICE_CODE']='USPSAP_AUCT'

df24['SERVICE_CODE']='USPSAG_AUCTCI'
df25['SERVICE_CODE']='USPSAP_AUCTCI'


df18['SERVICE_CODE']='MIPLE'
df19['SERVICE_CODE']='MIPH'

df11_1['SERVICE_CODE']='DHLG_5114358'
df12_2['SERVICE_CODE']='DHLG<1_5114358'
df13_3['SERVICE_CODE']='DHLE_5114358'
df14_4['SERVICE_CODE']='DHLE<1_5114358'
df15_5['SERVICE_CODE']='DHLEM_5114358'

df11_6['SERVICE_CODE']='DHLG_5122444'
df12_7['SERVICE_CODE']='DHLG<1_5122444'
df13_8['SERVICE_CODE']='DHLE_5122444'
df14_9['SERVICE_CODE']='DHLE<1_5122444'
df15_10['SERVICE_CODE']='DHLEM_5122444'

df11_11['SERVICE_CODE']='DHLG_5122723'
df12_12['SERVICE_CODE']='DHLG<1_5122723'
df13_13['SERVICE_CODE']='DHLE_5122723'
df14_14['SERVICE_CODE']='DHLE<1_5122723'
df15_15['SERVICE_CODE']='DHLEM_5122723'

df11_16['SERVICE_CODE']='DHLG_5122739'
df12_17['SERVICE_CODE']='DHLG<1_5122739'
df13_18['SERVICE_CODE']='DHLE_5122739'
df14_19['SERVICE_CODE']='DHLE<1_5122739'
df15_20['SERVICE_CODE']='DHLEM_5122739'

df11_21['SERVICE_CODE']='DHLG_5122855'
df12_22['SERVICE_CODE']='DHLG<1_5122855'
df13_23['SERVICE_CODE']='DHLE_5122855'
df14_24['SERVICE_CODE']='DHLE<1_5122855'
df15_25['SERVICE_CODE']='DHLEM_5122855'

df11_26['SERVICE_CODE']='DHLG_5122890'
df12_27['SERVICE_CODE']='DHLG<1_5122890'
df13_28['SERVICE_CODE']='DHLE_5122890'
df14_29['SERVICE_CODE']='DHLE<1_5122890'
df15_30['SERVICE_CODE']='DHLEM_5122890'

df11_31['SERVICE_CODE']='DHLG_5122893'
df12_32['SERVICE_CODE']='DHLG<1_5122893'
df13_33['SERVICE_CODE']='DHLE_5122893'
df14_34['SERVICE_CODE']='DHLE<1_5122893'
df15_35['SERVICE_CODE']='DHLEM_5122893'

df11_36['SERVICE_CODE']='DHLG_5123280'
df12_37['SERVICE_CODE']='DHLG<1_5123280'
df13_38['SERVICE_CODE']='DHLE_5123280'
df14_39['SERVICE_CODE']='DHLE<1_5123280'
df15_40['SERVICE_CODE']='DHLEM_5123280'

df11_41['SERVICE_CODE']='DHLG_5123282'
df12_42['SERVICE_CODE']='DHLG<1_5123282'
df13_43['SERVICE_CODE']='DHLE_5123282'
df14_44['SERVICE_CODE']='DHLE<1_5123282'
df15_45['SERVICE_CODE']='DHLEM_5123282'

df11_46['SERVICE_CODE']='DHLG_5123283'
df12_47['SERVICE_CODE']='DHLG<1_5123283'
df13_48['SERVICE_CODE']='DHLE_5123283'
df14_49['SERVICE_CODE']='DHLE<1_5123283'
df15_50['SERVICE_CODE']='DHLEM_5123283'

df11_51['SERVICE_CODE']='DHLG_5123556'
df12_52['SERVICE_CODE']='DHLG<1_5123556'
df13_53['SERVICE_CODE']='DHLE_5123556'
df14_54['SERVICE_CODE']='DHLE<1_5123556'
df15_55['SERVICE_CODE']='DHLEM_5123556'


df26['SERVICE_CODE']='1STO'
df27['SERVICE_CODE']='PROV'
df28['SERVICE_CODE']='STOV'
df29['SERVICE_CODE']='2AM'
df30['SERVICE_CODE']='2DAY'
df31['SERVICE_CODE']='EXSA'
df32['SERVICE_CODE']='GRNDF'
df33['SERVICE_CODE']='HOME'
df34['SERVICE_CODE']='SPST<1'
df35['SERVICE_CODE']='SPST'  


#Concatenate all together after rearranging the columns
df1 = df1.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df2 = df2.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df3 = df3.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df4 = df4.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df5 = df5.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df6 = df6.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df7 = df7.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df8 = df8.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df9 = df9.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df10 = df10.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])

df16 = df16.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df17 = df17.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df20 = df20.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])
df21 = df21.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])

df22 = df22.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df23 = df23.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df24 = df24.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])
df25 = df25.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'CUBIC_FT', 'BARRETT_RATES'])


df18 = df18.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df19 = df19.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_1 = df11_1.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_2 = df12_2.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_3 = df13_3.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_4 = df14_4.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_5 = df15_5.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_6 = df11_6.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_7 = df12_7.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_8 = df13_8.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_9 = df14_9.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_10 = df15_10.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_11 = df11_11.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_12 = df12_12.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_13 = df13_13.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_14 = df14_14.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_15 = df15_15.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_16 = df11_16.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_17 = df12_17.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_18 = df13_18.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_19 = df14_19.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_20 = df15_20.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_21 = df11_21.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_22 = df12_22.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_23 = df13_23.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_24 = df14_24.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_25 = df15_25.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_26 = df11_26.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_27 = df12_27.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_28 = df13_28.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_29 = df14_29.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_30 = df15_30.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_31 = df11_31.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_32 = df12_32.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_33 = df13_33.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_34 = df14_34.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_35 = df15_35.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_36 = df11_36.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_37 = df12_37.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_38 = df13_38.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_39 = df14_39.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_40 = df15_40.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_41 = df11_41.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_42 = df12_42.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_43 = df13_43.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_44 = df14_44.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_45 = df15_45.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_46 = df11_46.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_47 = df12_47.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_48 = df13_48.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_49 = df14_49.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_50 = df15_50.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df11_51 = df11_51.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df12_52 = df12_52.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df13_53 = df13_53.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df14_54 = df14_54.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])
df15_55 = df15_55.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'BARRETT_RATES'])

df26 = df26.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df27 = df27.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df28 = df28.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df29 = df29.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df30 = df30.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df31 = df31.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df32 = df32.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df33 = df33.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df34 = df34.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])
df35 = df35.reindex(columns=['SERVICE_CODE', 'SERVICE_PLAINTEXT', 'ZONE', 'WEIGHT_OZ', 'WEIGHT_LB', 'PUBLISHED_RATES'])



tbl_publishedRates = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df16, df17, df20, df21, df22, df23, df24, df25, df18, df19,
                               df11_1, df12_2, df13_3, df14_4, df15_5, df11_6, df12_7, df13_8, df14_9, 
                               df11_11, df12_12, df13_13, df14_14, df15_15, df11_16, df12_17, df13_18, df14_19, df15_20, 
                               df11_21, df12_22, df13_23, df14_24, df15_25, df11_26, df12_27, df13_28, df14_29, df15_30, 
                               df11_31, df12_32, df13_33, df14_34, df15_35, df11_36, df12_37, df13_38, df14_39, df15_40, 
                               df11_41, df12_42, df13_43, df14_44, df15_45, df11_46, df12_47, df13_48, df14_49, df15_50,
                               df11_51, df12_52, df13_53, df14_54, df15_55, df26, df27, df28, df29, df30, df31, df32, df33, df34, df35
                               ], axis=0)



In [18]:
# Add the version column
tbl_publishedRates['VERSION'] ='2024'
tbl_publishedRates['START_DATE'] ='2024/01/01'
tbl_publishedRates['END_DATE'] ='2024/12/31'

# Unify the Zone values
def get_portion(value):
    if isinstance(value, int):
        return str(value)  # convert integer to string
    if 'Zone' in value:
        return value[5:]
    elif 'US' in value:
        return value[3:]
    
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE'].apply(get_portion)
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('20', '')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('30', '')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('102', '2')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('103', '3')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('104', '4')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('105', '5')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('106', '6')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('107', '7')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('108', '8')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('124', '9')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('125', '10')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('126', '11')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('224', '9')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('225', '10')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('226', '11')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('44', '9')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('45', '10')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('46', '11')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('03', '3')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('04', '4')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('05', '5')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('06', '6')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('07', '7')
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('08', '8')   
tbl_publishedRates['ZONE_new'] = tbl_publishedRates['ZONE_new'].str.replace('09', '9')  

#delete the original Zone column and use the new one
tbl_publishedRates['ZONE'] = tbl_publishedRates['ZONE_new']
tbl_publishedRates = tbl_publishedRates.drop('ZONE_new', axis=1)

#adjust the index and finish creating this table.
tbl_publishedRates.reset_index(drop=True, inplace=True)

# tbl_publishedRates.head()
# tbl_publishedRates.to_csv('test.csv')





In [19]:
# Keep rows where 'ServiceCode' is not in ['cpp1', 'cpp2', 'cpp3']
tbl_publishedRates = tbl_publishedRates[~tbl_publishedRates['SERVICE_CODE'].isin(['USPSAG_AUCT', 'USPSAP_AUCT', 'USPSAG_AUCTCI', 'USPSAP_AUCTCI'])]
         
# 2. Replacing values
# Define a mapping of old values to new values
replacement_dict = {
    'USPSAG_CPP': 'USPSAG',
    'USPSAP_CPP': 'USPSAP',
    'USPSAG_CPPCI': 'USPSAG_CI',
    'USPSAP_CPPCI': 'USPSAP_CI'
    
}

# Replace the values in 'ServiceCode' column
tbl_publishedRates['SERVICE_CODE'] = tbl_publishedRates['SERVICE_CODE'].replace(replacement_dict)



# Step 1 - b. Create Freight Rate Card (tbl_BarrettRates)
*Add ups BarrettRates to tbl_publishedRates

In [20]:
#The result is BarrettRate when margin = 0.
#For UPS client freight & accessorial discounts, write "YES"/"NO" for rounding/not rounding to 2 decimal places.
Round_Discount = "NO"

#Set UPS margin for the client
Margin_on_Min_GRND_BR_UPS = 0
Margin_on_Min_AIR_BR_UPS = 0

Margin_on_UPS_Parcel_Freight_GRND_BR = 0
Margin_on_UPS_Parcel_Freight_AIR_BR = 0

Margin_on_Value_Add_GRND_BR_UPS = 0
Margin_on_Value_Add_AIR_BR_UPS = 0


# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('Accessorial_2024.xlsx')
UPS_Rates = pd.read_excel(xls, 'UPS_Rates')

# This is the value of the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates


#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 6
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 8
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-16oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-3lb'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-6lb'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '7-8lb'):
        return 7
    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '9lb'):
        return 9 

UPS_Rates['WEIGHT_LOWER'] = UPS_Rates.apply(g, axis=1)
UPS_Rates['WEIGHT_UPPER'] = UPS_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'Next Day Air') | (col['SERVICE_LEVEL'] == 'Next Day Air Saver') | (col['SERVICE_LEVEL'] == '2nd A.M. Day Air') | (col['SERVICE_LEVEL'] == '2nd Day Air') | (col['SERVICE_LEVEL'] == '3 Day Select'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR_BR_UPS))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_UPS))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        try: 
            return min(9.1, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_UPS))
        except ValueError:
            return 0
    
    elif (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb'): 
        try: 
            return min(8.86, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_UPS))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'Ground CWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_UPS))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_UPS))
        except ValueError:
            return 0

UPS_Rates['CLIENT_MARGIN_MINS'] = UPS_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
UPS_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - UPS_Rates['NET_CONTRACT_DISC']
UPS_Rates.loc[UPS_Rates['SERVICE_LEVEL'] == 'Next Day Air Early AM', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '9lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR_BR)
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    elif (col['SERVICE_LEVEL'] == 'Ground Commercial') | (col['SERVICE_LEVEL'] == 'Ground Residential') | (col['SERVICE_LEVEL'] == 'SurePost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'SurePost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_GRND_BR)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_UPS_Parcel_Freight_AIR_BR)

UPS_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = UPS_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'Next Day Air Early AM':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

UPS_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = UPS_Rates.apply(j, axis=1)


# This completes the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc
# UPS_Rates



In [21]:
#The result is BarrettRate when margin = 0.
#For UPS client freight & accessorial discounts, write "YES"/"NO" for rounding/not rounding to 2 decimal places.
Round_Discount = "NO"

BarrettDisc_1STO_FedEx = 0.55

#Set UPS margin for the client
Margin_on_Min_GRND_BR_FedEx = 0
Margin_on_Min_AIR_BR_FedEx = 0

Margin_on_FedEx_Parcel_Freight_GRND_BR = 0
Margin_on_FedEx_Parcel_Freight_AIR_BR = 0

Margin_on_Value_Add_GRND_BR_FedEx = 0
Margin_on_Value_Add_AIR_BR_FedEx = 0


# UPSAccessorial_2023 has 2 tabs, UPS_Rates and UPS_Accessorial. UPS_Rates is used to create Barrett Rates
xls = pd.ExcelFile('Accessorial_2024.xlsx')
FedEx_Rates = pd.read_excel(xls, 'FedEx_Rates')

# This is the value of the left side of "Rates" tab from "Customer Rate Contract Cards 2023" doc


#add 2 columns to this table with min and max weight
def f(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 5
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 10
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 20
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 30
    elif (col['WEIGHT_BREAK'] == '31+lb'):
        return 999
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-3oz'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '4-16oz'):
        return 16
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-2lbs'):
        return 2
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '3 lbs'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-70lbs'):
        return 70
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '70+lbs'):
        return 999  

def g(col):
    if col['WEIGHT_BREAK'] == '1-5lb':
        return 1
    elif col['WEIGHT_BREAK'] == '6-10lb':
        return 6
    elif col['WEIGHT_BREAK'] == '11-20lb':
        return 11
    elif col['WEIGHT_BREAK'] == '21-30lb':
        return 21
    elif col['WEIGHT_BREAK'] == '31+lb':
        return 31
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '1-3oz'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') & (col['WEIGHT_BREAK'] == '4-16oz'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '1-2lbs'):
        return 1
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '3 lbs'):
        return 3
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '4-70lbs'):
        return 4
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') & (col['WEIGHT_BREAK'] == '70+lbs'):
        return 70

FedEx_Rates['WEIGHT_LOWER'] = FedEx_Rates.apply(g, axis=1)
FedEx_Rates['WEIGHT_UPPER'] = FedEx_Rates.apply(f, axis=1)


# add client margin mins column
def h(col):   
    if col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return col['MIN_WITH_REBATE']
    
    elif (col['SERVICE_LEVEL'] == 'FedEx Priority Overnight') | (col['SERVICE_LEVEL'] == 'FedEx Standard Overnight') | (col['SERVICE_LEVEL'] == 'FedEx 2nd Day AM') | (col['SERVICE_LEVEL'] == 'FedEx 2Day') | (col['SERVICE_LEVEL'] == 'FedEx Express Saver'): 
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_AIR_BR_FedEx))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'FedEx Ground') | (col['SERVICE_LEVEL'] == 'FedEx Home Delivery'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_FedEx))
        except ValueError:
            return 0

    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_FedEx))
        except ValueError:
            return 0
    
    elif (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb'): 
        try: 
            return min(10.7, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_FedEx))
        except ValueError:
            return 0
  
    elif col['SERVICE_LEVEL'] == 'FedEx MWT': 
        try: 
            return min(80, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_FedEx))
        except ValueError:
            return 0
  
    elif (col['SERVICE_LEVEL'] == 'FedEx Ground Canada') | (col['SERVICE_LEVEL'] == 'FedEx Ground Mexico'):
        try: 
            return min(col['MIN_WITH_REBATE'] + 4, col['MIN_WITH_REBATE'] / (1-Margin_on_Min_GRND_BR_FedEx))
        except ValueError:
            return 0

FedEx_Rates['CLIENT_MARGIN_MINS'] = FedEx_Rates.apply(h, axis=1)

# add ADD_MARGIN_(COL_M/DECIMAL) column
FedEx_Rates['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = 1 - FedEx_Rates['NET_CONTRACT_DISC']
FedEx_Rates.loc[FedEx_Rates['SERVICE_LEVEL'] == 'FedEx First Overnight', 'DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] = None

def i(col):   
    if (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb') | (col['WEIGHT_BREAK'] == '70lb'):
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_AIR_BR)
    elif col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return
    elif (col['SERVICE_LEVEL'] == 'FedEx Ground') | (col['SERVICE_LEVEL'] == 'FedEx Home Delivery') | (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Less than 1lb') | (col['SERVICE_LEVEL'] == 'FedEx SmartPost - Greater than 1lb'): 
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_GRND_BR)
    else:
        return col['DISCOUNT_RECIPROCAL_(BDC_PAY_ON_THE_DOLLAR)'] / (1-Margin_on_FedEx_Parcel_Freight_AIR_BR)

FedEx_Rates['ADD_MARGIN_(COL_M/DECIMAL)'] = FedEx_Rates.apply(i, axis=1) 

# add DISCOUNT_TO_CLIENT(1-COLN) column 
def j(col):   
    if (col['SERVICE_LEVEL'] == 'Standard to Canada') | (col['SERVICE_LEVEL'] == 'Standard to Mexico'):
        return max(0, 1 - col['ADD_MARGIN_(COL_M/DECIMAL)'])
    elif col['SERVICE_LEVEL'] == 'FedEx First Overnight':
        return
    else:
        return 1 - col['ADD_MARGIN_(COL_M/DECIMAL)']

FedEx_Rates['DISCOUNT_TO_CLIENT(1-COLN)'] = FedEx_Rates.apply(j, axis=1)




In [22]:
#GRND ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_barrettRates_1 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_barrettRates_1['BARRETT_RATES_1'] = tbl_barrettRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_barrettRates_1['BARRETT_RATES'] = round(tbl_barrettRates_1['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_barrettRates_1 = tbl_barrettRates_1.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_2 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_2['BARRETT_RATES_1'] = tbl_barrettRates_2['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_barrettRates_2['BARRETT_RATES'] = round(tbl_barrettRates_2['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_barrettRates_2 = tbl_barrettRates_2.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_3 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_3['BARRETT_RATES_1'] = tbl_barrettRates_3['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_barrettRates_3['BARRETT_RATES'] = round(tbl_barrettRates_3['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_barrettRates_3 = tbl_barrettRates_3.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_4 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_4['BARRETT_RATES_1'] = tbl_barrettRates_4['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_barrettRates_4['BARRETT_RATES'] = round(tbl_barrettRates_4['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_barrettRates_4 = tbl_barrettRates_4.drop('BARRETT_RATES_1', axis=1)


#GRND ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_5 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRND') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRND') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_5['BARRETT_RATES_1'] = tbl_barrettRates_5['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_barrettRates_5['BARRETT_RATES'] = round(tbl_barrettRates_5['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_barrettRates_5 = tbl_barrettRates_5.drop('BARRETT_RATES_1', axis=1)


#GRES 
#GRES ['WEIGHT_BREAK'] == '1-5lb'
tbl_barrettRates_6 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_6['BARRETT_RATES_1'] = tbl_barrettRates_6['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_barrettRates_6['BARRETT_RATES'] = round(tbl_barrettRates_6['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_barrettRates_6 = tbl_barrettRates_6.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_7 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_7['BARRETT_RATES_1'] = tbl_barrettRates_7['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_barrettRates_7['BARRETT_RATES'] = round(tbl_barrettRates_7['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_barrettRates_7 = tbl_barrettRates_7.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_8 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_8['BARRETT_RATES_1'] = tbl_barrettRates_8['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_barrettRates_8['BARRETT_RATES'] = round(tbl_barrettRates_8['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_barrettRates_8 = tbl_barrettRates_8.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_9 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_9['BARRETT_RATES_1'] = tbl_barrettRates_9['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_barrettRates_9['BARRETT_RATES'] = round(tbl_barrettRates_9['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_barrettRates_9 = tbl_barrettRates_9.drop('BARRETT_RATES_1', axis=1)


#GRES ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_10 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRES') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'GRES') & (UPS_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_10['BARRETT_RATES_1'] = tbl_barrettRates_10['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_barrettRates_10['BARRETT_RATES'] = round(tbl_barrettRates_10['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_barrettRates_10 = tbl_barrettRates_10.drop('BARRETT_RATES_1', axis=1)


#SRPT
#SRPT ['WEIGHT_BREAK'] == '1-3lb'
tbl_barrettRates_11 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '1-3lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_11['BARRETT_RATES_1'] = tbl_barrettRates_11['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_barrettRates_11['BARRETT_RATES'] = round(tbl_barrettRates_11['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_barrettRates_11 = tbl_barrettRates_11.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '4-6lb'
tbl_barrettRates_12 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 6)]
DISCOUNT_TO_CLIENT_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '4-6lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_12['BARRETT_RATES_1'] = tbl_barrettRates_12['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_barrettRates_12['BARRETT_RATES'] = round(tbl_barrettRates_12['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_barrettRates_12 = tbl_barrettRates_12.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '7-8lb'
tbl_barrettRates_13 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] >= 7) & (tbl_publishedRates['WEIGHT_LB'] <= 8)]
DISCOUNT_TO_CLIENT_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '7-8lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_13['BARRETT_RATES_1'] = tbl_barrettRates_13['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_barrettRates_13['BARRETT_RATES'] = round(tbl_barrettRates_13['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_barrettRates_13 = tbl_barrettRates_13.drop('BARRETT_RATES_1', axis=1)


#SRPT ['WEIGHT_BREAK'] == '9lb'
tbl_barrettRates_14 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT') & (tbl_publishedRates['WEIGHT_LB'] == 9)]
DISCOUNT_TO_CLIENT_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT') & (UPS_Rates['WEIGHT_BREAK'] == '9lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_14['BARRETT_RATES_1'] = tbl_barrettRates_14['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_barrettRates_14['BARRETT_RATES'] = round(tbl_barrettRates_14['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_barrettRates_14 = tbl_barrettRates_14.drop('BARRETT_RATES_1', axis=1)


#Others:
#UPS REDE, there is no discount, therefore REDE's client rate = published rate
tbl_barrettRates_15 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDE')]
tbl_barrettRates_15['BARRETT_RATES'] = tbl_barrettRates_15['PUBLISHED_RATES']


#UPS RED
tbl_barrettRates_16 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'RED')]
DISCOUNT_TO_CLIENT_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'RED'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_16['BARRETT_RATES_1'] = tbl_barrettRates_16['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_barrettRates_16['BARRETT_RATES'] = round(tbl_barrettRates_16['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_barrettRates_16 = tbl_barrettRates_16.drop('BARRETT_RATES_1', axis=1)


#UPS REDS, this does not have round up option (1/2)
tbl_barrettRates_17 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'REDS')]
DISCOUNT_TO_CLIENT_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'REDS'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_17['BARRETT_RATES_1'] = tbl_barrettRates_17['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_barrettRates_17['BARRETT_RATES'] = round(tbl_barrettRates_17['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_barrettRates_17 = tbl_barrettRates_17.drop('BARRETT_RATES_1', axis=1)


#UPS 2DAM, this does not have round up option (2/2)
tbl_barrettRates_18 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAM')]
DISCOUNT_TO_CLIENT_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == '2DAM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_18['BARRETT_RATES_1'] = tbl_barrettRates_18['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_barrettRates_18['BARRETT_RATES'] = round(tbl_barrettRates_18['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_barrettRates_18 = tbl_barrettRates_18.drop('BARRETT_RATES_1', axis=1)


#UPS BLUE
tbl_barrettRates_19 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'BLUE')]
DISCOUNT_TO_CLIENT_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'BLUE') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_19['BARRETT_RATES_1'] = tbl_barrettRates_19['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_barrettRates_19['BARRETT_RATES'] = round(tbl_barrettRates_19['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_barrettRates_19 = tbl_barrettRates_19.drop('BARRETT_RATES_1', axis=1)


#UPS ORNG
tbl_barrettRates_20 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'ORNG')]
DISCOUNT_TO_CLIENT_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'ORNG'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_20['BARRETT_RATES_1'] = tbl_barrettRates_20['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_barrettRates_20['BARRETT_RATES'] = round(tbl_barrettRates_20['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_barrettRates_20 = tbl_barrettRates_20.drop('BARRETT_RATES_1', axis=1)


#UPS SRPT<1
tbl_barrettRates_21 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SRPT<1')]
DISCOUNT_TO_CLIENT_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = UPS_Rates.loc[(UPS_Rates['SERVICE_CODE'] == 'SRPT<1'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_21['BARRETT_RATES_1'] = tbl_barrettRates_21['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_barrettRates_21['BARRETT_RATES'] = round(tbl_barrettRates_21['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_barrettRates_21 = tbl_barrettRates_21.drop('BARRETT_RATES_1', axis=1)

C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\1387918324.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_1['BARRETT_RATES_1'] = tbl_barrettRates_1['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\1387918324.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_1['BARRETT_RATES'] = round(tbl_barrettRates_1['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\138791

In [23]:
#GRNDF ['WEIGHT_BREAK'] == '1-5lb'
# slice the published rates
tbl_barrettRates_23 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]

# get DISCOUNT_TO_CLIENT(1-COLN) and CLIENT_MARGIN_MINS
DISCOUNT_TO_CLIENT_1 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_1 = round(DISCOUNT_TO_CLIENT_1,2)
else:
    DISCOUNT_TO_CLIENT_1 = DISCOUNT_TO_CLIENT_1
CLIENT_MARGIN_MINS_1 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]

# use max(published rates * (1-round(DISCOUNT_TO_CLIENT)), CLIENT_MARGIN_MINS) to get the client rate
tbl_barrettRates_23['BARRETT_RATES_1'] = tbl_barrettRates_23['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
tbl_barrettRates_23['BARRETT_RATES'] = round(tbl_barrettRates_23['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
tbl_barrettRates_23 = tbl_barrettRates_23.drop('BARRETT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_24 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_2 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_2 = round(DISCOUNT_TO_CLIENT_2,2)
else:
    DISCOUNT_TO_CLIENT_2 = DISCOUNT_TO_CLIENT_2
CLIENT_MARGIN_MINS_2 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_24['BARRETT_RATES_1'] = tbl_barrettRates_24['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_2)
tbl_barrettRates_24['BARRETT_RATES'] = round(tbl_barrettRates_24['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_2)),2)
tbl_barrettRates_24 = tbl_barrettRates_24.drop('BARRETT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_25 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_3 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_3 = round(DISCOUNT_TO_CLIENT_3,2)
else:
    DISCOUNT_TO_CLIENT_3 = DISCOUNT_TO_CLIENT_3
CLIENT_MARGIN_MINS_3 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_25['BARRETT_RATES_1'] = tbl_barrettRates_25['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_3)
tbl_barrettRates_25['BARRETT_RATES'] = round(tbl_barrettRates_25['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_3)),2)
tbl_barrettRates_25 = tbl_barrettRates_25.drop('BARRETT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_26 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_4 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_4 = round(DISCOUNT_TO_CLIENT_4,2)
else:
    DISCOUNT_TO_CLIENT_4 = DISCOUNT_TO_CLIENT_4
CLIENT_MARGIN_MINS_4 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_26['BARRETT_RATES_1'] = tbl_barrettRates_26['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_4)
tbl_barrettRates_26['BARRETT_RATES'] = round(tbl_barrettRates_26['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_4)),2)
tbl_barrettRates_26 = tbl_barrettRates_26.drop('BARRETT_RATES_1', axis=1)


#GRNDF ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_27 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'GRNDF') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_5 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_5 = round(DISCOUNT_TO_CLIENT_5,2)
else:
    DISCOUNT_TO_CLIENT_5 = DISCOUNT_TO_CLIENT_5
CLIENT_MARGIN_MINS_5 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'GRNDF') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_27['BARRETT_RATES_1'] = tbl_barrettRates_27['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_5)
tbl_barrettRates_27['BARRETT_RATES'] = round(tbl_barrettRates_27['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_5)),2)
tbl_barrettRates_27 = tbl_barrettRates_27.drop('BARRETT_RATES_1', axis=1)


#HOME 
#HOME ['WEIGHT_BREAK'] == '1-5lb'
tbl_barrettRates_28 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 5)]
DISCOUNT_TO_CLIENT_6 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_6 = round(DISCOUNT_TO_CLIENT_6,2)
else:
    DISCOUNT_TO_CLIENT_6 = DISCOUNT_TO_CLIENT_6
CLIENT_MARGIN_MINS_6 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '1-5lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_28['BARRETT_RATES_1'] = tbl_barrettRates_28['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_6)
tbl_barrettRates_28['BARRETT_RATES'] = round(tbl_barrettRates_28['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_6)),2)
tbl_barrettRates_28 = tbl_barrettRates_28.drop('BARRETT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '6-10lb'
tbl_barrettRates_29 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 10) & (tbl_publishedRates['WEIGHT_LB'] >= 6)]
DISCOUNT_TO_CLIENT_7 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_7 = round(DISCOUNT_TO_CLIENT_7,2)
else:
    DISCOUNT_TO_CLIENT_7 = DISCOUNT_TO_CLIENT_7
CLIENT_MARGIN_MINS_7 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '6-10lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_29['BARRETT_RATES_1'] = tbl_barrettRates_29['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_7)
tbl_barrettRates_29['BARRETT_RATES'] = round(tbl_barrettRates_29['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_7)),2)
tbl_barrettRates_29 = tbl_barrettRates_29.drop('BARRETT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '11-20lb'
tbl_barrettRates_30 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 20) & (tbl_publishedRates['WEIGHT_LB'] >= 11)]
DISCOUNT_TO_CLIENT_8 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_8 = round(DISCOUNT_TO_CLIENT_8,2)
else:
    DISCOUNT_TO_CLIENT_8 = DISCOUNT_TO_CLIENT_8
CLIENT_MARGIN_MINS_8 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '11-20lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_30['BARRETT_RATES_1'] = tbl_barrettRates_30['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_8)
tbl_barrettRates_30['BARRETT_RATES'] = round(tbl_barrettRates_30['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_8)),2)
tbl_barrettRates_30 = tbl_barrettRates_30.drop('BARRETT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '21-30lb'
tbl_barrettRates_31 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] <= 30) & (tbl_publishedRates['WEIGHT_LB'] >= 21)]
DISCOUNT_TO_CLIENT_9 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_9 = round(DISCOUNT_TO_CLIENT_9,2)
else:
    DISCOUNT_TO_CLIENT_9 = DISCOUNT_TO_CLIENT_9
CLIENT_MARGIN_MINS_9 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '21-30lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_31['BARRETT_RATES_1'] = tbl_barrettRates_31['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_9)
tbl_barrettRates_31['BARRETT_RATES'] = round(tbl_barrettRates_31['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_9)),2)
tbl_barrettRates_31 = tbl_barrettRates_31.drop('BARRETT_RATES_1', axis=1)


#HOME ['WEIGHT_BREAK'] == '31+lb'
tbl_barrettRates_32 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'HOME') & (tbl_publishedRates['WEIGHT_LB'] >= 31)]
DISCOUNT_TO_CLIENT_10 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_10 = round(DISCOUNT_TO_CLIENT_10,2)
else:
    DISCOUNT_TO_CLIENT_10 = DISCOUNT_TO_CLIENT_10
CLIENT_MARGIN_MINS_10 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'HOME') & (FedEx_Rates['WEIGHT_BREAK'] == '31+lb'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_32['BARRETT_RATES_1'] = tbl_barrettRates_32['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_10)
tbl_barrettRates_32['BARRETT_RATES'] = round(tbl_barrettRates_32['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_10)),2)
tbl_barrettRates_32 = tbl_barrettRates_32.drop('BARRETT_RATES_1', axis=1)


#SPST
#SPST ['WEIGHT_BREAK'] == '1-2lb'
tbl_barrettRates_33 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] <= 2)]
DISCOUNT_TO_CLIENT_11 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '1-2lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_11 = round(DISCOUNT_TO_CLIENT_11,2)
else:
    DISCOUNT_TO_CLIENT_11 = DISCOUNT_TO_CLIENT_11
CLIENT_MARGIN_MINS_11 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '1-2lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_33['BARRETT_RATES_1'] = tbl_barrettRates_33['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_11)
tbl_barrettRates_33['BARRETT_RATES'] = round(tbl_barrettRates_33['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_11)),2)
tbl_barrettRates_33 = tbl_barrettRates_33.drop('BARRETT_RATES_1', axis=1)



#SPST ['WEIGHT_BREAK'] == '3lb'
tbl_barrettRates_34 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] >= 3) & (tbl_publishedRates['WEIGHT_LB'] <= 3)]
DISCOUNT_TO_CLIENT_12 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '3 lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_12 = round(DISCOUNT_TO_CLIENT_12,2)
else:
    DISCOUNT_TO_CLIENT_12 = DISCOUNT_TO_CLIENT_12
CLIENT_MARGIN_MINS_12 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '3 lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_34['BARRETT_RATES_1'] = tbl_barrettRates_34['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_12)
tbl_barrettRates_34['BARRETT_RATES'] = round(tbl_barrettRates_34['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_12)),2)
tbl_barrettRates_34 = tbl_barrettRates_34.drop('BARRETT_RATES_1', axis=1)


#SPST ['WEIGHT_BREAK'] == '4-70lb'
tbl_barrettRates_35 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] >= 4) & (tbl_publishedRates['WEIGHT_LB'] <= 70)]
DISCOUNT_TO_CLIENT_13 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '4-70lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_13 = round(DISCOUNT_TO_CLIENT_13,2)
else:
    DISCOUNT_TO_CLIENT_13 = DISCOUNT_TO_CLIENT_13
CLIENT_MARGIN_MINS_13 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '4-70lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_35['BARRETT_RATES_1'] = tbl_barrettRates_35['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_13)
tbl_barrettRates_35['BARRETT_RATES'] = round(tbl_barrettRates_35['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_13)),2)
tbl_barrettRates_35 = tbl_barrettRates_35.drop('BARRETT_RATES_1', axis=1)


#SPST ['WEIGHT_BREAK'] == '70lb+'
tbl_barrettRates_36 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST') & (tbl_publishedRates['WEIGHT_LB'] == 70)]
DISCOUNT_TO_CLIENT_14 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '70+lbs'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_14 = round(DISCOUNT_TO_CLIENT_14,2)
else:
    DISCOUNT_TO_CLIENT_14 = DISCOUNT_TO_CLIENT_14
CLIENT_MARGIN_MINS_14 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST') & (FedEx_Rates['WEIGHT_BREAK'] == '70+lbs'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_36['BARRETT_RATES_1'] = tbl_barrettRates_36['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_14)
tbl_barrettRates_36['BARRETT_RATES'] = round(tbl_barrettRates_36['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_14)),2)
tbl_barrettRates_36 = tbl_barrettRates_36.drop('BARRETT_RATES_1', axis=1)


#Others:
#FedEx 1STO, there is no discount, therefore 1STO's client rate = published rate
tbl_barrettRates_37 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '1STO')]
tbl_barrettRates_37['BARRETT_RATES'] = tbl_barrettRates_37['PUBLISHED_RATES']*(1-BarrettDisc_1STO_FedEx)


#FedEx PROV
tbl_barrettRates_38 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'PROV')]
DISCOUNT_TO_CLIENT_16 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'PROV'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_16 = round(DISCOUNT_TO_CLIENT_16,2)
else:
    DISCOUNT_TO_CLIENT_16 = DISCOUNT_TO_CLIENT_16
CLIENT_MARGIN_MINS_16 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'PROV'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_38['BARRETT_RATES_1'] = tbl_barrettRates_38['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_16)
tbl_barrettRates_38['BARRETT_RATES'] = round(tbl_barrettRates_38['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_16)),2)
tbl_barrettRates_38 = tbl_barrettRates_38.drop('BARRETT_RATES_1', axis=1)


#FedEx STOV, this does not have round up option (1/2)
tbl_barrettRates_39 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'STOV')]
DISCOUNT_TO_CLIENT_17 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'STOV'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_17 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'STOV'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_39['BARRETT_RATES_1'] = tbl_barrettRates_39['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_17)
tbl_barrettRates_39['BARRETT_RATES'] = round(tbl_barrettRates_39['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_17)),2)
tbl_barrettRates_39 = tbl_barrettRates_39.drop('BARRETT_RATES_1', axis=1)


#FedEx 2AM, this does not have round up option (2/2)
tbl_barrettRates_40 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2AM')]
DISCOUNT_TO_CLIENT_18 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2AM'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
CLIENT_MARGIN_MINS_18 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2AM'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_40['BARRETT_RATES_1'] = tbl_barrettRates_40['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_18)
tbl_barrettRates_40['BARRETT_RATES'] = round(tbl_barrettRates_40['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_18)),2)
tbl_barrettRates_40 = tbl_barrettRates_40.drop('BARRETT_RATES_1', axis=1)


#FedEx 2DAY
tbl_barrettRates_41 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == '2DAY')]
DISCOUNT_TO_CLIENT_19 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2DAY'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_19 = round(DISCOUNT_TO_CLIENT_19,2)
else:
    DISCOUNT_TO_CLIENT_19 = DISCOUNT_TO_CLIENT_19
CLIENT_MARGIN_MINS_19 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == '2DAY') , 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_41['BARRETT_RATES_1'] = tbl_barrettRates_41['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_19)
tbl_barrettRates_41['BARRETT_RATES'] = round(tbl_barrettRates_41['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_19)),2)
tbl_barrettRates_41 = tbl_barrettRates_41.drop('BARRETT_RATES_1', axis=1)


#FedEx EXSA
tbl_barrettRates_42 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'EXSA')]
DISCOUNT_TO_CLIENT_20 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'EXSA'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_20 = round(DISCOUNT_TO_CLIENT_20,2)
else:
    DISCOUNT_TO_CLIENT_20 = DISCOUNT_TO_CLIENT_20
CLIENT_MARGIN_MINS_20 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'EXSA'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_42['BARRETT_RATES_1'] = tbl_barrettRates_42['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_20)
tbl_barrettRates_42['BARRETT_RATES'] = round(tbl_barrettRates_42['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_20)),2)
tbl_barrettRates_42 = tbl_barrettRates_42.drop('BARRETT_RATES_1', axis=1)


#FedEx SPST<1 1-3oz
tbl_barrettRates_43 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST<1') & (tbl_publishedRates['WEIGHT_OZ'] <= 3)]
DISCOUNT_TO_CLIENT_21 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '1-3oz'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_21 = round(DISCOUNT_TO_CLIENT_21,2)
else:
    DISCOUNT_TO_CLIENT_21 = DISCOUNT_TO_CLIENT_21
CLIENT_MARGIN_MINS_21 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '1-3oz'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_43['BARRETT_RATES_1'] = tbl_barrettRates_43['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_21)
tbl_barrettRates_43['BARRETT_RATES'] = round(tbl_barrettRates_43['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_21)),2)
tbl_barrettRates_43 = tbl_barrettRates_43.drop('BARRETT_RATES_1', axis=1)


#FedEx SPST<1 4-16oz
tbl_barrettRates_44 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'SPST<1') & (tbl_publishedRates['WEIGHT_OZ'] <= 16) & (tbl_publishedRates['WEIGHT_OZ'] > 3)]
DISCOUNT_TO_CLIENT_22 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '4-16oz'), 'DISCOUNT_TO_CLIENT(1-COLN)'].values[0]
if Round_Discount == 'YES':
    DISCOUNT_TO_CLIENT_22 = round(DISCOUNT_TO_CLIENT_22,2)
else:
    DISCOUNT_TO_CLIENT_22 = DISCOUNT_TO_CLIENT_22
CLIENT_MARGIN_MINS_22 = FedEx_Rates.loc[(FedEx_Rates['SERVICE_CODE'] == 'SPST<1') & (FedEx_Rates['WEIGHT_BREAK'] == '4-16oz'), 'CLIENT_MARGIN_MINS'].values[0]
tbl_barrettRates_44['BARRETT_RATES_1'] = tbl_barrettRates_44['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_22)
tbl_barrettRates_44['BARRETT_RATES'] = round(tbl_barrettRates_44['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_22)),2)
tbl_barrettRates_44 = tbl_barrettRates_44.drop('BARRETT_RATES_1', axis=1)




C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\3352589481.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_23['BARRETT_RATES_1'] = tbl_barrettRates_23['PUBLISHED_RATES']*(1-DISCOUNT_TO_CLIENT_1)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\3352589481.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tbl_barrettRates_23['BARRETT_RATES'] = round(tbl_barrettRates_23['BARRETT_RATES_1'].apply(lambda x: max(x, CLIENT_MARGIN_MINS_1)),2)
C:\Users\SLiu\AppData\Local\Temp\ipykernel_7228\33

In [24]:
#Get dhl, usps, and mi rates
tbl_barrettRates_22 = tbl_publishedRates[(tbl_publishedRates['SERVICE_CODE'] == 'USPSAG') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAG_CI') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'USPSAP_CI') | 
                                         (tbl_publishedRates['SERVICE_CODE'] == 'MIPLE') |
                                         (tbl_publishedRates['SERVICE_CODE'] == 'MIPH') |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG_'))   |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLG<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE_'))   |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLE<1_')) |
                                         (tbl_publishedRates['SERVICE_CODE'].str.startswith('DHLEM_'))]


#combine them to rename as BarrettRates
tbl_BarrettRates = pd.concat([tbl_barrettRates_1, tbl_barrettRates_2, tbl_barrettRates_3, tbl_barrettRates_4, 
                    tbl_barrettRates_5, tbl_barrettRates_6, tbl_barrettRates_7, tbl_barrettRates_8,
                    tbl_barrettRates_9, tbl_barrettRates_10, tbl_barrettRates_11, tbl_barrettRates_12,
                    tbl_barrettRates_13, tbl_barrettRates_14, tbl_barrettRates_15, tbl_barrettRates_16,
                    tbl_barrettRates_17, tbl_barrettRates_18, tbl_barrettRates_19, tbl_barrettRates_20,
                    tbl_barrettRates_21, tbl_barrettRates_22, tbl_barrettRates_23, tbl_barrettRates_24,
                    tbl_barrettRates_25, tbl_barrettRates_26, tbl_barrettRates_27, tbl_barrettRates_28,
                    tbl_barrettRates_29, tbl_barrettRates_30, tbl_barrettRates_31, tbl_barrettRates_32,
                    tbl_barrettRates_33, tbl_barrettRates_34, tbl_barrettRates_35, tbl_barrettRates_36,
                    tbl_barrettRates_37, tbl_barrettRates_38, tbl_barrettRates_39, tbl_barrettRates_40,
                    tbl_barrettRates_41, tbl_barrettRates_42, tbl_barrettRates_43, tbl_barrettRates_44])
#Keep this table as it only contains published rates and Barrett rates
#tbl_BarrettRates.head()

In [25]:

# Define the conditions
condition1 = (tbl_BarrettRates['SERVICE_CODE'] == 'SPST') & (tbl_BarrettRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_BarrettRates['WEIGHT_LB'] <= 2)
condition2 = (tbl_BarrettRates['SERVICE_CODE'] == 'SPST') & (tbl_BarrettRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_BarrettRates['WEIGHT_LB'] > 2) & (tbl_BarrettRates['WEIGHT_LB'] < 4)
condition3 = (tbl_BarrettRates['SERVICE_CODE'] == 'SPST') & (tbl_BarrettRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_BarrettRates['WEIGHT_LB'] >= 4) & (tbl_BarrettRates['WEIGHT_LB'] < 70)
condition4 = (tbl_BarrettRates['SERVICE_CODE'] == 'SPST') & (tbl_BarrettRates['ZONE'].isin(['17', '9', '10', '26', '99'])) & (tbl_BarrettRates['WEIGHT_LB'] >= 70)

# Overwrite values where the condition is met
tbl_BarrettRates.loc[condition1, 'BARRETT_RATES'] = tbl_BarrettRates['PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - 0.81)))
tbl_BarrettRates.loc[condition2, 'BARRETT_RATES'] = tbl_BarrettRates['PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - 0.58)))
tbl_BarrettRates.loc[condition3, 'BARRETT_RATES'] = tbl_BarrettRates['PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - 0.61)))
tbl_BarrettRates.loc[condition4, 'BARRETT_RATES'] = tbl_BarrettRates['PUBLISHED_RATES'].apply(lambda x: max(18.28, x * (1 - 0.61)))



In [26]:

# Define two different mappings
mapping_for_ground = {0.1: 'Tier1', 0.2: 'Tier2', 0.3: 'Tier3', 0.4: 'Tier4', 0.5: 'Tier5', 0.6: 'Tier6', 0.7: 'Tier7', 
                      0.8: 'Tier8', 0.9: 'Tier9', 1: 'Tier10'}

mapping_for_priority = {0.1: 'Tier1', 0.2: 'Tier2', 0.3: 'Tier3', 0.4: 'Tier4', 0.5: 'Tier5'}

def apply_custom_mapping(row):
    if row['SERVICE_CODE'] == 'USPSAG_CI':
        return mapping_for_ground.get(row['CUBIC_FT'], None)  # None for default
    elif row['SERVICE_CODE'] == 'USPSAP_CI':
        return mapping_for_priority.get(row['CUBIC_FT'], None)  # None for default
    else:
        return None  # Or some default value for other ServiceCodes

# Apply the custom function to create a new column
tbl_publishedRates['TIER'] = tbl_publishedRates.apply(apply_custom_mapping, axis=1)
tbl_BarrettRates['TIER'] = tbl_BarrettRates.apply(apply_custom_mapping, axis=1)



In [27]:
tbl_BarrettRates.to_csv('tbl_BarrettRates.csv')

In [28]:
tbl_publishedRates.to_csv('tbl_publishedRates.csv')

In [29]:
# Get Zone from facility doc
# When the distribution is all over the U.S., we use TN Facility. Needs business input. Make sure the facility matches 
# between dhl rates and Zone finder.
xls = pd.ExcelFile('zipOutbound.xlsx')
# Load Outbound card
OB_MA = pd.read_excel(xls, '02038_Outbound') # 5114358_dhl_sm_parcel_Franklin
OB_NJ = pd.read_excel(xls, '08873_Outbound') # 5122444_dhl_sm_parcel_Somerset
OB_MD = pd.read_excel(xls, '21226_Outbound') # 5123283_dhl_sm_parcel_Curtis Bay, GBM
OB_TN = pd.read_excel(xls, '38141_Outbound') # 5122893_dhl_sm_parcell Memphis
OB_MS = pd.read_excel(xls, '38654_Outbound') # 5122723_dhl_sm_parcel_8150 Nail Olive Branch, MS3/5122739_dhl_sm_parcel_Nail rd Olive branch, MS2
OB_TX = pd.read_excel(xls, '75041_Outbound') # 5122855_dhl_sm_parcel_Garland TX
OB_CA = pd.read_excel(xls, '90640_Outbound') # none

OB_02324 = pd.read_excel(xls, '02324_Outbound')
OB_06096 = pd.read_excel(xls, '06096_Outbound')
OB_18105 = pd.read_excel(xls, '18105_Outbound')
OB_21240 = pd.read_excel(xls, '21240_Outbound')
OB_43004 = pd.read_excel(xls, '43004_Outbound')


OB_MA_1 = OB_MA.copy()
OB_MA_1 = OB_MA_1.drop('Origin', axis=1)
OB_MA_1['Origin'] = '2048'

OB_STATE_UPS = pd.concat([OB_MA, OB_NJ, OB_MD, OB_TN, OB_MS, OB_TX, OB_CA, OB_MA_1, OB_02324, OB_06096, OB_18105, OB_21240, OB_43004])

# n/a - 5122890_dhl_sm_parcel_Oofos, Oofos is (or will be in Byhalia, MS)
# n/a - 5123280_dhl_sm_parcel_MAN, 02048
# n/a - 5123282_dhl_sm_parcel Bridgewater, MA5
# 90640 is Montebello, CA (I don't see that here)   

In [30]:
OB_STATE_UPS.to_csv('OB_STATE_UPS.csv')

In [31]:
# Get Zone from facility doc
# When the distribution is all over the U.S., we use TN Facility. Needs business input. Make sure the facility matches 
# between dhl rates and Zone finder.
xls = pd.ExcelFile('6-2.ZipToZoneFedex.xlsx')

# Load Outbound card


OB_18101_FXG = pd.read_excel(xls, 'TDR29112023_18101_FXG') 
OB_18101_FHD = pd.read_excel(xls, 'TDR29112023_18101_FHD')

OB_90640_FXG = pd.read_excel(xls, 'TDR14122023_90640_FXG') 
OB_90640_FHD = pd.read_excel(xls, 'TDR14122023_90640_FHD') 

OB_75041_FXG = pd.read_excel(xls, 'TDR14122023_75041_FXG') # 5123283_dhl_sm_parcel_Curtis Bay, GBM
OB_75041_FHD = pd.read_excel(xls, 'TDR14122023_75041_FHD') # 5123283_dhl_sm_parcel_Curtis Bay, GBM

OB_38654_FXG = pd.read_excel(xls, 'TDR14122023_38654_FXG') 
OB_38654_FHD = pd.read_excel(xls, 'TDR14122023_38654_FHD') 

OB_38141_FXG = pd.read_excel(xls, 'TDR14122023_38141_FXG') # 5122893_dhl_sm_parcell Memphis
OB_38141_FHD = pd.read_excel(xls, 'TDR14122023_38141_FHD') # 5122893_dhl_sm_parcell Memphis

OB_21226_FXG = pd.read_excel(xls, 'TDR14122023_21226_FXG') # 5122855_dhl_sm_parcel_Garland TX
OB_21226_FHD = pd.read_excel(xls, 'TDR14122023_21226_FHD') # 5122855_dhl_sm_parcel_Garland TX

OB_08873_FXG = pd.read_excel(xls, 'TDR14122023_08873_FXG') # 5122444_dhl_sm_parcel_Somerset
OB_08873_FHD = pd.read_excel(xls, 'TDR14122023_08873_FHD') # 5122444_dhl_sm_parcel_Somerset

OB_02038_FXG = pd.read_excel(xls, 'TDR14122023_02038_FXG') # 5114358_dhl_sm_parcel_Franklin
OB_02038_FHD = pd.read_excel(xls, 'TDR14122023_02038_FHD') # 5114358_dhl_sm_parcel_Franklin

# 5122723_dhl_sm_parcel_8150 Nail Olive Branch, MS3/5122739_dhl_sm_parcel_Nail rd Olive branch, MS2
# 5122723_dhl_sm_parcel_8150 Nail Olive Branch, MS3/5122739_dhl_sm_parcel_Nail rd Olive branch, MS2

OB_18101_FXG['FileID'] = '18101_FXG'
OB_18101_FHD['FileID'] = '18101_FHD'
OB_90640_FXG['FileID'] = '90640_FXG'
OB_90640_FHD['FileID'] = '90640_FHD'
OB_75041_FXG['FileID'] = '75041_FXG'
OB_75041_FHD['FileID'] = '75041_FHD'
OB_38654_FXG['FileID'] = '38654_FXG'
OB_38654_FHD['FileID'] = '38654_FHD'
OB_38141_FXG['FileID'] = '38141_FXG'
OB_38141_FHD['FileID'] = '38141_FHD'
OB_21226_FXG['FileID'] = '21226_FXG'
OB_21226_FHD['FileID'] = '21226_FHD'
OB_08873_FXG['FileID'] = '08873_FXG'
OB_08873_FHD['FileID'] = '08873_FHD'
OB_02038_FXG['FileID'] = '02038_FXG'
OB_02038_FHD['FileID'] = '02038_FHD'

OB_STATE_FedEx = pd.concat([OB_18101_FXG, OB_18101_FHD, OB_90640_FXG, OB_90640_FHD, OB_75041_FXG, OB_75041_FHD, 
                      OB_38654_FXG, OB_38654_FHD, OB_38141_FXG, OB_38141_FHD, OB_21226_FXG, OB_21226_FHD, 
                      OB_08873_FXG, OB_08873_FHD, OB_02038_FXG, OB_02038_FHD])



In [32]:
OB_STATE_FedEx.to_csv('OB_STATE_FedEx.csv')